In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing


# Предсказание сердечно-сосудистых заболеваний

Рассмотрим уже известный нам датасет и попробуем использовать деревья решений и случайный лес для предсказания наличия у пациента ССЗ. Этот подход вполне оправдан, так как мы имеем большое количество категориальных признаков, с которыми деревья хорошо работают. Затем мы сравним полученные результаты с результатами метода $kNN$.

In [ ]:
# Загружаем данные
df = pd.read_csv('../input/cardio_train.csv', sep=';')
df.head()

## Небольшая предобработка данных

1. Преобразуем признак **age**: переведём дни в годы, разделив число дней на 365.25 и округлив вниз до ближайшего целого (функция np.floor()).
2. Построим три бинарных признака на основе признака **cholesterol**, используя one hot encodeing (OHE) или dummy-кодирование.
3. Аналогично построим три бинарных признака на основе **gluc**.
4. Удалим исходные признаки **cholesterol** и **gluc**.

In [ ]:
# Предобработка
df['age'] = np.floor(df['age'] / 365.25)
df1=pd.get_dummies(df, columns=['cholesterol','gluc'])
df1.head()

Разделим выборку на обучающую и валидационную (тестовую) в соотношении 7:3.

In [ ]:
from sklearn.model_selection import train_test_split
X = df1.drop(['id', 'cardio'], axis=1)
y = df1['cardio']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.3, random_state=2019) 

## Обучение дерева решений и настройка гиперпараметров

1. Обучим дерево решений с максимальной глубиной 3 и параметром **random_state=2019**.
2. Изобразим полученное дерево графически.
3. Сделаем предсказание с помощью полученного дерева на валидационном множестве. Вычислим метрику **accuracy** на нём.
4. Произведём подбор гиперпараметров **max_depth** и **min_samples_leaf** с помощью GridSearchCV.
5. Построим графики зависимости средней доли правильных ответов (**mean_test_score**) от значений каждого из гиперпараметров.
6. Изобразим графически полученное "оптимальное" дерево.

In [ ]:
# Обучение дерева решений
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=3, random_state=2019)
tree.fit(X_train, y_train)

In [ ]:
# Визуализация
from sklearn.tree import export_graphviz

export_graphviz(tree, out_file='tree.dot')
print(open('tree.dot').read()) 
# Далее скопировать полученный текст на сайт https://dreampuf.github.io/GraphvizOnline/ и сгенерировать граф
# Вставить картинку в блокнот: ![](ссылка)

![](data:image/svg+xml;charset=utf-8,%3C%3Fxml%20version%3D%221.0%22%20encoding%3D%22UTF-8%22%20standalone%3D%22no%22%3F%3E%3C!DOCTYPE%20svg%20PUBLIC%20%22-%2F%2FW3C%2F%2FDTD%20SVG%201.1%2F%2FEN%22%20%22http%3A%2F%2Fwww.w3.org%2FGraphics%2FSVG%2F1.1%2FDTD%2Fsvg11.dtd%22%3E%3C!--%20Generated%20by%20graphviz%20version%202.40.1%20(20161225.0304)%0A%20--%3E%3C!--%20Title%3A%20Tree%20Pages%3A%201%20--%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20xmlns%3Axlink%3D%22http%3A%2F%2Fwww.w3.org%2F1999%2Fxlink%22%20width%3D%221163pt%22%20height%3D%22400pt%22%20viewBox%3D%220.00%200.00%201162.54%20400.00%22%3E%0A%3Cg%20id%3D%22graph0%22%20class%3D%22graph%22%20transform%3D%22scale(1%201)%20rotate(0)%20translate(4%20396)%22%3E%0A%3Ctitle%3ETree%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22%23ffffff%22%20stroke%3D%22transparent%22%20points%3D%22-4%2C4%20-4%2C-396%201158.5356%2C-396%201158.5356%2C4%20-4%2C4%22%2F%3E%0A%3C!--%200%20--%3E%0A%3Cg%20id%3D%22node1%22%20class%3D%22node%22%3E%0A%3Ctitle%3E0%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22646.8041%2C-392.1003%20499.7315%2C-392.1003%20499.7315%2C-316.6997%20646.8041%2C-316.6997%20646.8041%2C-392.1003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22573.2678%22%20y%3D%22-375.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B4%5D%20%26lt%3B%3D%20129.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22573.2678%22%20y%3D%22-358.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22573.2678%22%20y%3D%22-341.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2049000%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22573.2678%22%20y%3D%22-325%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B24521%2C%2024479%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%201%20--%3E%0A%3Cg%20id%3D%22node2%22%20class%3D%22node%22%3E%0A%3Ctitle%3E1%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22503.3042%2C-280.9003%20363.2314%2C-280.9003%20363.2314%2C-205.4997%20503.3042%2C-205.4997%20503.3042%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22433.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2054.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22433.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.435%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22433.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2028945%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22433.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B19707%2C%209238%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%200%26%2345%3B%26gt%3B1%20--%3E%0A%3Cg%20id%3D%22edge1%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E0-%26gt%3B1%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M525.8702%2C-316.7528C513.8969%2C-307.2425%20500.9352%2C-296.9473%20488.6085%2C-287.1564%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22490.624%2C-284.2875%20480.6166%2C-280.8085%20486.2702%2C-289.7688%20490.624%2C-284.2875%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22483.4642%22%20y%3D%22-301.4458%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3ETrue%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%208%20--%3E%0A%3Cg%20id%3D%22node9%22%20class%3D%22node%22%3E%0A%3Ctitle%3E8%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22790.3042%2C-280.9003%20650.2314%2C-280.9003%20650.2314%2C-205.4997%20790.3042%2C-205.4997%20790.3042%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22720.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B4%5D%20%26lt%3B%3D%20138.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22720.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.365%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22720.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2020055%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22720.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B4814%2C%2015241%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%200%26%2345%3B%26gt%3B8%20--%3E%0A%3Cg%20id%3D%22edge8%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E0-%26gt%3B8%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M623.0353%2C-316.7528C635.727%2C-307.1519%20649.4763%2C-296.7511%20662.5296%2C-286.8768%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22664.6879%2C-289.6328%20670.5516%2C-280.8085%20660.4648%2C-284.0501%20664.6879%2C-289.6328%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22667.1181%22%20y%3D%22-301.3716%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EFalse%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%202%20--%3E%0A%3Cg%20id%3D%22node3%22%20class%3D%22node%22%3E%0A%3Ctitle%3E2%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22288.3042%2C-169.7003%20148.2314%2C-169.7003%20148.2314%2C-94.2997%20288.3042%2C-94.2997%20288.3042%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22218.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B6%5D%20%26lt%3B%3D%202.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22218.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.364%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22218.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2017925%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22218.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B13643%2C%204282%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%201%26%2345%3B%26gt%3B2%20--%3E%0A%3Cg%20id%3D%22edge2%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E1-%26gt%3B2%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M363.1663%2C-206.9428C342.2576%2C-196.1287%20319.1927%2C-184.1993%20297.7307%2C-173.099%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22299.1321%2C-169.8834%20288.6418%2C-168.3981%20295.9162%2C-176.101%20299.1321%2C-169.8834%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%205%20--%3E%0A%3Cg%20id%3D%22node6%22%20class%3D%22node%22%3E%0A%3Ctitle%3E5%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22499.8042%2C-169.7003%20366.7314%2C-169.7003%20366.7314%2C-94.2997%20499.8042%2C-94.2997%20499.8042%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22433.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B6%5D%20%26lt%3B%3D%202.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22433.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.495%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22433.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2011020%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22433.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B6064%2C%204956%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%201%26%2345%3B%26gt%3B5%20--%3E%0A%3Cg%20id%3D%22edge5%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E1-%26gt%3B5%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M433.2678%2C-205.5528C433.2678%2C-197.22%20433.2678%2C-188.2845%20433.2678%2C-179.6137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22436.7679%2C-179.6084%20433.2678%2C-169.6085%20429.7679%2C-179.6085%20436.7679%2C-179.6084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%203%20--%3E%0A%3Cg%20id%3D%22node4%22%20class%3D%22node%22%3E%0A%3Ctitle%3E3%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22140.3042%2C-58.6014%20.2314%2C-58.6014%20.2314%2C.2014%20140.3042%2C.2014%20140.3042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%2270.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.348%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%2270.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2017161%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%2270.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B13316%2C%203845%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%202%26%2345%3B%26gt%3B3%20--%3E%0A%3Cg%20id%3D%22edge3%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E2-%26gt%3B3%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M163.9985%2C-94.3048C149.9248%2C-84.5293%20134.8239%2C-74.0403%20120.9428%2C-64.3986%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22122.6873%2C-61.3489%20112.4775%2C-58.5186%20118.694%2C-67.0981%20122.6873%2C-61.3489%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%204%20--%3E%0A%3Cg%20id%3D%22node5%22%20class%3D%22node%22%3E%0A%3Ctitle%3E4%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22277.8043%2C-58.6014%20158.7313%2C-58.6014%20158.7313%2C.2014%20277.8043%2C.2014%20277.8043%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22218.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.49%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22218.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20764%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22218.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B327%2C%20437%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%202%26%2345%3B%26gt%3B4%20--%3E%0A%3Cg%20id%3D%22edge4%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E2-%26gt%3B4%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M218.2678%2C-94.3048C218.2678%2C-85.9126%20218.2678%2C-76.9946%20218.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22221.7679%2C-68.5186%20218.2678%2C-58.5186%20214.7679%2C-68.5187%20221.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%206%20--%3E%0A%3Cg%20id%3D%22node7%22%20class%3D%22node%22%3E%0A%3Ctitle%3E6%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22429.8042%2C-58.6014%20296.7314%2C-58.6014%20296.7314%2C.2014%20429.8042%2C.2014%20429.8042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22363.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.487%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22363.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%209877%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22363.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B5728%2C%204149%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%205%26%2345%3B%26gt%3B6%20--%3E%0A%3Cg%20id%3D%22edge6%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E5-%26gt%3B6%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M407.5999%2C-94.3048C401.5086%2C-85.3593%20395.0104%2C-75.8163%20388.9198%2C-66.8718%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22391.7533%2C-64.8144%20383.2318%2C-58.5186%20385.9673%2C-68.7543%20391.7533%2C-64.8144%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%207%20--%3E%0A%3Cg%20id%3D%22node8%22%20class%3D%22node%22%3E%0A%3Ctitle%3E7%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22567.8043%2C-58.6014%20448.7313%2C-58.6014%20448.7313%2C.2014%20567.8043%2C.2014%20567.8043%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22508.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.415%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22508.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201143%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22508.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B336%2C%20807%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%205%26%2345%3B%26gt%3B7%20--%3E%0A%3Cg%20id%3D%22edge7%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E5-%26gt%3B7%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M460.7692%2C-94.3048C467.2956%2C-85.3593%20474.2578%2C-75.8163%20480.7835%2C-66.8718%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22483.8113%2C-68.66%20486.8777%2C-58.5186%20478.1564%2C-64.5343%20483.8113%2C-68.66%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%209%20--%3E%0A%3Cg%20id%3D%22node10%22%20class%3D%22node%22%3E%0A%3Ctitle%3E9%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22786.8042%2C-169.7003%20653.7314%2C-169.7003%20653.7314%2C-94.2997%20786.8042%2C-94.2997%20786.8042%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22720.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B6%5D%20%26lt%3B%3D%202.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22720.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.481%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22720.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%206533%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22720.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B2636%2C%203897%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%208%26%2345%3B%26gt%3B9%20--%3E%0A%3Cg%20id%3D%22edge9%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E8-%26gt%3B9%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M720.2678%2C-205.5528C720.2678%2C-197.22%20720.2678%2C-188.2845%20720.2678%2C-179.6137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22723.7679%2C-179.6084%20720.2678%2C-169.6085%20716.7679%2C-179.6085%20723.7679%2C-179.6084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2012%20--%3E%0A%3Cg%20id%3D%22node13%22%20class%3D%22node%22%3E%0A%3Ctitle%3E12%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221013.3042%2C-169.7003%20873.2314%2C-169.7003%20873.2314%2C-94.2997%201013.3042%2C-94.2997%201013.3042%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22943.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B4%5D%20%26lt%3B%3D%20149.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22943.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.27%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22943.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2013522%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22943.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B2178%2C%2011344%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%208%26%2345%3B%26gt%3B12%20--%3E%0A%3Cg%20id%3D%22edge12%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E8-%26gt%3B12%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M790.5203%2C-208.1683C813.8405%2C-196.5395%20839.963%2C-183.5134%20863.92%2C-171.5671%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22865.5982%2C-174.6414%20872.9854%2C-167.0466%20862.4744%2C-168.377%20865.5982%2C-174.6414%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2010%20--%3E%0A%3Cg%20id%3D%22node11%22%20class%3D%22node%22%3E%0A%3Ctitle%3E10%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22719.8042%2C-58.6014%20586.7314%2C-58.6014%20586.7314%2C.2014%20719.8042%2C.2014%20719.8042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22653.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.495%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22653.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%205294%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22653.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B2379%2C%202915%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%209%26%2345%3B%26gt%3B10%20--%3E%0A%3Cg%20id%3D%22edge10%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E9-%26gt%3B10%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M695.6999%2C-94.3048C689.9298%2C-85.4515%20683.7782%2C-76.013%20678.0009%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22680.7687%2C-64.9853%20672.3762%2C-58.5186%20674.9043%2C-68.8075%20680.7687%2C-64.9853%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2011%20--%3E%0A%3Cg%20id%3D%22node12%22%20class%3D%22node%22%3E%0A%3Ctitle%3E11%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22857.8043%2C-58.6014%20738.7313%2C-58.6014%20738.7313%2C.2014%20857.8043%2C.2014%20857.8043%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22798.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.329%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22798.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201239%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22798.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B257%2C%20982%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%209%26%2345%3B%26gt%3B11%20--%3E%0A%3Cg%20id%3D%22edge11%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E9-%26gt%3B11%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M748.8692%2C-94.3048C755.7266%2C-85.2671%20763.0468%2C-75.6195%20769.8939%2C-66.5954%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22772.7658%2C-68.6007%20776.0221%2C-58.5186%20767.1893%2C-64.3694%20772.7658%2C-68.6007%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2013%20--%3E%0A%3Cg%20id%3D%22node14%22%20class%3D%22node%22%3E%0A%3Ctitle%3E13%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221009.8042%2C-58.6014%20876.7314%2C-58.6014%20876.7314%2C.2014%201009.8042%2C.2014%201009.8042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22943.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.302%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22943.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%206849%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22943.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1269%2C%205580%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2012%26%2345%3B%26gt%3B13%20--%3E%0A%3Cg%20id%3D%22edge13%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E12-%26gt%3B13%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M943.2678%2C-94.3048C943.2678%2C-85.9126%20943.2678%2C-76.9946%20943.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22946.7679%2C-68.5186%20943.2678%2C-58.5186%20939.7679%2C-68.5187%20946.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2014%20--%3E%0A%3Cg%20id%3D%22node15%22%20class%3D%22node%22%3E%0A%3Ctitle%3E14%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221154.3042%2C-58.6014%201028.2314%2C-58.6014%201028.2314%2C.2014%201154.3042%2C.2014%201154.3042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221091.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.235%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221091.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%206673%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221091.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B909%2C%205764%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2012%26%2345%3B%26gt%3B14%20--%3E%0A%3Cg%20id%3D%22edge14%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E12-%26gt%3B14%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M997.5371%2C-94.3048C1011.6108%2C-84.5293%201026.7117%2C-74.0403%201040.5928%2C-64.3986%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221042.8416%2C-67.0981%201049.0581%2C-58.5186%201038.8483%2C-61.3489%201042.8416%2C-67.0981%22%2F%3E%0A%3C%2Fg%3E%0A%3C%2Fg%3E%0A%3C%2Fsvg%3E)

In [ ]:
X_train.head()

In [ ]:
# Предсказания для валидационного множества
from sklearn.metrics import accuracy_score

y_pred = tree.predict(X_valid)
accuracy_score(y_valid, y_pred)

In [ ]:
# Кросс-валидация и подбор гиперпараметров
from sklearn.model_selection import GridSearchCV

tree_params = {'max_depth': np.arange(2, 11)}

tree_grid = GridSearchCV(tree, tree_params, cv=5, scoring='accuracy') # кросс-валидация по 5 блокам
tree_grid.fit(X_train, y_train)

In [ ]:
tree_params1 = {'min_samples_leaf': np.arange(2, 11)}

tree_grid1 = GridSearchCV(tree, tree_params1, cv=5, scoring='accuracy') # кросс-валидация по 5 блокам
tree_grid1.fit(X_train, y_train)

In [ ]:
# Отрисовка графиков
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True) # 2 графика рядом с одинаковым масштабом по оси Оу

ax[0].plot(tree_params['max_depth'], tree_grid.cv_results_['mean_test_score']) # accuracy vs max_depth
ax[0].set_xlabel('max_depth')
ax[0].set_ylabel('Mean accuracy on test set')

ax[1].plot(tree_params1['min_samples_leaf'], tree_grid1.cv_results_['mean_test_score']) # accuracy vs min_samples_leaf
ax[1].set_xlabel('min_samples_leaf')
ax[1].set_ylabel('Mean accuracy on test set')

In [ ]:
# Выбор и отрисовка наилучшего дерева

pd.DataFrame(tree_grid.cv_results_).head().T

best_tree = tree_grid.best_estimator_
y_pred = best_tree.predict(X_valid)
accuracy_score(y_valid, y_pred)

export_graphviz(best_tree, out_file='best_tree.dot')
print(open('best_tree.dot').read()) 
# Далее скопировать полученный текст на сайт https://dreampuf.github.io/GraphvizOnline/ и сгенерировать граф
# Вставить картинку в блокнот: ![](ссылка)

![](data:image/svg+xml;charset=utf-8,%3C%3Fxml%20version%3D%221.0%22%20encoding%3D%22UTF-8%22%20standalone%3D%22no%22%3F%3E%3C!DOCTYPE%20svg%20PUBLIC%20%22-%2F%2FW3C%2F%2FDTD%20SVG%201.1%2F%2FEN%22%20%22http%3A%2F%2Fwww.w3.org%2FGraphics%2FSVG%2F1.1%2FDTD%2Fsvg11.dtd%22%3E%3C!--%20Generated%20by%20graphviz%20version%202.40.1%20(20161225.0304)%0A%20--%3E%3C!--%20Title%3A%20Tree%20Pages%3A%201%20--%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20xmlns%3Axlink%3D%22http%3A%2F%2Fwww.w3.org%2F1999%2Fxlink%22%20width%3D%228032pt%22%20height%3D%22734pt%22%20viewBox%3D%220.00%200.00%208031.54%20733.60%22%3E%0A%3Cg%20id%3D%22graph0%22%20class%3D%22graph%22%20transform%3D%22scale(1%201)%20rotate(0)%20translate(4%20729.6)%22%3E%0A%3Ctitle%3ETree%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22%23ffffff%22%20stroke%3D%22transparent%22%20points%3D%22-4%2C4%20-4%2C-729.6%208027.5356%2C-729.6%208027.5356%2C4%20-4%2C4%22%2F%3E%0A%3C!--%200%20--%3E%0A%3Cg%20id%3D%22node1%22%20class%3D%22node%22%3E%0A%3Ctitle%3E0%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224186.8041%2C-725.7003%204039.7315%2C-725.7003%204039.7315%2C-650.2997%204186.8041%2C-650.2997%204186.8041%2C-725.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224113.2678%22%20y%3D%22-709%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B4%5D%20%26lt%3B%3D%20129.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224113.2678%22%20y%3D%22-692.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224113.2678%22%20y%3D%22-675.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2049000%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224113.2678%22%20y%3D%22-658.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B24521%2C%2024479%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%201%20--%3E%0A%3Cg%20id%3D%22node2%22%20class%3D%22node%22%3E%0A%3Ctitle%3E1%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223256.3042%2C-614.5003%203116.2314%2C-614.5003%203116.2314%2C-539.0997%203256.3042%2C-539.0997%203256.3042%2C-614.5003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223186.2678%22%20y%3D%22-597.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2054.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223186.2678%22%20y%3D%22-581%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.435%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223186.2678%22%20y%3D%22-564.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2028945%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223186.2678%22%20y%3D%22-547.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B19707%2C%209238%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%200%26%2345%3B%26gt%3B1%20--%3E%0A%3Cg%20id%3D%22edge1%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E0-%26gt%3B1%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4039.3641%2C-679.1347C3867.9013%2C-658.5666%203444.163%2C-607.7363%203266.5901%2C-586.4352%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223266.8762%2C-582.9445%203256.5305%2C-585.2285%203266.0424%2C-589.8947%203266.8762%2C-582.9445%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223271.9769%22%20y%3D%22-600.6858%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3ETrue%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2062%20--%3E%0A%3Cg%20id%3D%22node63%22%20class%3D%22node%22%3E%0A%3Ctitle%3E62%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225101.3042%2C-614.5003%204961.2314%2C-614.5003%204961.2314%2C-539.0997%205101.3042%2C-539.0997%205101.3042%2C-614.5003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225031.2678%22%20y%3D%22-597.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B4%5D%20%26lt%3B%3D%20138.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225031.2678%22%20y%3D%22-581%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.365%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225031.2678%22%20y%3D%22-564.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2020055%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225031.2678%22%20y%3D%22-547.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B4814%2C%2015241%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%200%26%2345%3B%26gt%3B62%20--%3E%0A%3Cg%20id%3D%22edge62%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E0-%26gt%3B62%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4186.8398%2C-679.088C4356.7064%2C-658.5116%204775.1202%2C-607.8279%204951.1651%2C-586.5031%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224951.6335%2C-589.972%204961.14%2C-585.2948%204950.7917%2C-583.0228%204951.6335%2C-589.972%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224945.7165%22%20y%3D%22-600.77%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EFalse%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%202%20--%3E%0A%3Cg%20id%3D%22node3%22%20class%3D%22node%22%3E%0A%3Ctitle%3E2%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221641.3042%2C-503.3003%201501.2314%2C-503.3003%201501.2314%2C-427.8997%201641.3042%2C-427.8997%201641.3042%2C-503.3003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221571.2678%22%20y%3D%22-486.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B6%5D%20%26lt%3B%3D%202.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221571.2678%22%20y%3D%22-469.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.364%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221571.2678%22%20y%3D%22-453%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2017925%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221571.2678%22%20y%3D%22-436.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B13643%2C%204282%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%201%26%2345%3B%26gt%3B2%20--%3E%0A%3Cg%20id%3D%22edge2%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E1-%26gt%3B2%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3116.0951%2C-571.9683C2853.6391%2C-553.897%201930.6277%2C-490.3435%201651.6968%2C-471.1379%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221651.8702%2C-467.6416%201641.6534%2C-470.4464%201651.3893%2C-474.6251%201651.8702%2C-467.6416%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2033%20--%3E%0A%3Cg%20id%3D%22node34%22%20class%3D%22node%22%3E%0A%3Ctitle%3E33%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223252.8042%2C-503.3003%203119.7314%2C-503.3003%203119.7314%2C-427.8997%203252.8042%2C-427.8997%203252.8042%2C-503.3003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223186.2678%22%20y%3D%22-486.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B6%5D%20%26lt%3B%3D%202.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223186.2678%22%20y%3D%22-469.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.495%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223186.2678%22%20y%3D%22-453%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2011020%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223186.2678%22%20y%3D%22-436.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B6064%2C%204956%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%201%26%2345%3B%26gt%3B33%20--%3E%0A%3Cg%20id%3D%22edge33%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E1-%26gt%3B33%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3186.2678%2C-539.1528C3186.2678%2C-530.82%203186.2678%2C-521.8845%203186.2678%2C-513.2137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223189.7679%2C-513.2084%203186.2678%2C-503.2085%203182.7679%2C-513.2085%203189.7679%2C-513.2084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%203%20--%3E%0A%3Cg%20id%3D%22node4%22%20class%3D%22node%22%3E%0A%3Ctitle%3E3%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22896.3042%2C-392.1003%20756.2314%2C-392.1003%20756.2314%2C-316.6997%20896.3042%2C-316.6997%20896.3042%2C-392.1003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22826.2678%22%20y%3D%22-375.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2043.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22826.2678%22%20y%3D%22-358.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.348%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22826.2678%22%20y%3D%22-341.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2017161%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22826.2678%22%20y%3D%22-325%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B13316%2C%203845%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%202%26%2345%3B%26gt%3B3%20--%3E%0A%3Cg%20id%3D%22edge3%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E2-%26gt%3B3%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1501.18%2C-455.1386C1362.4286%2C-434.4283%201053.5179%2C-388.3197%20906.7245%2C-366.4091%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22906.8472%2C-362.8887%20896.4401%2C-364.874%20905.8138%2C-369.812%20906.8472%2C-362.8887%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2018%20--%3E%0A%3Cg%20id%3D%22node19%22%20class%3D%22node%22%3E%0A%3Ctitle%3E18%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221630.8043%2C-392.1003%201511.7313%2C-392.1003%201511.7313%2C-316.6997%201630.8043%2C-316.6997%201630.8043%2C-392.1003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221571.2678%22%20y%3D%22-375.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B7%5D%20%26lt%3B%3D%202.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221571.2678%22%20y%3D%22-358.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.49%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221571.2678%22%20y%3D%22-341.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20764%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221571.2678%22%20y%3D%22-325%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B327%2C%20437%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%202%26%2345%3B%26gt%3B18%20--%3E%0A%3Cg%20id%3D%22edge18%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E2-%26gt%3B18%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1571.2678%2C-427.9528C1571.2678%2C-419.62%201571.2678%2C-410.6845%201571.2678%2C-402.0137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221574.7679%2C-402.0084%201571.2678%2C-392.0085%201567.7679%2C-402.0085%201574.7679%2C-402.0084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%204%20--%3E%0A%3Cg%20id%3D%22node5%22%20class%3D%22node%22%3E%0A%3Ctitle%3E4%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22474.3042%2C-280.9003%20348.2314%2C-280.9003%20348.2314%2C-205.4997%20474.3042%2C-205.4997%20474.3042%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22411.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B6%5D%20%26lt%3B%3D%201.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22411.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.239%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22411.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%204568%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22411.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B3935%2C%20633%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%203%26%2345%3B%26gt%3B4%20--%3E%0A%3Cg%20id%3D%22edge4%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E3-%26gt%3B4%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M755.8687%2C-335.5364C680.4%2C-315.3144%20561.1354%2C-283.3573%20484.5668%2C-262.8406%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22485.0847%2C-259.356%20474.5196%2C-260.1484%20483.2729%2C-266.1174%20485.0847%2C-259.356%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2011%20--%3E%0A%3Cg%20id%3D%22node12%22%20class%3D%22node%22%3E%0A%3Ctitle%3E11%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22892.8042%2C-280.9003%20759.7314%2C-280.9003%20759.7314%2C-205.4997%20892.8042%2C-205.4997%20892.8042%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22826.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B4%5D%20%26lt%3B%3D%20119.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22826.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.38%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22826.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2012593%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22826.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B9381%2C%203212%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%203%26%2345%3B%26gt%3B11%20--%3E%0A%3Cg%20id%3D%22edge11%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E3-%26gt%3B11%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M826.2678%2C-316.7528C826.2678%2C-308.42%20826.2678%2C-299.4845%20826.2678%2C-290.8137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22829.7679%2C-290.8084%20826.2678%2C-280.8085%20822.7679%2C-290.8085%20829.7679%2C-290.8084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%205%20--%3E%0A%3Cg%20id%3D%22node6%22%20class%3D%22node%22%3E%0A%3Ctitle%3E5%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22271.3042%2C-169.7003%20145.2314%2C-169.7003%20145.2314%2C-94.2997%20271.3042%2C-94.2997%20271.3042%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22208.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B4%5D%20%26lt%3B%3D%20113.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22208.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.214%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22208.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%204121%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22208.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B3618%2C%20503%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%204%26%2345%3B%26gt%3B5%20--%3E%0A%3Cg%20id%3D%22edge5%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E4-%26gt%3B5%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M348.1501%2C-208.6252C326.6682%2C-196.8577%20302.4962%2C-183.6167%20280.3653%2C-171.4938%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22281.7983%2C-168.2881%20271.3465%2C-166.5534%20278.4353%2C-174.4274%20281.7983%2C-168.2881%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%208%20--%3E%0A%3Cg%20id%3D%22node9%22%20class%3D%22node%22%3E%0A%3Ctitle%3E8%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22470.8043%2C-169.7003%20351.7313%2C-169.7003%20351.7313%2C-94.2997%20470.8043%2C-94.2997%20470.8043%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22411.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B3%5D%20%26lt%3B%3D%2081.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22411.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.412%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22411.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20447%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22411.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B317%2C%20130%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%204%26%2345%3B%26gt%3B8%20--%3E%0A%3Cg%20id%3D%22edge8%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E4-%26gt%3B8%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M411.2678%2C-205.5528C411.2678%2C-197.22%20411.2678%2C-188.2845%20411.2678%2C-179.6137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22414.7679%2C-179.6084%20411.2678%2C-169.6085%20407.7679%2C-179.6085%20414.7679%2C-179.6084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%206%20--%3E%0A%3Cg%20id%3D%22node7%22%20class%3D%22node%22%3E%0A%3Ctitle%3E6%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22126.3042%2C-58.6014%20.2314%2C-58.6014%20.2314%2C.2014%20126.3042%2C.2014%20126.3042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%2263.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.154%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%2263.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201741%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%2263.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1595%2C%20146%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%205%26%2345%3B%26gt%3B6%20--%3E%0A%3Cg%20id%3D%22edge6%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E5-%26gt%3B6%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M155.0985%2C-94.3048C141.3101%2C-84.5293%20126.5153%2C-74.0403%20112.9156%2C-64.3986%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22114.804%2C-61.447%20104.6219%2C-58.5186%20110.7555%2C-67.1575%20114.804%2C-61.447%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%207%20--%3E%0A%3Cg%20id%3D%22node8%22%20class%3D%22node%22%3E%0A%3Ctitle%3E7%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22271.3042%2C-58.6014%20145.2314%2C-58.6014%20145.2314%2C.2014%20271.3042%2C.2014%20271.3042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22208.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.255%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22208.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%202380%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22208.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B2023%2C%20357%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%205%26%2345%3B%26gt%3B7%20--%3E%0A%3Cg%20id%3D%22edge7%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E5-%26gt%3B7%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M208.2678%2C-94.3048C208.2678%2C-85.9126%20208.2678%2C-76.9946%20208.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22211.7679%2C-68.5186%20208.2678%2C-58.5186%20204.7679%2C-68.5187%20211.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%209%20--%3E%0A%3Cg%20id%3D%22node10%22%20class%3D%22node%22%3E%0A%3Ctitle%3E9%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22402.3044%2C-58.6014%20290.2312%2C-58.6014%20290.2312%2C.2014%20402.3044%2C.2014%20402.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22346.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.374%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22346.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20338%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22346.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B254%2C%2084%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%208%26%2345%3B%26gt%3B9%20--%3E%0A%3Cg%20id%3D%22edge9%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E8-%26gt%3B9%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M387.4333%2C-94.3048C381.8354%2C-85.4515%20375.8675%2C-76.013%20370.2626%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22373.1084%2C-65.1003%20364.8058%2C-58.5186%20367.1919%2C-68.8413%20373.1084%2C-65.1003%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2010%20--%3E%0A%3Cg%20id%3D%22node11%22%20class%3D%22node%22%3E%0A%3Ctitle%3E10%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22525.8044%2C-58.6014%20420.7312%2C-58.6014%20420.7312%2C.2014%20525.8044%2C.2014%20525.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22473.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.488%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22473.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20109%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22473.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B63%2C%2046%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%208%26%2345%3B%26gt%3B10%20--%3E%0A%3Cg%20id%3D%22edge10%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E8-%26gt%3B10%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M434.0023%2C-94.3048C439.3418%2C-85.4515%20445.0343%2C-76.013%20450.3805%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22453.4179%2C-68.8894%20455.5854%2C-58.5186%20447.4237%2C-65.2742%20453.4179%2C-68.8894%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2012%20--%3E%0A%3Cg%20id%3D%22node13%22%20class%3D%22node%22%3E%0A%3Ctitle%3E12%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22815.3042%2C-169.7003%20689.2314%2C-169.7003%20689.2314%2C-94.2997%20815.3042%2C-94.2997%20815.3042%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22752.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B3%5D%20%26lt%3B%3D%2064.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22752.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.323%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22752.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%204152%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22752.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B3312%2C%20840%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2011%26%2345%3B%26gt%3B12%20--%3E%0A%3Cg%20id%3D%22edge12%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E11-%26gt%3B12%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M801.2148%2C-205.5528C795.3682%2C-196.7671%20789.0758%2C-187.3115%20783.0141%2C-178.2025%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22785.749%2C-175.9946%20777.295%2C-169.6085%20779.9214%2C-179.8726%20785.749%2C-175.9946%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2015%20--%3E%0A%3Cg%20id%3D%22node16%22%20class%3D%22node%22%3E%0A%3Ctitle%3E15%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22966.8042%2C-169.7003%20833.7314%2C-169.7003%20833.7314%2C-94.2997%20966.8042%2C-94.2997%20966.8042%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22900.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B6%5D%20%26lt%3B%3D%201.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22900.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.404%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22900.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%208441%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22900.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B6069%2C%202372%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2011%26%2345%3B%26gt%3B15%20--%3E%0A%3Cg%20id%3D%22edge15%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E11-%26gt%3B15%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M851.3208%2C-205.5528C857.1674%2C-196.7671%20863.4598%2C-187.3115%20869.5215%2C-178.2025%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22872.6142%2C-179.8726%20875.2406%2C-169.6085%20866.7866%2C-175.9946%20872.6142%2C-179.8726%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2013%20--%3E%0A%3Cg%20id%3D%22node14%22%20class%3D%22node%22%3E%0A%3Ctitle%3E13%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22670.3042%2C-58.6014%20544.2314%2C-58.6014%20544.2314%2C.2014%20670.3042%2C.2014%20670.3042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22607.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.249%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22607.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201599%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22607.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1366%2C%20233%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2012%26%2345%3B%26gt%3B13%20--%3E%0A%3Cg%20id%3D%22edge13%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E12-%26gt%3B13%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M699.0985%2C-94.3048C685.3101%2C-84.5293%20670.5153%2C-74.0403%20656.9156%2C-64.3986%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22658.804%2C-61.447%20648.6219%2C-58.5186%20654.7555%2C-67.1575%20658.804%2C-61.447%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2014%20--%3E%0A%3Cg%20id%3D%22node15%22%20class%3D%22node%22%3E%0A%3Ctitle%3E14%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22815.3042%2C-58.6014%20689.2314%2C-58.6014%20689.2314%2C.2014%20815.3042%2C.2014%20815.3042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22752.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.362%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22752.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%202553%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22752.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1946%2C%20607%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2012%26%2345%3B%26gt%3B14%20--%3E%0A%3Cg%20id%3D%22edge14%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E12-%26gt%3B14%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M752.2678%2C-94.3048C752.2678%2C-85.9126%20752.2678%2C-76.9946%20752.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22755.7679%2C-68.5186%20752.2678%2C-58.5186%20748.7679%2C-68.5187%20755.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2016%20--%3E%0A%3Cg%20id%3D%22node17%22%20class%3D%22node%22%3E%0A%3Ctitle%3E16%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22966.8042%2C-58.6014%20833.7314%2C-58.6014%20833.7314%2C.2014%20966.8042%2C.2014%20966.8042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22900.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.394%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22900.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%207646%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22900.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B5584%2C%202062%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2015%26%2345%3B%26gt%3B16%20--%3E%0A%3Cg%20id%3D%22edge16%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E15-%26gt%3B16%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M900.2678%2C-94.3048C900.2678%2C-85.9126%20900.2678%2C-76.9946%20900.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22903.7679%2C-68.5186%20900.2678%2C-58.5186%20896.7679%2C-68.5187%20903.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2017%20--%3E%0A%3Cg%20id%3D%22node18%22%20class%3D%22node%22%3E%0A%3Ctitle%3E17%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221104.8043%2C-58.6014%20985.7313%2C-58.6014%20985.7313%2C.2014%201104.8043%2C.2014%201104.8043%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221045.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.476%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221045.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20795%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221045.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B485%2C%20310%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2015%26%2345%3B%26gt%3B17%20--%3E%0A%3Cg%20id%3D%22edge17%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E15-%26gt%3B17%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M953.4371%2C-94.3048C967.2255%2C-84.5293%20982.0203%2C-74.0403%20995.62%2C-64.3986%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22997.7801%2C-67.1575%201003.9137%2C-58.5186%20993.7316%2C-61.447%20997.7801%2C-67.1575%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2019%20--%3E%0A%3Cg%20id%3D%22node20%22%20class%3D%22node%22%3E%0A%3Ctitle%3E19%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221533.3044%2C-280.9003%201421.2312%2C-280.9003%201421.2312%2C-205.4997%201533.3044%2C-205.4997%201533.3044%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221477.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B5%5D%20%26lt%3B%3D%2071.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221477.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.368%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221477.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20325%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221477.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B79%2C%20246%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2018%26%2345%3B%26gt%3B19%20--%3E%0A%3Cg%20id%3D%22edge19%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E18-%26gt%3B19%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1539.4437%2C-316.7528C1531.7873%2C-307.6954%201523.529%2C-297.926%201515.6103%2C-288.5584%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221518.1879%2C-286.186%201509.0591%2C-280.8085%201512.8419%2C-290.705%201518.1879%2C-286.186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2026%20--%3E%0A%3Cg%20id%3D%22node27%22%20class%3D%22node%22%3E%0A%3Ctitle%3E26%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221789.8043%2C-280.9003%201670.7313%2C-280.9003%201670.7313%2C-205.4997%201789.8043%2C-205.4997%201789.8043%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221730.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B3%5D%20%26lt%3B%3D%2080.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221730.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.492%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221730.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20439%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221730.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B248%2C%20191%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2018%26%2345%3B%26gt%3B26%20--%3E%0A%3Cg%20id%3D%22edge26%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E18-%26gt%3B26%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1625.0979%2C-316.7528C1638.9552%2C-307.0614%201653.9781%2C-296.5548%201668.2157%2C-286.5974%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221670.3043%2C-289.4078%201676.4931%2C-280.8085%201666.2924%2C-283.6715%201670.3043%2C-289.4078%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2020%20--%3E%0A%3Cg%20id%3D%22node21%22%20class%3D%22node%22%3E%0A%3Ctitle%3E20%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221338.8044%2C-169.7003%201233.7312%2C-169.7003%201233.7312%2C-94.2997%201338.8044%2C-94.2997%201338.8044%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221286.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B2%5D%20%26lt%3B%3D%20152.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221286.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.477%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221286.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2084%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221286.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B33%2C%2051%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2019%26%2345%3B%26gt%3B20%20--%3E%0A%3Cg%20id%3D%22edge20%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E19-%26gt%3B20%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1420.9946%2C-210.4378C1397.967%2C-197.0312%201371.2566%2C-181.4804%201347.7207%2C-167.7778%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221349.2732%2C-164.6317%201338.8701%2C-162.625%201345.7512%2C-170.6812%201349.2732%2C-164.6317%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2023%20--%3E%0A%3Cg%20id%3D%22node24%22%20class%3D%22node%22%3E%0A%3Ctitle%3E23%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221533.3044%2C-169.7003%201421.2312%2C-169.7003%201421.2312%2C-94.2997%201533.3044%2C-94.2997%201533.3044%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221477.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2051.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221477.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.309%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221477.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20241%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221477.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B46%2C%20195%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2019%26%2345%3B%26gt%3B23%20--%3E%0A%3Cg%20id%3D%22edge23%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E19-%26gt%3B23%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1477.2678%2C-205.5528C1477.2678%2C-197.22%201477.2678%2C-188.2845%201477.2678%2C-179.6137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221480.7679%2C-179.6084%201477.2678%2C-169.6085%201473.7679%2C-179.6085%201480.7679%2C-179.6084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2021%20--%3E%0A%3Cg%20id%3D%22node22%22%20class%3D%22node%22%3E%0A%3Ctitle%3E21%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221214.8046%2C-58.6014%201123.731%2C-58.6014%201123.731%2C.2014%201214.8046%2C.2014%201214.8046%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221169.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.278%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221169.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%206%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221169.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B5%2C%201%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2020%26%2345%3B%26gt%3B21%20--%3E%0A%3Cg%20id%3D%22edge21%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E20-%26gt%3B21%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1243.3657%2C-94.3048C1232.5547%2C-84.806%201220.9771%2C-74.6335%201210.2624%2C-65.2192%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221212.4587%2C-62.4899%201202.6363%2C-58.5186%201207.8384%2C-67.7484%201212.4587%2C-62.4899%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2022%20--%3E%0A%3Cg%20id%3D%22node23%22%20class%3D%22node%22%3E%0A%3Ctitle%3E22%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221338.8044%2C-58.6014%201233.7312%2C-58.6014%201233.7312%2C.2014%201338.8044%2C.2014%201338.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221286.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.46%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221286.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2078%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221286.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B28%2C%2050%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2020%26%2345%3B%26gt%3B22%20--%3E%0A%3Cg%20id%3D%22edge22%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E20-%26gt%3B22%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1286.2678%2C-94.3048C1286.2678%2C-85.9126%201286.2678%2C-76.9946%201286.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221289.7679%2C-68.5186%201286.2678%2C-58.5186%201282.7679%2C-68.5187%201289.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2024%20--%3E%0A%3Cg%20id%3D%22node25%22%20class%3D%22node%22%3E%0A%3Ctitle%3E24%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221469.3044%2C-58.6014%201357.2312%2C-58.6014%201357.2312%2C.2014%201469.3044%2C.2014%201469.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221413.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.259%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221413.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20170%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221413.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B26%2C%20144%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2023%26%2345%3B%26gt%3B24%20--%3E%0A%3Cg%20id%3D%22edge24%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E23-%26gt%3B24%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1453.8%2C-94.3048C1448.2882%2C-85.4515%201442.4121%2C-76.013%201436.8934%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221439.7771%2C-65.1581%201431.5206%2C-58.5186%201433.8346%2C-68.8577%201439.7771%2C-65.1581%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2025%20--%3E%0A%3Cg%20id%3D%22node26%22%20class%3D%22node%22%3E%0A%3Ctitle%3E25%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221592.8044%2C-58.6014%201487.7312%2C-58.6014%201487.7312%2C.2014%201592.8044%2C.2014%201592.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221540.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.405%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221540.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2071%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221540.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B20%2C%2051%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2023%26%2345%3B%26gt%3B25%20--%3E%0A%3Cg%20id%3D%22edge25%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E23-%26gt%3B25%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1500.3689%2C-94.3048C1505.7946%2C-85.4515%201511.5789%2C-76.013%201517.0113%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221520.0591%2C-68.8737%201522.3002%2C-58.5186%201514.0907%2C-65.216%201520.0591%2C-68.8737%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2027%20--%3E%0A%3Cg%20id%3D%22node28%22%20class%3D%22node%22%3E%0A%3Ctitle%3E27%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221789.8043%2C-169.7003%201670.7313%2C-169.7003%201670.7313%2C-94.2997%201789.8043%2C-94.2997%201789.8043%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221730.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B5%5D%20%26lt%3B%3D%2074.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221730.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.469%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221730.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20348%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221730.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B217%2C%20131%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2026%26%2345%3B%26gt%3B27%20--%3E%0A%3Cg%20id%3D%22edge27%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E26-%26gt%3B27%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1730.2678%2C-205.5528C1730.2678%2C-197.22%201730.2678%2C-188.2845%201730.2678%2C-179.6137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221733.7679%2C-179.6084%201730.2678%2C-169.6085%201726.7679%2C-179.6085%201733.7679%2C-179.6084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2030%20--%3E%0A%3Cg%20id%3D%22node31%22%20class%3D%22node%22%3E%0A%3Ctitle%3E30%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221978.8044%2C-169.7003%201873.7312%2C-169.7003%201873.7312%2C-94.2997%201978.8044%2C-94.2997%201978.8044%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221926.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B2%5D%20%26lt%3B%3D%20171.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221926.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.449%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221926.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2091%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221926.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B31%2C%2060%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2026%26%2345%3B%26gt%3B30%20--%3E%0A%3Cg%20id%3D%22edge30%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E26-%26gt%3B30%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1789.8726%2C-209.3834C1813.5411%2C-195.9552%201840.7588%2C-180.5133%201864.6246%2C-166.9731%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221866.6223%2C-169.8638%201873.5929%2C-161.885%201863.168%2C-163.7754%201866.6223%2C-169.8638%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2028%20--%3E%0A%3Cg%20id%3D%22node29%22%20class%3D%22node%22%3E%0A%3Ctitle%3E28%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221716.8044%2C-58.6014%201611.7312%2C-58.6014%201611.7312%2C.2014%201716.8044%2C.2014%201716.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221664.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.366%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221664.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2087%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221664.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B66%2C%2021%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2027%26%2345%3B%26gt%3B28%20--%3E%0A%3Cg%20id%3D%22edge28%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E27-%26gt%3B28%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1706.0666%2C-94.3048C1700.3826%2C-85.4515%201694.3228%2C-76.013%201688.6317%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221691.4389%2C-65.0427%201683.091%2C-58.5186%201685.5485%2C-68.8246%201691.4389%2C-65.0427%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2029%20--%3E%0A%3Cg%20id%3D%22node30%22%20class%3D%22node%22%3E%0A%3Ctitle%3E29%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221854.8043%2C-58.6014%201735.7313%2C-58.6014%201735.7313%2C.2014%201854.8043%2C.2014%201854.8043%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221795.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.488%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221795.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20261%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221795.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B151%2C%20110%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2027%26%2345%3B%26gt%3B29%20--%3E%0A%3Cg%20id%3D%22edge29%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E27-%26gt%3B29%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1754.1023%2C-94.3048C1759.7002%2C-85.4515%201765.6681%2C-76.013%201771.273%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221774.3437%2C-68.8413%201776.7298%2C-58.5186%201768.4272%2C-65.1003%201774.3437%2C-68.8413%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2031%20--%3E%0A%3Cg%20id%3D%22node32%22%20class%3D%22node%22%3E%0A%3Ctitle%3E31%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221978.8044%2C-58.6014%201873.7312%2C-58.6014%201873.7312%2C.2014%201978.8044%2C.2014%201978.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221926.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.313%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221926.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2067%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221926.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B13%2C%2054%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2030%26%2345%3B%26gt%3B31%20--%3E%0A%3Cg%20id%3D%22edge31%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E30-%26gt%3B31%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1926.2678%2C-94.3048C1926.2678%2C-85.9126%201926.2678%2C-76.9946%201926.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221929.7679%2C-68.5186%201926.2678%2C-58.5186%201922.7679%2C-68.5187%201929.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2032%20--%3E%0A%3Cg%20id%3D%22node33%22%20class%3D%22node%22%3E%0A%3Ctitle%3E32%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%222095.3045%2C-58.6014%201997.2311%2C-58.6014%201997.2311%2C.2014%202095.3045%2C.2014%202095.3045%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222046.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.375%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222046.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2024%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222046.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B18%2C%206%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2030%26%2345%3B%26gt%3B32%20--%3E%0A%3Cg%20id%3D%22edge32%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E30-%26gt%3B32%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1970.27%2C-94.3048C1981.3581%2C-84.806%201993.2326%2C-74.6335%202004.222%2C-65.2192%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222006.7264%2C-67.6825%202012.0437%2C-58.5186%202002.1723%2C-62.3665%202006.7264%2C-67.6825%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2034%20--%3E%0A%3Cg%20id%3D%22node35%22%20class%3D%22node%22%3E%0A%3Ctitle%3E34%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%222980.8042%2C-392.1003%202847.7314%2C-392.1003%202847.7314%2C-316.6997%202980.8042%2C-316.6997%202980.8042%2C-392.1003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222914.2678%22%20y%3D%22-375.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2060.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222914.2678%22%20y%3D%22-358.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.487%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222914.2678%22%20y%3D%22-341.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%209877%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222914.2678%22%20y%3D%22-325%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B5728%2C%204149%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2033%26%2345%3B%26gt%3B34%20--%3E%0A%3Cg%20id%3D%22edge34%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E33-%26gt%3B34%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3119.7291%2C-438.3974C3080.6252%2C-422.4108%203031.0484%2C-402.1426%202990.3916%2C-385.5212%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222991.6431%2C-382.2517%202981.0623%2C-381.7071%202988.9941%2C-388.7311%202991.6431%2C-382.2517%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2049%20--%3E%0A%3Cg%20id%3D%22node50%22%20class%3D%22node%22%3E%0A%3Ctitle%3E49%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223514.8043%2C-392.1003%203395.7313%2C-392.1003%203395.7313%2C-316.6997%203514.8043%2C-316.6997%203514.8043%2C-392.1003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223455.2678%22%20y%3D%22-375.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B3%5D%20%26lt%3B%3D%2070.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223455.2678%22%20y%3D%22-358.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.415%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223455.2678%22%20y%3D%22-341.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201143%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223455.2678%22%20y%3D%22-325%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B336%2C%20807%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2033%26%2345%3B%26gt%3B49%20--%3E%0A%3Cg%20id%3D%22edge49%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E33-%26gt%3B49%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3253.1078%2C-437.9695C3293.672%2C-421.2009%203345.3384%2C-399.8429%203386.2851%2C-382.9163%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223387.7438%2C-386.1006%203395.6482%2C-379.0457%203385.0695%2C-379.6315%203387.7438%2C-386.1006%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2035%20--%3E%0A%3Cg%20id%3D%22node36%22%20class%3D%22node%22%3E%0A%3Ctitle%3E35%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%222589.8042%2C-280.9003%202456.7314%2C-280.9003%202456.7314%2C-205.4997%202589.8042%2C-205.4997%202589.8042%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222523.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B4%5D%20%26lt%3B%3D%20119.0%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222523.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.472%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222523.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%207292%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222523.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B4511%2C%202781%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2034%26%2345%3B%26gt%3B35%20--%3E%0A%3Cg%20id%3D%22edge35%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E34-%26gt%3B35%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M2847.4968%2C-335.4104C2778.8567%2C-315.8892%202672.1052%2C-285.5292%202600.0237%2C-265.0293%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222600.6664%2C-261.5734%202590.0904%2C-262.2043%202598.7515%2C-268.3064%202600.6664%2C-261.5734%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2042%20--%3E%0A%3Cg%20id%3D%22node43%22%20class%3D%22node%22%3E%0A%3Ctitle%3E42%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%222980.8042%2C-280.9003%202847.7314%2C-280.9003%202847.7314%2C-205.4997%202980.8042%2C-205.4997%202980.8042%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222914.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B3%5D%20%26lt%3B%3D%2052.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222914.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.498%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222914.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%202585%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222914.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1217%2C%201368%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2034%26%2345%3B%26gt%3B42%20--%3E%0A%3Cg%20id%3D%22edge42%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E34-%26gt%3B42%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M2914.2678%2C-316.7528C2914.2678%2C-308.42%202914.2678%2C-299.4845%202914.2678%2C-290.8137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222917.7679%2C-290.8084%202914.2678%2C-280.8085%202910.7679%2C-290.8085%202917.7679%2C-290.8084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2036%20--%3E%0A%3Cg%20id%3D%22node37%22%20class%3D%22node%22%3E%0A%3Ctitle%3E36%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%222374.3042%2C-169.7003%202248.2314%2C-169.7003%202248.2314%2C-94.2997%202374.3042%2C-94.2997%202374.3042%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222311.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B3%5D%20%26lt%3B%3D%2091.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222311.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.415%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222311.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201854%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222311.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1310%2C%20544%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2035%26%2345%3B%26gt%3B36%20--%3E%0A%3Cg%20id%3D%22edge36%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E35-%26gt%3B36%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M2456.4807%2C-208.1683C2433.2345%2C-195.975%202407.0595%2C-182.2454%202383.3999%2C-169.8353%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222384.9481%2C-166.6952%202374.4666%2C-165.1496%202381.6965%2C-172.8942%202384.9481%2C-166.6952%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2039%20--%3E%0A%3Cg%20id%3D%22node40%22%20class%3D%22node%22%3E%0A%3Ctitle%3E39%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%222589.8042%2C-169.7003%202456.7314%2C-169.7003%202456.7314%2C-94.2997%202589.8042%2C-94.2997%202589.8042%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222523.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B10%5D%20%26lt%3B%3D%200.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222523.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.484%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222523.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%205438%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222523.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B3201%2C%202237%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2035%26%2345%3B%26gt%3B39%20--%3E%0A%3Cg%20id%3D%22edge39%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E35-%26gt%3B39%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M2523.2678%2C-205.5528C2523.2678%2C-197.22%202523.2678%2C-188.2845%202523.2678%2C-179.6137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222526.7679%2C-179.6084%202523.2678%2C-169.6085%202519.7679%2C-179.6085%202526.7679%2C-179.6084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2037%20--%3E%0A%3Cg%20id%3D%22node38%22%20class%3D%22node%22%3E%0A%3Ctitle%3E37%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%222240.3042%2C-58.6014%202114.2314%2C-58.6014%202114.2314%2C.2014%202240.3042%2C.2014%202240.3042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222177.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.402%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222177.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201755%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222177.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1265%2C%20490%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2036%26%2345%3B%26gt%3B37%20--%3E%0A%3Cg%20id%3D%22edge37%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E36-%26gt%3B37%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M2262.132%2C-94.3048C2249.5098%2C-84.6215%202235.9751%2C-74.2382%202223.5052%2C-64.6717%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222225.5493%2C-61.8285%202215.4847%2C-58.5186%202221.2885%2C-67.3824%202225.5493%2C-61.8285%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2038%20--%3E%0A%3Cg%20id%3D%22node39%22%20class%3D%22node%22%3E%0A%3Ctitle%3E38%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%222363.8044%2C-58.6014%202258.7312%2C-58.6014%202258.7312%2C.2014%202363.8044%2C.2014%202363.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222311.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.496%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222311.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2099%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222311.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B45%2C%2054%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2036%26%2345%3B%26gt%3B38%20--%3E%0A%3Cg%20id%3D%22edge38%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E36-%26gt%3B38%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M2311.2678%2C-94.3048C2311.2678%2C-85.9126%202311.2678%2C-76.9946%202311.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222314.7679%2C-68.5186%202311.2678%2C-58.5186%202307.7679%2C-68.5187%202314.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2040%20--%3E%0A%3Cg%20id%3D%22node41%22%20class%3D%22node%22%3E%0A%3Ctitle%3E40%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%222501.8043%2C-58.6014%202382.7313%2C-58.6014%202382.7313%2C.2014%202501.8043%2C.2014%202501.8043%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222442.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222442.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201094%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222442.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B564%2C%20530%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2039%26%2345%3B%26gt%3B40%20--%3E%0A%3Cg%20id%3D%22edge40%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E39-%26gt%3B40%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M2493.5663%2C-94.3048C2486.4452%2C-85.2671%202478.8435%2C-75.6195%202471.733%2C-66.5954%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222474.3073%2C-64.2072%202465.3691%2C-58.5186%202468.809%2C-68.5395%202474.3073%2C-64.2072%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2041%20--%3E%0A%3Cg%20id%3D%22node42%22%20class%3D%22node%22%3E%0A%3Ctitle%3E41%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%222653.8042%2C-58.6014%202520.7314%2C-58.6014%202520.7314%2C.2014%202653.8042%2C.2014%202653.8042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222587.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.477%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222587.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%204344%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222587.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B2637%2C%201707%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2039%26%2345%3B%26gt%3B41%20--%3E%0A%3Cg%20id%3D%22edge41%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E39-%26gt%3B41%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M2546.7356%2C-94.3048C2552.2474%2C-85.4515%202558.1235%2C-76.013%202563.6422%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222566.701%2C-68.8577%202569.015%2C-58.5186%202560.7585%2C-65.1581%202566.701%2C-68.8577%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2043%20--%3E%0A%3Cg%20id%3D%22node44%22%20class%3D%22node%22%3E%0A%3Ctitle%3E43%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%222897.8044%2C-169.7003%202792.7312%2C-169.7003%202792.7312%2C-94.2997%202897.8044%2C-94.2997%202897.8044%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222845.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B5%5D%20%26lt%3B%3D%2079.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222845.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.448%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222845.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20115%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222845.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B76%2C%2039%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2042%26%2345%3B%26gt%3B43%20--%3E%0A%3Cg%20id%3D%22edge43%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E42-%26gt%3B43%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M2890.9075%2C-205.5528C2885.456%2C-196.7671%202879.5888%2C-187.3115%202873.9366%2C-178.2025%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222876.8505%2C-176.2602%202868.604%2C-169.6085%202870.9025%2C-179.951%202876.8505%2C-176.2602%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2046%20--%3E%0A%3Cg%20id%3D%22node47%22%20class%3D%22node%22%3E%0A%3Ctitle%3E46%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223049.8042%2C-169.7003%202916.7314%2C-169.7003%202916.7314%2C-94.2997%203049.8042%2C-94.2997%203049.8042%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222983.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B10%5D%20%26lt%3B%3D%200.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222983.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.497%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222983.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%202470%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222983.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1141%2C%201329%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2042%26%2345%3B%26gt%3B46%20--%3E%0A%3Cg%20id%3D%22edge46%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E42-%26gt%3B46%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M2937.6281%2C-205.5528C2943.0796%2C-196.7671%202948.9468%2C-187.3115%202954.599%2C-178.2025%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222957.6331%2C-179.951%202959.9316%2C-169.6085%202951.6851%2C-176.2602%202957.6331%2C-179.951%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2044%20--%3E%0A%3Cg%20id%3D%22node45%22%20class%3D%22node%22%3E%0A%3Ctitle%3E44%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%222777.8044%2C-58.6014%202672.7312%2C-58.6014%202672.7312%2C.2014%202777.8044%2C.2014%202777.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222725.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.334%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222725.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2052%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222725.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B41%2C%2011%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2043%26%2345%3B%26gt%3B44%20--%3E%0A%3Cg%20id%3D%22edge44%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E43-%26gt%3B44%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M2801.2656%2C-94.3048C2790.1775%2C-84.806%202778.303%2C-74.6335%202767.3136%2C-65.2192%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222769.3633%2C-62.3665%202759.4919%2C-58.5186%202764.8092%2C-67.6825%202769.3633%2C-62.3665%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2045%20--%3E%0A%3Cg%20id%3D%22node46%22%20class%3D%22node%22%3E%0A%3Ctitle%3E45%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%222901.8044%2C-58.6014%202796.7312%2C-58.6014%202796.7312%2C.2014%202901.8044%2C.2014%202901.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222849.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.494%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222849.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2063%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222849.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B35%2C%2028%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2043%26%2345%3B%26gt%3B45%20--%3E%0A%3Cg%20id%3D%22edge45%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E43-%26gt%3B45%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M2846.7345%2C-94.3048C2847.0611%2C-85.9126%202847.4081%2C-76.9946%202847.7371%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222851.2354%2C-68.6472%202848.127%2C-58.5186%202844.2407%2C-68.375%202851.2354%2C-68.6472%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2047%20--%3E%0A%3Cg%20id%3D%22node48%22%20class%3D%22node%22%3E%0A%3Ctitle%3E47%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223039.8043%2C-58.6014%202920.7313%2C-58.6014%202920.7313%2C.2014%203039.8043%2C.2014%203039.8043%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222980.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.478%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222980.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20520%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222980.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B205%2C%20315%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2046%26%2345%3B%26gt%3B47%20--%3E%0A%3Cg%20id%3D%22edge47%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E46-%26gt%3B47%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M2982.1677%2C-94.3048C2981.9228%2C-85.9126%202981.6626%2C-76.9946%202981.4158%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222984.9137%2C-68.4122%202981.1234%2C-58.5186%202977.9167%2C-68.6165%202984.9137%2C-68.4122%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2048%20--%3E%0A%3Cg%20id%3D%22node49%22%20class%3D%22node%22%3E%0A%3Ctitle%3E48%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223184.3042%2C-58.6014%203058.2314%2C-58.6014%203058.2314%2C.2014%203184.3042%2C.2014%203184.3042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223121.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.499%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223121.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201950%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223121.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B936%2C%201014%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2046%26%2345%3B%26gt%3B48%20--%3E%0A%3Cg%20id%3D%22edge48%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E46-%26gt%3B48%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3033.8703%2C-94.3048C3046.8693%2C-84.6215%203060.808%2C-74.2382%203073.6502%2C-64.6717%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223075.9815%2C-67.2994%203081.9101%2C-58.5186%203071.7997%2C-61.6858%203075.9815%2C-67.2994%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2050%20--%3E%0A%3Cg%20id%3D%22node51%22%20class%3D%22node%22%3E%0A%3Ctitle%3E50%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223514.8043%2C-280.9003%203395.7313%2C-280.9003%203395.7313%2C-205.4997%203514.8043%2C-205.4997%203514.8043%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223455.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2060.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223455.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.466%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223455.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20538%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223455.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B199%2C%20339%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2049%26%2345%3B%26gt%3B50%20--%3E%0A%3Cg%20id%3D%22edge50%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E49-%26gt%3B50%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3455.2678%2C-316.7528C3455.2678%2C-308.42%203455.2678%2C-299.4845%203455.2678%2C-290.8137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223458.7679%2C-290.8084%203455.2678%2C-280.8085%203451.7679%2C-290.8085%203458.7679%2C-290.8084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2057%20--%3E%0A%3Cg%20id%3D%22node58%22%20class%3D%22node%22%3E%0A%3Ctitle%3E57%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223828.8043%2C-280.9003%203709.7313%2C-280.9003%203709.7313%2C-205.4997%203828.8043%2C-205.4997%203828.8043%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223769.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B2%5D%20%26lt%3B%3D%20185.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223769.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.35%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223769.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20605%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223769.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B137%2C%20468%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2049%26%2345%3B%26gt%3B57%20--%3E%0A%3Cg%20id%3D%22edge57%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E49-%26gt%3B57%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3515.0679%2C-333.2224C3567.7758%2C-314.5564%203644.5802%2C-287.3569%203699.9572%2C-267.7457%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223701.4324%2C-270.9363%203709.6903%2C-264.2988%203699.0956%2C-264.3379%203701.4324%2C-270.9363%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2051%20--%3E%0A%3Cg%20id%3D%22node52%22%20class%3D%22node%22%3E%0A%3Ctitle%3E51%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223447.8043%2C-169.7003%203328.7313%2C-169.7003%203328.7313%2C-94.2997%203447.8043%2C-94.2997%203447.8043%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223388.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B2%5D%20%26lt%3B%3D%20161.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223388.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.489%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223388.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20366%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223388.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B156%2C%20210%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2050%26%2345%3B%26gt%3B51%20--%3E%0A%3Cg%20id%3D%22edge51%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E50-%26gt%3B51%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3432.5847%2C-205.5528C3427.2911%2C-196.7671%203421.594%2C-187.3115%203416.1056%2C-178.2025%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223419.0863%2C-176.3676%203410.9276%2C-169.6085%203413.0905%2C-179.9802%203419.0863%2C-176.3676%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2054%20--%3E%0A%3Cg%20id%3D%22node55%22%20class%3D%22node%22%3E%0A%3Ctitle%3E54%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223578.3044%2C-169.7003%203466.2312%2C-169.7003%203466.2312%2C-94.2997%203578.3044%2C-94.2997%203578.3044%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223522.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2063.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223522.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.375%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223522.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20172%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223522.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B43%2C%20129%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2050%26%2345%3B%26gt%3B54%20--%3E%0A%3Cg%20id%3D%22edge54%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E50-%26gt%3B54%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3477.9509%2C-205.5528C3483.2445%2C-196.7671%203488.9416%2C-187.3115%203494.43%2C-178.2025%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223497.4451%2C-179.9802%203499.608%2C-169.6085%203491.4493%2C-176.3676%203497.4451%2C-179.9802%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2052%20--%3E%0A%3Cg%20id%3D%22node53%22%20class%3D%22node%22%3E%0A%3Ctitle%3E52%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223315.3044%2C-58.6014%203203.2312%2C-58.6014%203203.2312%2C.2014%203315.3044%2C.2014%203315.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223259.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.458%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223259.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20163%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223259.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B58%2C%20105%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2051%26%2345%3B%26gt%3B52%20--%3E%0A%3Cg%20id%3D%22edge52%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E51-%26gt%3B52%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3340.9655%2C-94.3048C3328.93%2C-84.7137%203316.0327%2C-74.4359%203304.1232%2C-64.9452%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223306.0605%2C-62.0136%203296.0587%2C-58.5186%203301.6979%2C-67.488%203306.0605%2C-62.0136%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2053%20--%3E%0A%3Cg%20id%3D%22node54%22%20class%3D%22node%22%3E%0A%3Ctitle%3E53%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223446.3044%2C-58.6014%203334.2312%2C-58.6014%203334.2312%2C.2014%203446.3044%2C.2014%203446.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223390.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.499%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223390.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20203%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223390.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B98%2C%20105%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2051%26%2345%3B%26gt%3B53%20--%3E%0A%3Cg%20id%3D%22edge53%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E51-%26gt%3B53%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3389.0012%2C-94.3048C3389.1644%2C-85.9126%203389.3379%2C-76.9946%203389.5025%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223393.0021%2C-68.5849%203389.6974%2C-58.5186%203386.0034%2C-68.4486%203393.0021%2C-68.5849%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2055%20--%3E%0A%3Cg%20id%3D%22node56%22%20class%3D%22node%22%3E%0A%3Ctitle%3E55%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223577.3044%2C-58.6014%203465.2312%2C-58.6014%203465.2312%2C.2014%203577.3044%2C.2014%203577.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223521.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.407%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223521.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20144%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223521.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B41%2C%20103%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2054%26%2345%3B%26gt%3B55%20--%3E%0A%3Cg%20id%3D%22edge55%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E54-%26gt%3B55%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3521.9011%2C-94.3048C3521.8195%2C-85.9126%203521.7327%2C-76.9946%203521.6505%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223525.1502%2C-68.4841%203521.553%2C-58.5186%203518.1505%2C-68.5522%203525.1502%2C-68.4841%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2056%20--%3E%0A%3Cg%20id%3D%22node57%22%20class%3D%22node%22%3E%0A%3Ctitle%3E56%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223694.3045%2C-58.6014%203596.2311%2C-58.6014%203596.2311%2C.2014%203694.3045%2C.2014%203694.3045%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223645.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.133%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223645.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2028%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223645.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B2%2C%2026%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2054%26%2345%3B%26gt%3B56%20--%3E%0A%3Cg%20id%3D%22edge56%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E54-%26gt%3B56%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3567.37%2C-94.3048C3578.8457%2C-84.7137%203591.1432%2C-74.4359%203602.4987%2C-64.9452%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223604.7596%2C-67.6171%203610.1881%2C-58.5186%203600.2706%2C-62.246%203604.7596%2C-67.6171%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2058%20--%3E%0A%3Cg%20id%3D%22node59%22%20class%3D%22node%22%3E%0A%3Ctitle%3E58%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223828.8043%2C-169.7003%203709.7313%2C-169.7003%203709.7313%2C-94.2997%203828.8043%2C-94.2997%203828.8043%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223769.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2060.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223769.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.343%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223769.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20600%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223769.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B132%2C%20468%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2057%26%2345%3B%26gt%3B58%20--%3E%0A%3Cg%20id%3D%22edge58%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E57-%26gt%3B58%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3769.2678%2C-205.5528C3769.2678%2C-197.22%203769.2678%2C-188.2845%203769.2678%2C-179.6137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223772.7679%2C-179.6084%203769.2678%2C-169.6085%203765.7679%2C-179.6085%203772.7679%2C-179.6084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2061%20--%3E%0A%3Cg%20id%3D%22node62%22%20class%3D%22node%22%3E%0A%3Ctitle%3E61%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223938.8046%2C-161.4014%203847.731%2C-161.4014%203847.731%2C-102.5986%203938.8046%2C-102.5986%203938.8046%2C-161.4014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223893.2678%22%20y%3D%22-144.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.0%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223893.2678%22%20y%3D%22-127.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%205%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223893.2678%22%20y%3D%22-111%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B5%2C%200%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2057%26%2345%3B%26gt%3B61%20--%3E%0A%3Cg%20id%3D%22edge61%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E57-%26gt%3B61%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3811.2485%2C-205.5528C3824.6397%2C-193.5439%203839.4267%2C-180.2833%203852.7147%2C-168.367%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223855.2352%2C-170.8079%203860.3433%2C-161.5258%203850.5617%2C-165.5965%203855.2352%2C-170.8079%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2059%20--%3E%0A%3Cg%20id%3D%22node60%22%20class%3D%22node%22%3E%0A%3Ctitle%3E59%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223825.3044%2C-58.6014%203713.2312%2C-58.6014%203713.2312%2C.2014%203825.3044%2C.2014%203825.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223769.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.377%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223769.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20389%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223769.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B98%2C%20291%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2058%26%2345%3B%26gt%3B59%20--%3E%0A%3Cg%20id%3D%22edge59%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E58-%26gt%3B59%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3769.2678%2C-94.3048C3769.2678%2C-85.9126%203769.2678%2C-76.9946%203769.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223772.7679%2C-68.5186%203769.2678%2C-58.5186%203765.7679%2C-68.5187%203772.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2060%20--%3E%0A%3Cg%20id%3D%22node61%22%20class%3D%22node%22%3E%0A%3Ctitle%3E60%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223956.3044%2C-58.6014%203844.2312%2C-58.6014%203844.2312%2C.2014%203956.3044%2C.2014%203956.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223900.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.27%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223900.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20211%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223900.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B34%2C%20177%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2058%26%2345%3B%26gt%3B60%20--%3E%0A%3Cg%20id%3D%22edge60%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E58-%26gt%3B60%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3817.3035%2C-94.3048C3829.5256%2C-84.7137%203842.6229%2C-74.4359%203854.717%2C-64.9452%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223857.2003%2C-67.4455%203862.9065%2C-58.5186%203852.8788%2C-61.9387%203857.2003%2C-67.4455%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2063%20--%3E%0A%3Cg%20id%3D%22node64%22%20class%3D%22node%22%3E%0A%3Ctitle%3E63%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225097.8042%2C-503.3003%204964.7314%2C-503.3003%204964.7314%2C-427.8997%205097.8042%2C-427.8997%205097.8042%2C-503.3003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225031.2678%22%20y%3D%22-486.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B6%5D%20%26lt%3B%3D%202.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225031.2678%22%20y%3D%22-469.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.481%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225031.2678%22%20y%3D%22-453%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%206533%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225031.2678%22%20y%3D%22-436.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B2636%2C%203897%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2062%26%2345%3B%26gt%3B63%20--%3E%0A%3Cg%20id%3D%22edge63%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E62-%26gt%3B63%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5031.2678%2C-539.1528C5031.2678%2C-530.82%205031.2678%2C-521.8845%205031.2678%2C-513.2137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225034.7679%2C-513.2084%205031.2678%2C-503.2085%205027.7679%2C-513.2085%205034.7679%2C-513.2084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2094%20--%3E%0A%3Cg%20id%3D%22node95%22%20class%3D%22node%22%3E%0A%3Ctitle%3E94%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226634.3042%2C-503.3003%206494.2314%2C-503.3003%206494.2314%2C-427.8997%206634.3042%2C-427.8997%206634.3042%2C-503.3003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226564.2678%22%20y%3D%22-486.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B4%5D%20%26lt%3B%3D%20149.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226564.2678%22%20y%3D%22-469.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.27%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226564.2678%22%20y%3D%22-453%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2013522%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226564.2678%22%20y%3D%22-436.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B2178%2C%2011344%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2062%26%2345%3B%26gt%3B94%20--%3E%0A%3Cg%20id%3D%22edge94%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E62-%26gt%3B94%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5101.3735%2C-571.7147C5353.8506%2C-553.4006%206215.853%2C-490.8731%206483.995%2C-471.4228%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226484.3211%2C-474.9084%206494.0417%2C-470.694%206483.8146%2C-467.9268%206484.3211%2C-474.9084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2064%20--%3E%0A%3Cg%20id%3D%22node65%22%20class%3D%22node%22%3E%0A%3Ctitle%3E64%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224838.8042%2C-392.1003%204705.7314%2C-392.1003%204705.7314%2C-316.6997%204838.8042%2C-316.6997%204838.8042%2C-392.1003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224772.2678%22%20y%3D%22-375.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2058.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224772.2678%22%20y%3D%22-358.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.495%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224772.2678%22%20y%3D%22-341.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%205294%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224772.2678%22%20y%3D%22-325%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B2379%2C%202915%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2063%26%2345%3B%26gt%3B64%20--%3E%0A%3Cg%20id%3D%22edge64%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E63-%26gt%3B64%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4964.5689%2C-436.9632C4929.0584%2C-421.7171%204885.2353%2C-402.9019%204848.3964%2C-387.0853%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224849.7601%2C-383.8619%204839.1904%2C-383.1328%204846.9984%2C-390.2941%204849.7601%2C-383.8619%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2079%20--%3E%0A%3Cg%20id%3D%22node80%22%20class%3D%22node%22%3E%0A%3Ctitle%3E79%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225344.8043%2C-392.1003%205225.7313%2C-392.1003%205225.7313%2C-316.6997%205344.8043%2C-316.6997%205344.8043%2C-392.1003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225285.2678%22%20y%3D%22-375.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B7%5D%20%26lt%3B%3D%202.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225285.2678%22%20y%3D%22-358.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.329%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225285.2678%22%20y%3D%22-341.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201239%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225285.2678%22%20y%3D%22-325%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B257%2C%20982%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2063%26%2345%3B%26gt%3B79%20--%3E%0A%3Cg%20id%3D%22edge79%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E63-%26gt%3B79%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5098.0033%2C-436.3835C5134.4531%2C-420.426%205179.5246%2C-400.6939%205216.3154%2C-384.587%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225217.7234%2C-387.7914%205225.4803%2C-380.5747%205214.9161%2C-381.379%205217.7234%2C-387.7914%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2065%20--%3E%0A%3Cg%20id%3D%22node66%22%20class%3D%22node%22%3E%0A%3Ctitle%3E65%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224448.8042%2C-280.9003%204315.7314%2C-280.9003%204315.7314%2C-205.4997%204448.8042%2C-205.4997%204448.8042%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224382.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B5%5D%20%26lt%3B%3D%2088.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224382.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224382.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%203797%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224382.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1839%2C%201958%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2064%26%2345%3B%26gt%3B65%20--%3E%0A%3Cg%20id%3D%22edge65%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E64-%26gt%3B65%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4705.6675%2C-335.4104C4637.2031%2C-315.8892%204530.7246%2C-285.5292%204458.8274%2C-265.0293%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224459.496%2C-261.5805%204448.9195%2C-262.2043%204457.5765%2C-268.3122%204459.496%2C-261.5805%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2072%20--%3E%0A%3Cg%20id%3D%22node73%22%20class%3D%22node%22%3E%0A%3Ctitle%3E72%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224831.8043%2C-280.9003%204712.7313%2C-280.9003%204712.7313%2C-205.4997%204831.8043%2C-205.4997%204831.8043%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224772.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2061.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224772.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.461%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224772.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201497%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224772.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B540%2C%20957%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2064%26%2345%3B%26gt%3B72%20--%3E%0A%3Cg%20id%3D%22edge72%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E64-%26gt%3B72%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4772.2678%2C-316.7528C4772.2678%2C-308.42%204772.2678%2C-299.4845%204772.2678%2C-290.8137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224775.7679%2C-290.8084%204772.2678%2C-280.8085%204768.7679%2C-290.8085%204775.7679%2C-290.8084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2066%20--%3E%0A%3Cg%20id%3D%22node67%22%20class%3D%22node%22%3E%0A%3Ctitle%3E66%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224238.8042%2C-169.7003%204105.7314%2C-169.7003%204105.7314%2C-94.2997%204238.8042%2C-94.2997%204238.8042%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224172.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2041.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224172.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.498%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224172.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%202245%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224172.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1189%2C%201056%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2065%26%2345%3B%26gt%3B66%20--%3E%0A%3Cg%20id%3D%22edge66%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E65-%26gt%3B66%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4315.5339%2C-207.8628C4294.0004%2C-196.4603%204269.9795%2C-183.7407%204247.8539%2C-172.0246%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224249.3034%2C-168.8318%204238.828%2C-167.2452%204246.0276%2C-175.0181%204249.3034%2C-168.8318%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2069%20--%3E%0A%3Cg%20id%3D%22node70%22%20class%3D%22node%22%3E%0A%3Ctitle%3E69%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224441.8043%2C-169.7003%204322.7313%2C-169.7003%204322.7313%2C-94.2997%204441.8043%2C-94.2997%204441.8043%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224382.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B8%5D%20%26lt%3B%3D%200.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224382.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.487%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224382.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201552%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224382.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B650%2C%20902%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2065%26%2345%3B%26gt%3B69%20--%3E%0A%3Cg%20id%3D%22edge69%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E65-%26gt%3B69%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4382.2678%2C-205.5528C4382.2678%2C-197.22%204382.2678%2C-188.2845%204382.2678%2C-179.6137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224385.7679%2C-179.6084%204382.2678%2C-169.6085%204378.7679%2C-179.6085%204385.7679%2C-179.6084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2067%20--%3E%0A%3Cg%20id%3D%22node68%22%20class%3D%22node%22%3E%0A%3Ctitle%3E67%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224087.3044%2C-58.6014%203975.2312%2C-58.6014%203975.2312%2C.2014%204087.3044%2C.2014%204087.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224031.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.439%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224031.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20157%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224031.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B106%2C%2051%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2066%26%2345%3B%26gt%3B67%20--%3E%0A%3Cg%20id%3D%22edge67%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E66-%26gt%3B67%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4120.5652%2C-94.3048C4107.2837%2C-84.6215%204093.0419%2C-74.2382%204079.9206%2C-64.6717%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224081.6235%2C-61.5818%204071.4811%2C-58.5186%204077.4996%2C-67.2381%204081.6235%2C-61.5818%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2068%20--%3E%0A%3Cg%20id%3D%22node69%22%20class%3D%22node%22%3E%0A%3Ctitle%3E68%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224238.8042%2C-58.6014%204105.7314%2C-58.6014%204105.7314%2C.2014%204238.8042%2C.2014%204238.8042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224172.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.499%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224172.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%202088%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224172.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1083%2C%201005%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2066%26%2345%3B%26gt%3B68%20--%3E%0A%3Cg%20id%3D%22edge68%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E66-%26gt%3B68%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4172.2678%2C-94.3048C4172.2678%2C-85.9126%204172.2678%2C-76.9946%204172.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224175.7679%2C-68.5186%204172.2678%2C-58.5186%204168.7679%2C-68.5187%204175.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2070%20--%3E%0A%3Cg%20id%3D%22node71%22%20class%3D%22node%22%3E%0A%3Ctitle%3E70%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224376.8043%2C-58.6014%204257.7313%2C-58.6014%204257.7313%2C.2014%204376.8043%2C.2014%204376.8043%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224317.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.482%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224317.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201391%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224317.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B562%2C%20829%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2069%26%2345%3B%26gt%3B70%20--%3E%0A%3Cg%20id%3D%22edge70%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E69-%26gt%3B70%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4358.4333%2C-94.3048C4352.8354%2C-85.4515%204346.8675%2C-76.013%204341.2626%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224344.1084%2C-65.1003%204335.8058%2C-58.5186%204338.1919%2C-68.8413%204344.1084%2C-65.1003%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2071%20--%3E%0A%3Cg%20id%3D%22node72%22%20class%3D%22node%22%3E%0A%3Ctitle%3E71%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224500.8044%2C-58.6014%204395.7312%2C-58.6014%204395.7312%2C.2014%204500.8044%2C.2014%204500.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224448.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.496%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224448.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20161%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224448.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B88%2C%2073%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2069%26%2345%3B%26gt%3B71%20--%3E%0A%3Cg%20id%3D%22edge71%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E69-%26gt%3B71%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4406.469%2C-94.3048C4412.153%2C-85.4515%204418.2128%2C-76.013%204423.9039%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224426.9871%2C-68.8246%204429.4446%2C-58.5186%204421.0967%2C-65.0427%204426.9871%2C-68.8246%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2073%20--%3E%0A%3Cg%20id%3D%22node74%22%20class%3D%22node%22%3E%0A%3Ctitle%3E73%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224762.8043%2C-169.7003%204643.7313%2C-169.7003%204643.7313%2C-94.2997%204762.8043%2C-94.2997%204762.8043%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224703.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B8%5D%20%26lt%3B%3D%200.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224703.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.482%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224703.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20849%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224703.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B343%2C%20506%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2072%26%2345%3B%26gt%3B73%20--%3E%0A%3Cg%20id%3D%22edge73%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E72-%26gt%3B73%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4748.9075%2C-205.5528C4743.456%2C-196.7671%204737.5888%2C-187.3115%204731.9366%2C-178.2025%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224734.8505%2C-176.2602%204726.604%2C-169.6085%204728.9025%2C-179.951%204734.8505%2C-176.2602%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2076%20--%3E%0A%3Cg%20id%3D%22node77%22%20class%3D%22node%22%3E%0A%3Ctitle%3E76%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224900.8043%2C-169.7003%204781.7313%2C-169.7003%204781.7313%2C-94.2997%204900.8043%2C-94.2997%204900.8043%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224841.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B3%5D%20%26lt%3B%3D%2055.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224841.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.423%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224841.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20648%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224841.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B197%2C%20451%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2072%26%2345%3B%26gt%3B76%20--%3E%0A%3Cg%20id%3D%22edge76%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E72-%26gt%3B76%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4795.6281%2C-205.5528C4801.0796%2C-196.7671%204806.9468%2C-187.3115%204812.599%2C-178.2025%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224815.6331%2C-179.951%204817.9316%2C-169.6085%204809.6851%2C-176.2602%204815.6331%2C-179.951%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2074%20--%3E%0A%3Cg%20id%3D%22node75%22%20class%3D%22node%22%3E%0A%3Ctitle%3E74%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224638.8043%2C-58.6014%204519.7313%2C-58.6014%204519.7313%2C.2014%204638.8043%2C.2014%204638.8043%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224579.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.475%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224579.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20779%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224579.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B303%2C%20476%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2073%26%2345%3B%26gt%3B74%20--%3E%0A%3Cg%20id%3D%22edge74%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E73-%26gt%3B74%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4657.7989%2C-94.3048C4646.2299%2C-84.7137%204633.8325%2C-74.4359%204622.3846%2C-64.9452%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224624.565%2C-62.2065%204614.6327%2C-58.5186%204620.0974%2C-67.5954%204624.565%2C-62.2065%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2075%20--%3E%0A%3Cg%20id%3D%22node76%22%20class%3D%22node%22%3E%0A%3Ctitle%3E75%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224762.8044%2C-58.6014%204657.7312%2C-58.6014%204657.7312%2C.2014%204762.8044%2C.2014%204762.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224710.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.49%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224710.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2070%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224710.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B40%2C%2030%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2073%26%2345%3B%26gt%3B75%20--%3E%0A%3Cg%20id%3D%22edge75%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E73-%26gt%3B75%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4705.8346%2C-94.3048C4706.406%2C-85.9126%204707.0133%2C-76.9946%204707.5891%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224711.0839%2C-68.7333%204708.2714%2C-58.5186%204704.1%2C-68.2577%204711.0839%2C-68.7333%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2077%20--%3E%0A%3Cg%20id%3D%22node78%22%20class%3D%22node%22%3E%0A%3Ctitle%3E77%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224886.8044%2C-58.6014%204781.7312%2C-58.6014%204781.7312%2C.2014%204886.8044%2C.2014%204886.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224834.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.499%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224834.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2025%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224834.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B13%2C%2012%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2076%26%2345%3B%26gt%3B77%20--%3E%0A%3Cg%20id%3D%22edge77%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E76-%26gt%3B77%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4838.701%2C-94.3048C4838.1296%2C-85.9126%204837.5223%2C-76.9946%204836.9465%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224840.4356%2C-68.2577%204836.2642%2C-58.5186%204833.4517%2C-68.7333%204840.4356%2C-68.2577%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2078%20--%3E%0A%3Cg%20id%3D%22node79%22%20class%3D%22node%22%3E%0A%3Ctitle%3E78%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225024.8043%2C-58.6014%204905.7313%2C-58.6014%204905.7313%2C.2014%205024.8043%2C.2014%205024.8043%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224965.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.416%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224965.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20623%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224965.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B184%2C%20439%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2076%26%2345%3B%26gt%3B78%20--%3E%0A%3Cg%20id%3D%22edge78%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E76-%26gt%3B78%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4886.7367%2C-94.3048C4898.3057%2C-84.7137%204910.7031%2C-74.4359%204922.151%2C-64.9452%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224924.4382%2C-67.5954%204929.9029%2C-58.5186%204919.9706%2C-62.2065%204924.4382%2C-67.5954%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2080%20--%3E%0A%3Cg%20id%3D%22node81%22%20class%3D%22node%22%3E%0A%3Ctitle%3E80%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225344.8043%2C-280.9003%205225.7313%2C-280.9003%205225.7313%2C-205.4997%205344.8043%2C-205.4997%205344.8043%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225285.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B3%5D%20%26lt%3B%3D%2084.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225285.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.287%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225285.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20748%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225285.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B130%2C%20618%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2079%26%2345%3B%26gt%3B80%20--%3E%0A%3Cg%20id%3D%22edge80%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E79-%26gt%3B80%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5285.2678%2C-316.7528C5285.2678%2C-308.42%205285.2678%2C-299.4845%205285.2678%2C-290.8137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225288.7679%2C-290.8084%205285.2678%2C-280.8085%205281.7679%2C-290.8085%205288.7679%2C-290.8084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2087%20--%3E%0A%3Cg%20id%3D%22node88%22%20class%3D%22node%22%3E%0A%3Ctitle%3E87%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225705.8043%2C-280.9003%205586.7313%2C-280.9003%205586.7313%2C-205.4997%205705.8043%2C-205.4997%205705.8043%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225646.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B2%5D%20%26lt%3B%3D%20156.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225646.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.384%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225646.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20491%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225646.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B127%2C%20364%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2079%26%2345%3B%26gt%3B87%20--%3E%0A%3Cg%20id%3D%22edge87%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E79-%26gt%3B87%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5344.8796%2C-336.0376C5408.6318%2C-316.3998%205509.6204%2C-285.2919%205576.8948%2C-264.5692%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225578.0409%2C-267.8785%205586.5674%2C-261.5897%205575.9802%2C-261.1887%205578.0409%2C-267.8785%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2081%20--%3E%0A%3Cg%20id%3D%22node82%22%20class%3D%22node%22%3E%0A%3Ctitle%3E81%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225276.3044%2C-169.7003%205164.2312%2C-169.7003%205164.2312%2C-94.2997%205276.3044%2C-94.2997%205276.3044%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225220.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B3%5D%20%26lt%3B%3D%2080.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225220.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.314%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225220.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20497%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225220.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B97%2C%20400%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2080%26%2345%3B%26gt%3B81%20--%3E%0A%3Cg%20id%3D%22edge81%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E80-%26gt%3B81%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5263.2618%2C-205.5528C5258.1792%2C-196.8577%205252.7131%2C-187.5064%205247.4394%2C-178.4843%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225250.3193%2C-176.4755%205242.2512%2C-169.6085%205244.276%2C-180.008%205250.3193%2C-176.4755%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2084%20--%3E%0A%3Cg%20id%3D%22node85%22%20class%3D%22node%22%3E%0A%3Ctitle%3E84%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225407.3044%2C-169.7003%205295.2312%2C-169.7003%205295.2312%2C-94.2997%205407.3044%2C-94.2997%205407.3044%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225351.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2050.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225351.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.228%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225351.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20251%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225351.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B33%2C%20218%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2080%26%2345%3B%26gt%3B84%20--%3E%0A%3Cg%20id%3D%22edge84%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E80-%26gt%3B84%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5307.6124%2C-205.5528C5312.7731%2C-196.8577%205318.3234%2C-187.5064%205323.6782%2C-178.4843%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225326.852%2C-179.9943%205328.9462%2C-169.6085%205320.8324%2C-176.4215%205326.852%2C-179.9943%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2082%20--%3E%0A%3Cg%20id%3D%22node83%22%20class%3D%22node%22%3E%0A%3Ctitle%3E82%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225155.3044%2C-58.6014%205043.2312%2C-58.6014%205043.2312%2C.2014%205155.3044%2C.2014%205155.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225099.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.285%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225099.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20436%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225099.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B75%2C%20361%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2081%26%2345%3B%26gt%3B82%20--%3E%0A%3Cg%20id%3D%22edge82%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E81-%26gt%3B82%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5175.8989%2C-94.3048C5164.7184%2C-84.806%205152.7449%2C-74.6335%205141.6639%2C-65.2192%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225143.6642%2C-62.326%205133.7771%2C-58.5186%205139.1319%2C-67.6607%205143.6642%2C-62.326%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2083%20--%3E%0A%3Cg%20id%3D%22node84%22%20class%3D%22node%22%3E%0A%3Ctitle%3E83%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225278.8044%2C-58.6014%205173.7312%2C-58.6014%205173.7312%2C.2014%205278.8044%2C.2014%205278.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225226.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.461%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225226.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2061%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225226.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B22%2C%2039%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2081%26%2345%3B%26gt%3B83%20--%3E%0A%3Cg%20id%3D%22edge83%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E81-%26gt%3B83%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5222.4679%2C-94.3048C5222.9577%2C-85.9126%205223.4782%2C-76.9946%205223.9718%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225227.4679%2C-68.7056%205224.5566%2C-58.5186%205220.4798%2C-68.2977%205227.4679%2C-68.7056%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2085%20--%3E%0A%3Cg%20id%3D%22node86%22%20class%3D%22node%22%3E%0A%3Ctitle%3E85%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225395.3045%2C-58.6014%205297.2311%2C-58.6014%205297.2311%2C.2014%205395.3045%2C.2014%205395.3045%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225346.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.098%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225346.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2058%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225346.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B3%2C%2055%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2084%26%2345%3B%26gt%3B85%20--%3E%0A%3Cg%20id%3D%22edge85%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E84-%26gt%3B85%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5349.4344%2C-94.3048C5349.0262%2C-85.9126%205348.5924%2C-76.9946%205348.1811%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225351.6756%2C-68.3368%205347.6938%2C-58.5186%205344.6838%2C-68.6769%205351.6756%2C-68.3368%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2086%20--%3E%0A%3Cg%20id%3D%22node87%22%20class%3D%22node%22%3E%0A%3Ctitle%3E86%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225526.3044%2C-58.6014%205414.2312%2C-58.6014%205414.2312%2C.2014%205526.3044%2C.2014%205526.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225470.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.263%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225470.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20193%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225470.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B30%2C%20163%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2084%26%2345%3B%26gt%3B86%20--%3E%0A%3Cg%20id%3D%22edge86%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E84-%26gt%3B86%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5394.9033%2C-94.3048C5405.899%2C-84.806%205417.6746%2C-74.6335%205428.5724%2C-65.2192%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225431.0496%2C-67.7044%205436.3289%2C-58.5186%205426.4735%2C-62.4073%205431.0496%2C-67.7044%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2088%20--%3E%0A%3Cg%20id%3D%22node89%22%20class%3D%22node%22%3E%0A%3Ctitle%3E88%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225698.8044%2C-169.7003%205593.7312%2C-169.7003%205593.7312%2C-94.2997%205698.8044%2C-94.2997%205698.8044%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225646.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2040.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225646.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.247%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225646.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20104%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225646.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B15%2C%2089%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2087%26%2345%3B%26gt%3B88%20--%3E%0A%3Cg%20id%3D%22edge88%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E87-%26gt%3B88%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5646.2678%2C-205.5528C5646.2678%2C-197.22%205646.2678%2C-188.2845%205646.2678%2C-179.6137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225649.7679%2C-179.6084%205646.2678%2C-169.6085%205642.7679%2C-179.6085%205649.7679%2C-179.6084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2091%20--%3E%0A%3Cg%20id%3D%22node92%22%20class%3D%22node%22%3E%0A%3Ctitle%3E91%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225893.8043%2C-169.7003%205774.7313%2C-169.7003%205774.7313%2C-94.2997%205893.8043%2C-94.2997%205893.8043%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225834.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B3%5D%20%26lt%3B%3D%2083.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225834.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.411%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225834.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20387%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225834.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B112%2C%20275%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2087%26%2345%3B%26gt%3B91%20--%3E%0A%3Cg%20id%3D%22edge91%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E87-%26gt%3B91%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5706.0105%2C-207.8628C5725.1167%2C-196.5617%205746.4105%2C-183.9667%205766.0717%2C-172.3373%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225767.8553%2C-175.3488%205774.6805%2C-167.2452%205764.2916%2C-169.3238%205767.8553%2C-175.3488%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2089%20--%3E%0A%3Cg%20id%3D%22node90%22%20class%3D%22node%22%3E%0A%3Ctitle%3E89%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225635.8046%2C-58.6014%205544.731%2C-58.6014%205544.731%2C.2014%205635.8046%2C.2014%205635.8046%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225590.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.0%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225590.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225590.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1%2C%200%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2088%26%2345%3B%26gt%3B89%20--%3E%0A%3Cg%20id%3D%22edge89%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E88-%26gt%3B89%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5625.7335%2C-94.3048C5620.9609%2C-85.5437%205615.8761%2C-76.2096%205611.0912%2C-67.4259%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225614.0964%2C-65.6259%205606.239%2C-58.5186%205607.9493%2C-68.9745%205614.0964%2C-65.6259%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2090%20--%3E%0A%3Cg%20id%3D%22node91%22%20class%3D%22node%22%3E%0A%3Ctitle%3E90%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225759.8044%2C-58.6014%205654.7312%2C-58.6014%205654.7312%2C.2014%205759.8044%2C.2014%205759.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225707.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.235%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225707.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20103%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225707.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B14%2C%2089%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2088%26%2345%3B%26gt%3B90%20--%3E%0A%3Cg%20id%3D%22edge90%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E88-%26gt%3B90%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5668.6356%2C-94.3048C5673.889%2C-85.4515%205679.4896%2C-76.013%205684.7496%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225687.7774%2C-68.9047%205689.8706%2C-58.5186%205681.7575%2C-65.3325%205687.7774%2C-68.9047%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2092%20--%3E%0A%3Cg%20id%3D%22node93%22%20class%3D%22node%22%3E%0A%3Ctitle%3E92%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225890.3044%2C-58.6014%205778.2312%2C-58.6014%205778.2312%2C.2014%205890.3044%2C.2014%205890.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225834.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.451%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225834.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20239%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225834.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B82%2C%20157%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2091%26%2345%3B%26gt%3B92%20--%3E%0A%3Cg%20id%3D%22edge92%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E91-%26gt%3B92%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5834.2678%2C-94.3048C5834.2678%2C-85.9126%205834.2678%2C-76.9946%205834.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225837.7679%2C-68.5186%205834.2678%2C-58.5186%205830.7679%2C-68.5187%205837.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2093%20--%3E%0A%3Cg%20id%3D%22node94%22%20class%3D%22node%22%3E%0A%3Ctitle%3E93%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226021.3044%2C-58.6014%205909.2312%2C-58.6014%205909.2312%2C.2014%206021.3044%2C.2014%206021.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225965.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.323%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225965.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20148%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225965.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B30%2C%20118%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2091%26%2345%3B%26gt%3B93%20--%3E%0A%3Cg%20id%3D%22edge93%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E91-%26gt%3B93%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5882.3035%2C-94.3048C5894.5256%2C-84.7137%205907.6229%2C-74.4359%205919.717%2C-64.9452%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225922.2003%2C-67.4455%205927.9065%2C-58.5186%205917.8788%2C-61.9387%205922.2003%2C-67.4455%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2095%20--%3E%0A%3Cg%20id%3D%22node96%22%20class%3D%22node%22%3E%0A%3Ctitle%3E95%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226630.8042%2C-392.1003%206497.7314%2C-392.1003%206497.7314%2C-316.6997%206630.8042%2C-316.6997%206630.8042%2C-392.1003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226564.2678%22%20y%3D%22-375.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B7%5D%20%26lt%3B%3D%202.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226564.2678%22%20y%3D%22-358.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.302%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226564.2678%22%20y%3D%22-341.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%206849%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226564.2678%22%20y%3D%22-325%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1269%2C%205580%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2094%26%2345%3B%26gt%3B95%20--%3E%0A%3Cg%20id%3D%22edge95%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E94-%26gt%3B95%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6564.2678%2C-427.9528C6564.2678%2C-419.62%206564.2678%2C-410.6845%206564.2678%2C-402.0137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226567.7679%2C-402.0084%206564.2678%2C-392.0085%206560.7679%2C-402.0085%206567.7679%2C-402.0084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20110%20--%3E%0A%3Cg%20id%3D%22node111%22%20class%3D%22node%22%3E%0A%3Ctitle%3E110%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227356.3042%2C-392.1003%207230.2314%2C-392.1003%207230.2314%2C-316.6997%207356.3042%2C-316.6997%207356.3042%2C-392.1003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227293.2678%22%20y%3D%22-375.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B5%5D%20%26lt%3B%3D%2068.0%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227293.2678%22%20y%3D%22-358.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.235%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227293.2678%22%20y%3D%22-341.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%206673%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227293.2678%22%20y%3D%22-325%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B909%2C%205764%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2094%26%2345%3B%26gt%3B110%20--%3E%0A%3Cg%20id%3D%22edge110%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E94-%26gt%3B110%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6634.4983%2C-454.8872C6772.7962%2C-433.7916%207079.1725%2C-387.0576%207219.9397%2C-365.5853%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227220.8336%2C-368.9895%207230.1915%2C-364.0215%207219.778%2C-362.0696%207220.8336%2C-368.9895%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2096%20--%3E%0A%3Cg%20id%3D%22node97%22%20class%3D%22node%22%3E%0A%3Ctitle%3E96%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226506.8042%2C-280.9003%206373.7314%2C-280.9003%206373.7314%2C-205.4997%206506.8042%2C-205.4997%206506.8042%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226440.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2060.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226440.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.294%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226440.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%206133%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226440.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1099%2C%205034%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2095%26%2345%3B%26gt%3B96%20--%3E%0A%3Cg%20id%3D%22edge96%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E95-%26gt%3B96%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6522.2871%2C-316.7528C6511.8841%2C-307.4237%206500.6388%2C-297.3392%206489.9084%2C-287.7164%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226491.9869%2C-284.8792%206482.2053%2C-280.8085%206487.3135%2C-290.0906%206491.9869%2C-284.8792%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20103%20--%3E%0A%3Cg%20id%3D%22node104%22%20class%3D%22node%22%3E%0A%3Ctitle%3E103%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226761.8043%2C-280.9003%206642.7313%2C-280.9003%206642.7313%2C-205.4997%206761.8043%2C-205.4997%206761.8043%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226702.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2062.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226702.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.362%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226702.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20716%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226702.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B170%2C%20546%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2095%26%2345%3B%26gt%3B103%20--%3E%0A%3Cg%20id%3D%22edge103%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E95-%26gt%3B103%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6610.9883%2C-316.7528C6622.7906%2C-307.2425%206635.5671%2C-296.9473%206647.7176%2C-287.1564%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226650.0048%2C-289.8083%206655.5954%2C-280.8085%206645.6127%2C-284.3576%206650.0048%2C-289.8083%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2097%20--%3E%0A%3Cg%20id%3D%22node98%22%20class%3D%22node%22%3E%0A%3Ctitle%3E97%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226311.3042%2C-169.7003%206185.2314%2C-169.7003%206185.2314%2C-94.2997%206311.3042%2C-94.2997%206311.3042%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226248.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B6%5D%20%26lt%3B%3D%201.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226248.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.303%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226248.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%205034%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226248.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B938%2C%204096%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2096%26%2345%3B%26gt%3B97%20--%3E%0A%3Cg%20id%3D%22edge97%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E96-%26gt%3B97%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6375.2654%2C-205.5528C6357.5954%2C-195.3189%206338.356%2C-184.1761%206320.321%2C-173.7308%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226321.984%2C-170.6493%206311.5764%2C-168.6662%206318.4757%2C-176.7068%206321.984%2C-170.6493%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20100%20--%3E%0A%3Cg%20id%3D%22node101%22%20class%3D%22node%22%3E%0A%3Ctitle%3E100%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226499.8043%2C-169.7003%206380.7313%2C-169.7003%206380.7313%2C-94.2997%206499.8043%2C-94.2997%206499.8043%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226440.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B5%5D%20%26lt%3B%3D%2062.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226440.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.25%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226440.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201099%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226440.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B161%2C%20938%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2096%26%2345%3B%26gt%3B100%20--%3E%0A%3Cg%20id%3D%22edge100%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E96-%26gt%3B100%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6440.2678%2C-205.5528C6440.2678%2C-197.22%206440.2678%2C-188.2845%206440.2678%2C-179.6137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226443.7679%2C-179.6084%206440.2678%2C-169.6085%206436.7679%2C-179.6085%206443.7679%2C-179.6084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2098%20--%3E%0A%3Cg%20id%3D%22node99%22%20class%3D%22node%22%3E%0A%3Ctitle%3E98%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226166.3042%2C-58.6014%206040.2314%2C-58.6014%206040.2314%2C.2014%206166.3042%2C.2014%206166.3042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226103.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.315%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226103.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%203412%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226103.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B668%2C%202744%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2097%26%2345%3B%26gt%3B98%20--%3E%0A%3Cg%20id%3D%22edge98%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E97-%26gt%3B98%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6195.0985%2C-94.3048C6181.3101%2C-84.5293%206166.5153%2C-74.0403%206152.9156%2C-64.3986%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226154.804%2C-61.447%206144.6219%2C-58.5186%206150.7555%2C-67.1575%206154.804%2C-61.447%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2099%20--%3E%0A%3Cg%20id%3D%22node100%22%20class%3D%22node%22%3E%0A%3Ctitle%3E99%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226311.3042%2C-58.6014%206185.2314%2C-58.6014%206185.2314%2C.2014%206311.3042%2C.2014%206311.3042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226248.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.278%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226248.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201622%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226248.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B270%2C%201352%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2097%26%2345%3B%26gt%3B99%20--%3E%0A%3Cg%20id%3D%22edge99%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E97-%26gt%3B99%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6248.2678%2C-94.3048C6248.2678%2C-85.9126%206248.2678%2C-76.9946%206248.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226251.7679%2C-68.5186%206248.2678%2C-58.5186%206244.7679%2C-68.5187%206251.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20101%20--%3E%0A%3Cg%20id%3D%22node102%22%20class%3D%22node%22%3E%0A%3Ctitle%3E101%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226420.8046%2C-58.6014%206329.731%2C-58.6014%206329.731%2C.2014%206420.8046%2C.2014%206420.8046%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226375.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.497%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226375.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2013%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226375.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B6%2C%207%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20100%26%2345%3B%26gt%3B101%20--%3E%0A%3Cg%20id%3D%22edge101%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E100-%26gt%3B101%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6416.4333%2C-94.3048C6410.8354%2C-85.4515%206404.8675%2C-76.013%206399.2626%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226402.1084%2C-65.1003%206393.8058%2C-58.5186%206396.1919%2C-68.8413%206402.1084%2C-65.1003%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20102%20--%3E%0A%3Cg%20id%3D%22node103%22%20class%3D%22node%22%3E%0A%3Ctitle%3E102%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226558.8043%2C-58.6014%206439.7313%2C-58.6014%206439.7313%2C.2014%206558.8043%2C.2014%206558.8043%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226499.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.245%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226499.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201086%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226499.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B155%2C%20931%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20100%26%2345%3B%26gt%3B102%20--%3E%0A%3Cg%20id%3D%22edge102%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E100-%26gt%3B102%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6461.9022%2C-94.3048C6466.9304%2C-85.5437%206472.2876%2C-76.2096%206477.3288%2C-67.4259%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226480.4988%2C-68.9339%206482.441%2C-58.5186%206474.4276%2C-65.4495%206480.4988%2C-68.9339%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20104%20--%3E%0A%3Cg%20id%3D%22node105%22%20class%3D%22node%22%3E%0A%3Ctitle%3E104%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226761.8043%2C-169.7003%206642.7313%2C-169.7003%206642.7313%2C-94.2997%206761.8043%2C-94.2997%206761.8043%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226702.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B3%5D%20%26lt%3B%3D%2090.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226702.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.377%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226702.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20643%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226702.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B162%2C%20481%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20103%26%2345%3B%26gt%3B104%20--%3E%0A%3Cg%20id%3D%22edge104%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E103-%26gt%3B104%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6702.2678%2C-205.5528C6702.2678%2C-197.22%206702.2678%2C-188.2845%206702.2678%2C-179.6137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226705.7679%2C-179.6084%206702.2678%2C-169.6085%206698.7679%2C-179.6085%206705.7679%2C-179.6084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20107%20--%3E%0A%3Cg%20id%3D%22node108%22%20class%3D%22node%22%3E%0A%3Ctitle%3E107%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226944.3045%2C-169.7003%206846.2311%2C-169.7003%206846.2311%2C-94.2997%206944.3045%2C-94.2997%206944.3045%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226895.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B9%5D%20%26lt%3B%3D%200.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226895.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.195%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226895.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2073%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226895.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B8%2C%2065%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20103%26%2345%3B%26gt%3B107%20--%3E%0A%3Cg%20id%3D%22edge107%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E103-%26gt%3B107%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6762.0125%2C-208.7772C6785.965%2C-194.9765%206813.4696%2C-179.1293%206837.2469%2C-165.4297%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226839.2447%2C-168.318%206846.1621%2C-160.293%206835.75%2C-162.2527%206839.2447%2C-168.318%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20105%20--%3E%0A%3Cg%20id%3D%22node106%22%20class%3D%22node%22%3E%0A%3Ctitle%3E105%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226696.8043%2C-58.6014%206577.7313%2C-58.6014%206577.7313%2C.2014%206696.8043%2C.2014%206696.8043%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226637.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.353%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226637.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20489%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226637.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B112%2C%20377%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20104%26%2345%3B%26gt%3B105%20--%3E%0A%3Cg%20id%3D%22edge105%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E104-%26gt%3B105%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6678.4333%2C-94.3048C6672.8354%2C-85.4515%206666.8675%2C-76.013%206661.2626%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226664.1084%2C-65.1003%206655.8058%2C-58.5186%206658.1919%2C-68.8413%206664.1084%2C-65.1003%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20106%20--%3E%0A%3Cg%20id%3D%22node107%22%20class%3D%22node%22%3E%0A%3Ctitle%3E106%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226827.3044%2C-58.6014%206715.2312%2C-58.6014%206715.2312%2C.2014%206827.3044%2C.2014%206827.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226771.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.439%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226771.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20154%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226771.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B50%2C%20104%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20104%26%2345%3B%26gt%3B106%20--%3E%0A%3Cg%20id%3D%22edge106%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E104-%26gt%3B106%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6727.5691%2C-94.3048C6733.5733%2C-85.3593%206739.9786%2C-75.8163%206745.9822%2C-66.8718%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226748.9219%2C-68.7723%206751.589%2C-58.5186%206743.1098%2C-64.8711%206748.9219%2C-68.7723%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20108%20--%3E%0A%3Cg%20id%3D%22node109%22%20class%3D%22node%22%3E%0A%3Ctitle%3E108%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226944.3045%2C-58.6014%206846.2311%2C-58.6014%206846.2311%2C.2014%206944.3045%2C.2014%206944.3045%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226895.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.176%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226895.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2072%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226895.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B7%2C%2065%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20107%26%2345%3B%26gt%3B108%20--%3E%0A%3Cg%20id%3D%22edge108%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E107-%26gt%3B108%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6895.2678%2C-94.3048C6895.2678%2C-85.9126%206895.2678%2C-76.9946%206895.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226898.7679%2C-68.5186%206895.2678%2C-58.5186%206891.7679%2C-68.5187%206898.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20109%20--%3E%0A%3Cg%20id%3D%22node110%22%20class%3D%22node%22%3E%0A%3Ctitle%3E109%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227053.8046%2C-58.6014%206962.731%2C-58.6014%206962.731%2C.2014%207053.8046%2C.2014%207053.8046%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227008.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.0%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227008.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227008.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1%2C%200%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20107%26%2345%3B%26gt%3B109%20--%3E%0A%3Cg%20id%3D%22edge109%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E107-%26gt%3B109%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6936.7032%2C-94.3048C6947.0431%2C-84.8982%206958.1093%2C-74.8309%206968.3731%2C-65.4936%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226970.9983%2C-67.837%206976.0401%2C-58.5186%206966.2878%2C-62.659%206970.9983%2C-67.837%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20111%20--%3E%0A%3Cg%20id%3D%22node112%22%20class%3D%22node%22%3E%0A%3Ctitle%3E111%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227345.8044%2C-280.9003%207240.7312%2C-280.9003%207240.7312%2C-205.4997%207345.8044%2C-205.4997%207345.8044%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227293.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B4%5D%20%26lt%3B%3D%20249.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227293.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.483%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227293.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2066%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227293.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B27%2C%2039%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20110%26%2345%3B%26gt%3B111%20--%3E%0A%3Cg%20id%3D%22edge111%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E110-%26gt%3B111%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7293.2678%2C-316.7528C7293.2678%2C-308.42%207293.2678%2C-299.4845%207293.2678%2C-290.8137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227296.7679%2C-290.8084%207293.2678%2C-280.8085%207289.7679%2C-290.8085%207296.7679%2C-290.8084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20118%20--%3E%0A%3Cg%20id%3D%22node119%22%20class%3D%22node%22%3E%0A%3Ctitle%3E118%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227705.3042%2C-280.9003%207579.2314%2C-280.9003%207579.2314%2C-205.4997%207705.3042%2C-205.4997%207705.3042%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227642.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B7%5D%20%26lt%3B%3D%202.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227642.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.231%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227642.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%206607%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227642.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B882%2C%205725%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20110%26%2345%3B%26gt%3B118%20--%3E%0A%3Cg%20id%3D%22edge118%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E110-%26gt%3B118%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7356.4691%2C-334.2625C7416.3212%2C-315.1922%207506.1547%2C-286.569%207569.0874%2C-266.5171%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227570.4825%2C-269.746%207578.948%2C-263.3752%207568.3574%2C-263.0764%207570.4825%2C-269.746%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20112%20--%3E%0A%3Cg%20id%3D%22node113%22%20class%3D%22node%22%3E%0A%3Ctitle%3E112%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227287.8044%2C-169.7003%207182.7312%2C-169.7003%207182.7312%2C-94.2997%207287.8044%2C-94.2997%207287.8044%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227235.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B3%5D%20%26lt%3B%3D%2058.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227235.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.436%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227235.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2056%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227235.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B18%2C%2038%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20111%26%2345%3B%26gt%3B112%20--%3E%0A%3Cg%20id%3D%22edge112%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E111-%26gt%3B112%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7273.6316%2C-205.5528C7269.0964%2C-196.8577%207264.219%2C-187.5064%207259.5132%2C-178.4843%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227262.6116%2C-176.8563%207254.8837%2C-169.6085%207256.4051%2C-180.0935%207262.6116%2C-176.8563%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20115%20--%3E%0A%3Cg%20id%3D%22node116%22%20class%3D%22node%22%3E%0A%3Ctitle%3E115%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227397.8046%2C-169.7003%207306.731%2C-169.7003%207306.731%2C-94.2997%207397.8046%2C-94.2997%207397.8046%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227352.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B3%5D%20%26lt%3B%3D%2080.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227352.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.18%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227352.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2010%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227352.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B9%2C%201%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20111%26%2345%3B%26gt%3B115%20--%3E%0A%3Cg%20id%3D%22edge115%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E111-%26gt%3B115%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7313.2425%2C-205.5528C7317.8559%2C-196.8577%207322.8175%2C-187.5064%207327.6044%2C-178.4843%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227330.7185%2C-180.0825%207332.3137%2C-169.6085%207324.5349%2C-176.8017%207330.7185%2C-180.0825%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20113%20--%3E%0A%3Cg%20id%3D%22node114%22%20class%3D%22node%22%3E%0A%3Ctitle%3E113%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227163.8046%2C-58.6014%207072.731%2C-58.6014%207072.731%2C.2014%207163.8046%2C.2014%207163.8046%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227118.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.0%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227118.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%208%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227118.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B0%2C%208%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20112%26%2345%3B%26gt%3B113%20--%3E%0A%3Cg%20id%3D%22edge113%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E112-%26gt%3B113%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7192.3657%2C-94.3048C7181.5547%2C-84.806%207169.9771%2C-74.6335%207159.2624%2C-65.2192%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227161.4587%2C-62.4899%207151.6363%2C-58.5186%207156.8384%2C-67.7484%207161.4587%2C-62.4899%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20114%20--%3E%0A%3Cg%20id%3D%22node115%22%20class%3D%22node%22%3E%0A%3Ctitle%3E114%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227287.8044%2C-58.6014%207182.7312%2C-58.6014%207182.7312%2C.2014%207287.8044%2C.2014%207287.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227235.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.469%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227235.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2048%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227235.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B18%2C%2030%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20112%26%2345%3B%26gt%3B114%20--%3E%0A%3Cg%20id%3D%22edge114%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E112-%26gt%3B114%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7235.2678%2C-94.3048C7235.2678%2C-85.9126%207235.2678%2C-76.9946%207235.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227238.7679%2C-68.5186%207235.2678%2C-58.5186%207231.7679%2C-68.5187%207238.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20116%20--%3E%0A%3Cg%20id%3D%22node117%22%20class%3D%22node%22%3E%0A%3Ctitle%3E116%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227397.8046%2C-58.6014%207306.731%2C-58.6014%207306.731%2C.2014%207397.8046%2C.2014%207397.8046%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227352.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.0%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227352.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%209%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227352.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B9%2C%200%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20115%26%2345%3B%26gt%3B116%20--%3E%0A%3Cg%20id%3D%22edge116%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E115-%26gt%3B116%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7352.2678%2C-94.3048C7352.2678%2C-85.9126%207352.2678%2C-76.9946%207352.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227355.7679%2C-68.5186%207352.2678%2C-58.5186%207348.7679%2C-68.5187%207355.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20117%20--%3E%0A%3Cg%20id%3D%22node118%22%20class%3D%22node%22%3E%0A%3Ctitle%3E117%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227507.8046%2C-58.6014%207416.731%2C-58.6014%207416.731%2C.2014%207507.8046%2C.2014%207507.8046%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227462.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.0%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227462.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227462.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B0%2C%201%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20115%26%2345%3B%26gt%3B117%20--%3E%0A%3Cg%20id%3D%22edge117%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E115-%26gt%3B117%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7392.6031%2C-94.3048C7402.6686%2C-84.8982%207413.4409%2C-74.8309%207423.4322%2C-65.4936%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227425.9793%2C-67.9037%207430.8957%2C-58.5186%207421.1998%2C-62.7894%207425.9793%2C-67.9037%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20119%20--%3E%0A%3Cg%20id%3D%22node120%22%20class%3D%22node%22%3E%0A%3Ctitle%3E119%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227705.3042%2C-169.7003%207579.2314%2C-169.7003%207579.2314%2C-94.2997%207705.3042%2C-94.2997%207705.3042%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227642.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B4%5D%20%26lt%3B%3D%2012255.0%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227642.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.225%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227642.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%206014%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227642.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B778%2C%205236%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20118%26%2345%3B%26gt%3B119%20--%3E%0A%3Cg%20id%3D%22edge119%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E118-%26gt%3B119%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7642.2678%2C-205.5528C7642.2678%2C-197.22%207642.2678%2C-188.2845%207642.2678%2C-179.6137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227645.7679%2C-179.6084%207642.2678%2C-169.6085%207638.7679%2C-179.6085%207645.7679%2C-179.6084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20122%20--%3E%0A%3Cg%20id%3D%22node123%22%20class%3D%22node%22%3E%0A%3Ctitle%3E122%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227895.8043%2C-169.7003%207776.7313%2C-169.7003%207776.7313%2C-94.2997%207895.8043%2C-94.2997%207895.8043%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227836.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B2%5D%20%26lt%3B%3D%20166.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227836.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.289%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227836.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20593%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227836.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B104%2C%20489%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20118%26%2345%3B%26gt%3B122%20--%3E%0A%3Cg%20id%3D%22edge122%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E118-%26gt%3B122%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7705.5222%2C-206.9428C7725.4602%2C-195.5145%207747.5712%2C-182.8405%207767.8516%2C-171.2159%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227769.7865%2C-174.141%207776.7218%2C-166.1315%207766.3054%2C-168.068%207769.7865%2C-174.141%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20120%20--%3E%0A%3Cg%20id%3D%22node121%22%20class%3D%22node%22%3E%0A%3Ctitle%3E120%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227652.3042%2C-58.6014%207526.2314%2C-58.6014%207526.2314%2C.2014%207652.3042%2C.2014%207652.3042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227589.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.225%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227589.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%206011%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227589.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B776%2C%205235%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20119%26%2345%3B%26gt%3B120%20--%3E%0A%3Cg%20id%3D%22edge120%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E119-%26gt%3B120%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7622.8335%2C-94.3048C7618.3166%2C-85.5437%207613.5043%2C-76.2096%207608.9757%2C-67.4259%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227612.0768%2C-65.803%207604.3834%2C-58.5186%207605.8551%2C-69.0108%207612.0768%2C-65.803%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20121%20--%3E%0A%3Cg%20id%3D%22node122%22%20class%3D%22node%22%3E%0A%3Ctitle%3E121%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227761.8046%2C-58.6014%207670.731%2C-58.6014%207670.731%2C.2014%207761.8046%2C.2014%207761.8046%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227716.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.444%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227716.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%203%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227716.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B2%2C%201%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20119%26%2345%3B%26gt%3B121%20--%3E%0A%3Cg%20id%3D%22edge121%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E119-%26gt%3B121%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7669.4025%2C-94.3048C7675.8419%2C-85.3593%207682.7113%2C-75.8163%207689.1499%2C-66.8718%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227692.1613%2C-68.6794%207695.1629%2C-58.5186%207686.4801%2C-64.5898%207692.1613%2C-68.6794%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20123%20--%3E%0A%3Cg%20id%3D%22node124%22%20class%3D%22node%22%3E%0A%3Ctitle%3E123%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227892.3044%2C-58.6014%207780.2312%2C-58.6014%207780.2312%2C.2014%207892.3044%2C.2014%207892.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227836.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.242%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227836.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20390%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227836.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B55%2C%20335%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20122%26%2345%3B%26gt%3B123%20--%3E%0A%3Cg%20id%3D%22edge123%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E122-%26gt%3B123%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7836.2678%2C-94.3048C7836.2678%2C-85.9126%207836.2678%2C-76.9946%207836.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227839.7679%2C-68.5186%207836.2678%2C-58.5186%207832.7679%2C-68.5187%207839.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20124%20--%3E%0A%3Cg%20id%3D%22node125%22%20class%3D%22node%22%3E%0A%3Ctitle%3E124%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%228023.3044%2C-58.6014%207911.2312%2C-58.6014%207911.2312%2C.2014%208023.3044%2C.2014%208023.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227967.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.366%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227967.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20203%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227967.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B49%2C%20154%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20122%26%2345%3B%26gt%3B124%20--%3E%0A%3Cg%20id%3D%22edge124%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E122-%26gt%3B124%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7884.3035%2C-94.3048C7896.5256%2C-84.7137%207909.6229%2C-74.4359%207921.717%2C-64.9452%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227924.2003%2C-67.4455%207929.9065%2C-58.5186%207919.8788%2C-61.9387%207924.2003%2C-67.4455%22%2F%3E%0A%3C%2Fg%3E%0A%3C%2Fg%3E%0A%3C%2Fsvg%3E)

[](data:image/svg+xml;charset=utf-8,%3C%3Fxml%20version%3D%221.0%22%20encoding%3D%22UTF-8%22%20standalone%3D%22no%22%3F%3E%3C!DOCTYPE%20svg%20PUBLIC%20%22-%2F%2FW3C%2F%2FDTD%20SVG%201.1%2F%2FEN%22%20%22http%3A%2F%2Fwww.w3.org%2FGraphics%2FSVG%2F1.1%2FDTD%2Fsvg11.dtd%22%3E%3C!--%20Generated%20by%20graphviz%20version%202.40.1%20(20161225.0304)%0A%20--%3E%3C!--%20Title%3A%20Tree%20Pages%3A%201%20--%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20xmlns%3Axlink%3D%22http%3A%2F%2Fwww.w3.org%2F1999%2Fxlink%22%20width%3D%228046pt%22%20height%3D%22734pt%22%20viewBox%3D%220.00%200.00%208045.54%20733.60%22%3E%0A%3Cg%20id%3D%22graph0%22%20class%3D%22graph%22%20transform%3D%22scale(1%201)%20rotate(0)%20translate(4%20729.6)%22%3E%0A%3Ctitle%3ETree%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22%23ffffff%22%20stroke%3D%22transparent%22%20points%3D%22-4%2C4%20-4%2C-729.6%208041.5356%2C-729.6%208041.5356%2C4%20-4%2C4%22%2F%3E%0A%3C!--%200%20--%3E%0A%3Cg%20id%3D%22node1%22%20class%3D%22node%22%3E%0A%3Ctitle%3E0%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224186.8041%2C-725.7003%204039.7315%2C-725.7003%204039.7315%2C-650.2997%204186.8041%2C-650.2997%204186.8041%2C-725.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224113.2678%22%20y%3D%22-709%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B4%5D%20%26lt%3B%3D%20129.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224113.2678%22%20y%3D%22-692.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224113.2678%22%20y%3D%22-675.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2049000%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224113.2678%22%20y%3D%22-658.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B24521%2C%2024479%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%201%20--%3E%0A%3Cg%20id%3D%22node2%22%20class%3D%22node%22%3E%0A%3Ctitle%3E1%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223256.3042%2C-614.5003%203116.2314%2C-614.5003%203116.2314%2C-539.0997%203256.3042%2C-539.0997%203256.3042%2C-614.5003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223186.2678%22%20y%3D%22-597.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2054.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223186.2678%22%20y%3D%22-581%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.435%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223186.2678%22%20y%3D%22-564.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2028945%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223186.2678%22%20y%3D%22-547.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B19707%2C%209238%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%200%26%2345%3B%26gt%3B1%20--%3E%0A%3Cg%20id%3D%22edge1%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E0-%26gt%3B1%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4039.3641%2C-679.1347C3867.9013%2C-658.5666%203444.163%2C-607.7363%203266.5901%2C-586.4352%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223266.8762%2C-582.9445%203256.5305%2C-585.2285%203266.0424%2C-589.8947%203266.8762%2C-582.9445%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223271.9769%22%20y%3D%22-600.6858%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3ETrue%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2062%20--%3E%0A%3Cg%20id%3D%22node63%22%20class%3D%22node%22%3E%0A%3Ctitle%3E62%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225101.3042%2C-614.5003%204961.2314%2C-614.5003%204961.2314%2C-539.0997%205101.3042%2C-539.0997%205101.3042%2C-614.5003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225031.2678%22%20y%3D%22-597.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B4%5D%20%26lt%3B%3D%20138.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225031.2678%22%20y%3D%22-581%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.365%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225031.2678%22%20y%3D%22-564.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2020055%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225031.2678%22%20y%3D%22-547.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B4814%2C%2015241%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%200%26%2345%3B%26gt%3B62%20--%3E%0A%3Cg%20id%3D%22edge62%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E0-%26gt%3B62%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4186.8398%2C-679.088C4356.7064%2C-658.5116%204775.1202%2C-607.8279%204951.1651%2C-586.5031%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224951.6335%2C-589.972%204961.14%2C-585.2948%204950.7917%2C-583.0228%204951.6335%2C-589.972%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224945.7165%22%20y%3D%22-600.77%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EFalse%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%202%20--%3E%0A%3Cg%20id%3D%22node3%22%20class%3D%22node%22%3E%0A%3Ctitle%3E2%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221641.3042%2C-503.3003%201501.2314%2C-503.3003%201501.2314%2C-427.8997%201641.3042%2C-427.8997%201641.3042%2C-503.3003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221571.2678%22%20y%3D%22-486.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B6%5D%20%26lt%3B%3D%202.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221571.2678%22%20y%3D%22-469.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.364%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221571.2678%22%20y%3D%22-453%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2017925%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221571.2678%22%20y%3D%22-436.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B13643%2C%204282%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%201%26%2345%3B%26gt%3B2%20--%3E%0A%3Cg%20id%3D%22edge2%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E1-%26gt%3B2%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3116.0951%2C-571.9683C2853.6391%2C-553.897%201930.6277%2C-490.3435%201651.6968%2C-471.1379%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221651.8702%2C-467.6416%201641.6534%2C-470.4464%201651.3893%2C-474.6251%201651.8702%2C-467.6416%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2033%20--%3E%0A%3Cg%20id%3D%22node34%22%20class%3D%22node%22%3E%0A%3Ctitle%3E33%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223252.8042%2C-503.3003%203119.7314%2C-503.3003%203119.7314%2C-427.8997%203252.8042%2C-427.8997%203252.8042%2C-503.3003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223186.2678%22%20y%3D%22-486.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B6%5D%20%26lt%3B%3D%202.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223186.2678%22%20y%3D%22-469.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.495%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223186.2678%22%20y%3D%22-453%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2011020%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223186.2678%22%20y%3D%22-436.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B6064%2C%204956%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%201%26%2345%3B%26gt%3B33%20--%3E%0A%3Cg%20id%3D%22edge33%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E1-%26gt%3B33%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3186.2678%2C-539.1528C3186.2678%2C-530.82%203186.2678%2C-521.8845%203186.2678%2C-513.2137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223189.7679%2C-513.2084%203186.2678%2C-503.2085%203182.7679%2C-513.2085%203189.7679%2C-513.2084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%203%20--%3E%0A%3Cg%20id%3D%22node4%22%20class%3D%22node%22%3E%0A%3Ctitle%3E3%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22896.3042%2C-392.1003%20756.2314%2C-392.1003%20756.2314%2C-316.6997%20896.3042%2C-316.6997%20896.3042%2C-392.1003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22826.2678%22%20y%3D%22-375.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2043.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22826.2678%22%20y%3D%22-358.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.348%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22826.2678%22%20y%3D%22-341.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2017161%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22826.2678%22%20y%3D%22-325%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B13316%2C%203845%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%202%26%2345%3B%26gt%3B3%20--%3E%0A%3Cg%20id%3D%22edge3%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E2-%26gt%3B3%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1501.18%2C-455.1386C1362.4286%2C-434.4283%201053.5179%2C-388.3197%20906.7245%2C-366.4091%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22906.8472%2C-362.8887%20896.4401%2C-364.874%20905.8138%2C-369.812%20906.8472%2C-362.8887%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2018%20--%3E%0A%3Cg%20id%3D%22node19%22%20class%3D%22node%22%3E%0A%3Ctitle%3E18%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221630.8043%2C-392.1003%201511.7313%2C-392.1003%201511.7313%2C-316.6997%201630.8043%2C-316.6997%201630.8043%2C-392.1003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221571.2678%22%20y%3D%22-375.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B7%5D%20%26lt%3B%3D%202.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221571.2678%22%20y%3D%22-358.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.49%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221571.2678%22%20y%3D%22-341.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20764%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221571.2678%22%20y%3D%22-325%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B327%2C%20437%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%202%26%2345%3B%26gt%3B18%20--%3E%0A%3Cg%20id%3D%22edge18%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E2-%26gt%3B18%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1571.2678%2C-427.9528C1571.2678%2C-419.62%201571.2678%2C-410.6845%201571.2678%2C-402.0137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221574.7679%2C-402.0084%201571.2678%2C-392.0085%201567.7679%2C-402.0085%201574.7679%2C-402.0084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%204%20--%3E%0A%3Cg%20id%3D%22node5%22%20class%3D%22node%22%3E%0A%3Ctitle%3E4%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22474.3042%2C-280.9003%20348.2314%2C-280.9003%20348.2314%2C-205.4997%20474.3042%2C-205.4997%20474.3042%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22411.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B6%5D%20%26lt%3B%3D%201.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22411.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.239%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22411.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%204568%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22411.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B3935%2C%20633%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%203%26%2345%3B%26gt%3B4%20--%3E%0A%3Cg%20id%3D%22edge4%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E3-%26gt%3B4%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M755.8687%2C-335.5364C680.4%2C-315.3144%20561.1354%2C-283.3573%20484.5668%2C-262.8406%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22485.0847%2C-259.356%20474.5196%2C-260.1484%20483.2729%2C-266.1174%20485.0847%2C-259.356%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2011%20--%3E%0A%3Cg%20id%3D%22node12%22%20class%3D%22node%22%3E%0A%3Ctitle%3E11%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22892.8042%2C-280.9003%20759.7314%2C-280.9003%20759.7314%2C-205.4997%20892.8042%2C-205.4997%20892.8042%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22826.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B4%5D%20%26lt%3B%3D%20119.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22826.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.38%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22826.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2012593%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22826.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B9381%2C%203212%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%203%26%2345%3B%26gt%3B11%20--%3E%0A%3Cg%20id%3D%22edge11%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E3-%26gt%3B11%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M826.2678%2C-316.7528C826.2678%2C-308.42%20826.2678%2C-299.4845%20826.2678%2C-290.8137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22829.7679%2C-290.8084%20826.2678%2C-280.8085%20822.7679%2C-290.8085%20829.7679%2C-290.8084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%205%20--%3E%0A%3Cg%20id%3D%22node6%22%20class%3D%22node%22%3E%0A%3Ctitle%3E5%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22271.3042%2C-169.7003%20145.2314%2C-169.7003%20145.2314%2C-94.2997%20271.3042%2C-94.2997%20271.3042%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22208.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B4%5D%20%26lt%3B%3D%20113.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22208.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.214%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22208.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%204121%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22208.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B3618%2C%20503%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%204%26%2345%3B%26gt%3B5%20--%3E%0A%3Cg%20id%3D%22edge5%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E4-%26gt%3B5%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M348.1501%2C-208.6252C326.6682%2C-196.8577%20302.4962%2C-183.6167%20280.3653%2C-171.4938%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22281.7983%2C-168.2881%20271.3465%2C-166.5534%20278.4353%2C-174.4274%20281.7983%2C-168.2881%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%208%20--%3E%0A%3Cg%20id%3D%22node9%22%20class%3D%22node%22%3E%0A%3Ctitle%3E8%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22470.8043%2C-169.7003%20351.7313%2C-169.7003%20351.7313%2C-94.2997%20470.8043%2C-94.2997%20470.8043%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22411.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B3%5D%20%26lt%3B%3D%2081.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22411.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.412%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22411.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20447%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22411.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B317%2C%20130%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%204%26%2345%3B%26gt%3B8%20--%3E%0A%3Cg%20id%3D%22edge8%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E4-%26gt%3B8%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M411.2678%2C-205.5528C411.2678%2C-197.22%20411.2678%2C-188.2845%20411.2678%2C-179.6137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22414.7679%2C-179.6084%20411.2678%2C-169.6085%20407.7679%2C-179.6085%20414.7679%2C-179.6084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%206%20--%3E%0A%3Cg%20id%3D%22node7%22%20class%3D%22node%22%3E%0A%3Ctitle%3E6%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22126.3042%2C-58.6014%20.2314%2C-58.6014%20.2314%2C.2014%20126.3042%2C.2014%20126.3042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%2263.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.154%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%2263.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201741%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%2263.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1595%2C%20146%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%205%26%2345%3B%26gt%3B6%20--%3E%0A%3Cg%20id%3D%22edge6%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E5-%26gt%3B6%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M155.0985%2C-94.3048C141.3101%2C-84.5293%20126.5153%2C-74.0403%20112.9156%2C-64.3986%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22114.804%2C-61.447%20104.6219%2C-58.5186%20110.7555%2C-67.1575%20114.804%2C-61.447%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%207%20--%3E%0A%3Cg%20id%3D%22node8%22%20class%3D%22node%22%3E%0A%3Ctitle%3E7%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22271.3042%2C-58.6014%20145.2314%2C-58.6014%20145.2314%2C.2014%20271.3042%2C.2014%20271.3042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22208.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.255%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22208.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%202380%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22208.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B2023%2C%20357%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%205%26%2345%3B%26gt%3B7%20--%3E%0A%3Cg%20id%3D%22edge7%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E5-%26gt%3B7%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M208.2678%2C-94.3048C208.2678%2C-85.9126%20208.2678%2C-76.9946%20208.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22211.7679%2C-68.5186%20208.2678%2C-58.5186%20204.7679%2C-68.5187%20211.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%209%20--%3E%0A%3Cg%20id%3D%22node10%22%20class%3D%22node%22%3E%0A%3Ctitle%3E9%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22402.3044%2C-58.6014%20290.2312%2C-58.6014%20290.2312%2C.2014%20402.3044%2C.2014%20402.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22346.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.374%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22346.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20338%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22346.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B254%2C%2084%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%208%26%2345%3B%26gt%3B9%20--%3E%0A%3Cg%20id%3D%22edge9%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E8-%26gt%3B9%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M387.4333%2C-94.3048C381.8354%2C-85.4515%20375.8675%2C-76.013%20370.2626%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22373.1084%2C-65.1003%20364.8058%2C-58.5186%20367.1919%2C-68.8413%20373.1084%2C-65.1003%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2010%20--%3E%0A%3Cg%20id%3D%22node11%22%20class%3D%22node%22%3E%0A%3Ctitle%3E10%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22525.8044%2C-58.6014%20420.7312%2C-58.6014%20420.7312%2C.2014%20525.8044%2C.2014%20525.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22473.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.488%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22473.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20109%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22473.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B63%2C%2046%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%208%26%2345%3B%26gt%3B10%20--%3E%0A%3Cg%20id%3D%22edge10%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E8-%26gt%3B10%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M434.0023%2C-94.3048C439.3418%2C-85.4515%20445.0343%2C-76.013%20450.3805%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22453.4179%2C-68.8894%20455.5854%2C-58.5186%20447.4237%2C-65.2742%20453.4179%2C-68.8894%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2012%20--%3E%0A%3Cg%20id%3D%22node13%22%20class%3D%22node%22%3E%0A%3Ctitle%3E12%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22815.3042%2C-169.7003%20689.2314%2C-169.7003%20689.2314%2C-94.2997%20815.3042%2C-94.2997%20815.3042%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22752.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B3%5D%20%26lt%3B%3D%2064.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22752.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.323%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22752.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%204152%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22752.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B3312%2C%20840%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2011%26%2345%3B%26gt%3B12%20--%3E%0A%3Cg%20id%3D%22edge12%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E11-%26gt%3B12%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M801.2148%2C-205.5528C795.3682%2C-196.7671%20789.0758%2C-187.3115%20783.0141%2C-178.2025%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22785.749%2C-175.9946%20777.295%2C-169.6085%20779.9214%2C-179.8726%20785.749%2C-175.9946%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2015%20--%3E%0A%3Cg%20id%3D%22node16%22%20class%3D%22node%22%3E%0A%3Ctitle%3E15%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22966.8042%2C-169.7003%20833.7314%2C-169.7003%20833.7314%2C-94.2997%20966.8042%2C-94.2997%20966.8042%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22900.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B6%5D%20%26lt%3B%3D%201.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22900.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.404%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22900.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%208441%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22900.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B6069%2C%202372%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2011%26%2345%3B%26gt%3B15%20--%3E%0A%3Cg%20id%3D%22edge15%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E11-%26gt%3B15%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M851.3208%2C-205.5528C857.1674%2C-196.7671%20863.4598%2C-187.3115%20869.5215%2C-178.2025%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22872.6142%2C-179.8726%20875.2406%2C-169.6085%20866.7866%2C-175.9946%20872.6142%2C-179.8726%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2013%20--%3E%0A%3Cg%20id%3D%22node14%22%20class%3D%22node%22%3E%0A%3Ctitle%3E13%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22670.3042%2C-58.6014%20544.2314%2C-58.6014%20544.2314%2C.2014%20670.3042%2C.2014%20670.3042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22607.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.249%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22607.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201599%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22607.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1366%2C%20233%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2012%26%2345%3B%26gt%3B13%20--%3E%0A%3Cg%20id%3D%22edge13%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E12-%26gt%3B13%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M699.0985%2C-94.3048C685.3101%2C-84.5293%20670.5153%2C-74.0403%20656.9156%2C-64.3986%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22658.804%2C-61.447%20648.6219%2C-58.5186%20654.7555%2C-67.1575%20658.804%2C-61.447%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2014%20--%3E%0A%3Cg%20id%3D%22node15%22%20class%3D%22node%22%3E%0A%3Ctitle%3E14%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22815.3042%2C-58.6014%20689.2314%2C-58.6014%20689.2314%2C.2014%20815.3042%2C.2014%20815.3042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22752.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.362%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22752.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%202553%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22752.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1946%2C%20607%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2012%26%2345%3B%26gt%3B14%20--%3E%0A%3Cg%20id%3D%22edge14%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E12-%26gt%3B14%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M752.2678%2C-94.3048C752.2678%2C-85.9126%20752.2678%2C-76.9946%20752.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22755.7679%2C-68.5186%20752.2678%2C-58.5186%20748.7679%2C-68.5187%20755.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2016%20--%3E%0A%3Cg%20id%3D%22node17%22%20class%3D%22node%22%3E%0A%3Ctitle%3E16%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22966.8042%2C-58.6014%20833.7314%2C-58.6014%20833.7314%2C.2014%20966.8042%2C.2014%20966.8042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22900.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.394%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22900.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%207646%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22900.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B5584%2C%202062%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2015%26%2345%3B%26gt%3B16%20--%3E%0A%3Cg%20id%3D%22edge16%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E15-%26gt%3B16%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M900.2678%2C-94.3048C900.2678%2C-85.9126%20900.2678%2C-76.9946%20900.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22903.7679%2C-68.5186%20900.2678%2C-58.5186%20896.7679%2C-68.5187%20903.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2017%20--%3E%0A%3Cg%20id%3D%22node18%22%20class%3D%22node%22%3E%0A%3Ctitle%3E17%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221104.8043%2C-58.6014%20985.7313%2C-58.6014%20985.7313%2C.2014%201104.8043%2C.2014%201104.8043%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221045.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.476%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221045.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20795%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221045.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B485%2C%20310%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2015%26%2345%3B%26gt%3B17%20--%3E%0A%3Cg%20id%3D%22edge17%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E15-%26gt%3B17%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M953.4371%2C-94.3048C967.2255%2C-84.5293%20982.0203%2C-74.0403%20995.62%2C-64.3986%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22997.7801%2C-67.1575%201003.9137%2C-58.5186%20993.7316%2C-61.447%20997.7801%2C-67.1575%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2019%20--%3E%0A%3Cg%20id%3D%22node20%22%20class%3D%22node%22%3E%0A%3Ctitle%3E19%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221533.3044%2C-280.9003%201421.2312%2C-280.9003%201421.2312%2C-205.4997%201533.3044%2C-205.4997%201533.3044%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221477.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B5%5D%20%26lt%3B%3D%2071.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221477.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.368%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221477.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20325%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221477.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B79%2C%20246%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2018%26%2345%3B%26gt%3B19%20--%3E%0A%3Cg%20id%3D%22edge19%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E18-%26gt%3B19%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1539.4437%2C-316.7528C1531.7873%2C-307.6954%201523.529%2C-297.926%201515.6103%2C-288.5584%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221518.1879%2C-286.186%201509.0591%2C-280.8085%201512.8419%2C-290.705%201518.1879%2C-286.186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2026%20--%3E%0A%3Cg%20id%3D%22node27%22%20class%3D%22node%22%3E%0A%3Ctitle%3E26%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221789.8043%2C-280.9003%201670.7313%2C-280.9003%201670.7313%2C-205.4997%201789.8043%2C-205.4997%201789.8043%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221730.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B3%5D%20%26lt%3B%3D%2080.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221730.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.492%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221730.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20439%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221730.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B248%2C%20191%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2018%26%2345%3B%26gt%3B26%20--%3E%0A%3Cg%20id%3D%22edge26%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E18-%26gt%3B26%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1625.0979%2C-316.7528C1638.9552%2C-307.0614%201653.9781%2C-296.5548%201668.2157%2C-286.5974%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221670.3043%2C-289.4078%201676.4931%2C-280.8085%201666.2924%2C-283.6715%201670.3043%2C-289.4078%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2020%20--%3E%0A%3Cg%20id%3D%22node21%22%20class%3D%22node%22%3E%0A%3Ctitle%3E20%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221338.8044%2C-169.7003%201233.7312%2C-169.7003%201233.7312%2C-94.2997%201338.8044%2C-94.2997%201338.8044%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221286.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B2%5D%20%26lt%3B%3D%20152.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221286.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.477%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221286.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2084%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221286.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B33%2C%2051%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2019%26%2345%3B%26gt%3B20%20--%3E%0A%3Cg%20id%3D%22edge20%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E19-%26gt%3B20%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1420.9946%2C-210.4378C1397.967%2C-197.0312%201371.2566%2C-181.4804%201347.7207%2C-167.7778%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221349.2732%2C-164.6317%201338.8701%2C-162.625%201345.7512%2C-170.6812%201349.2732%2C-164.6317%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2023%20--%3E%0A%3Cg%20id%3D%22node24%22%20class%3D%22node%22%3E%0A%3Ctitle%3E23%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221533.3044%2C-169.7003%201421.2312%2C-169.7003%201421.2312%2C-94.2997%201533.3044%2C-94.2997%201533.3044%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221477.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2051.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221477.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.309%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221477.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20241%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221477.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B46%2C%20195%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2019%26%2345%3B%26gt%3B23%20--%3E%0A%3Cg%20id%3D%22edge23%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E19-%26gt%3B23%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1477.2678%2C-205.5528C1477.2678%2C-197.22%201477.2678%2C-188.2845%201477.2678%2C-179.6137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221480.7679%2C-179.6084%201477.2678%2C-169.6085%201473.7679%2C-179.6085%201480.7679%2C-179.6084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2021%20--%3E%0A%3Cg%20id%3D%22node22%22%20class%3D%22node%22%3E%0A%3Ctitle%3E21%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221214.8046%2C-58.6014%201123.731%2C-58.6014%201123.731%2C.2014%201214.8046%2C.2014%201214.8046%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221169.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.278%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221169.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%206%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221169.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B5%2C%201%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2020%26%2345%3B%26gt%3B21%20--%3E%0A%3Cg%20id%3D%22edge21%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E20-%26gt%3B21%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1243.3657%2C-94.3048C1232.5547%2C-84.806%201220.9771%2C-74.6335%201210.2624%2C-65.2192%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221212.4587%2C-62.4899%201202.6363%2C-58.5186%201207.8384%2C-67.7484%201212.4587%2C-62.4899%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2022%20--%3E%0A%3Cg%20id%3D%22node23%22%20class%3D%22node%22%3E%0A%3Ctitle%3E22%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221338.8044%2C-58.6014%201233.7312%2C-58.6014%201233.7312%2C.2014%201338.8044%2C.2014%201338.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221286.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.46%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221286.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2078%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221286.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B28%2C%2050%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2020%26%2345%3B%26gt%3B22%20--%3E%0A%3Cg%20id%3D%22edge22%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E20-%26gt%3B22%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1286.2678%2C-94.3048C1286.2678%2C-85.9126%201286.2678%2C-76.9946%201286.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221289.7679%2C-68.5186%201286.2678%2C-58.5186%201282.7679%2C-68.5187%201289.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2024%20--%3E%0A%3Cg%20id%3D%22node25%22%20class%3D%22node%22%3E%0A%3Ctitle%3E24%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221469.3044%2C-58.6014%201357.2312%2C-58.6014%201357.2312%2C.2014%201469.3044%2C.2014%201469.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221413.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.259%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221413.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20170%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221413.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B26%2C%20144%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2023%26%2345%3B%26gt%3B24%20--%3E%0A%3Cg%20id%3D%22edge24%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E23-%26gt%3B24%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1453.8%2C-94.3048C1448.2882%2C-85.4515%201442.4121%2C-76.013%201436.8934%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221439.7771%2C-65.1581%201431.5206%2C-58.5186%201433.8346%2C-68.8577%201439.7771%2C-65.1581%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2025%20--%3E%0A%3Cg%20id%3D%22node26%22%20class%3D%22node%22%3E%0A%3Ctitle%3E25%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221592.8044%2C-58.6014%201487.7312%2C-58.6014%201487.7312%2C.2014%201592.8044%2C.2014%201592.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221540.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.405%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221540.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2071%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221540.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B20%2C%2051%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2023%26%2345%3B%26gt%3B25%20--%3E%0A%3Cg%20id%3D%22edge25%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E23-%26gt%3B25%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1500.3689%2C-94.3048C1505.7946%2C-85.4515%201511.5789%2C-76.013%201517.0113%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221520.0591%2C-68.8737%201522.3002%2C-58.5186%201514.0907%2C-65.216%201520.0591%2C-68.8737%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2027%20--%3E%0A%3Cg%20id%3D%22node28%22%20class%3D%22node%22%3E%0A%3Ctitle%3E27%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221789.8043%2C-169.7003%201670.7313%2C-169.7003%201670.7313%2C-94.2997%201789.8043%2C-94.2997%201789.8043%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221730.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B5%5D%20%26lt%3B%3D%2074.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221730.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.469%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221730.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20348%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221730.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B217%2C%20131%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2026%26%2345%3B%26gt%3B27%20--%3E%0A%3Cg%20id%3D%22edge27%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E26-%26gt%3B27%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1730.2678%2C-205.5528C1730.2678%2C-197.22%201730.2678%2C-188.2845%201730.2678%2C-179.6137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221733.7679%2C-179.6084%201730.2678%2C-169.6085%201726.7679%2C-179.6085%201733.7679%2C-179.6084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2030%20--%3E%0A%3Cg%20id%3D%22node31%22%20class%3D%22node%22%3E%0A%3Ctitle%3E30%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221978.8044%2C-169.7003%201873.7312%2C-169.7003%201873.7312%2C-94.2997%201978.8044%2C-94.2997%201978.8044%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221926.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B2%5D%20%26lt%3B%3D%20171.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221926.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.449%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221926.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2091%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221926.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B31%2C%2060%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2026%26%2345%3B%26gt%3B30%20--%3E%0A%3Cg%20id%3D%22edge30%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E26-%26gt%3B30%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1789.8726%2C-209.3834C1813.5411%2C-195.9552%201840.7588%2C-180.5133%201864.6246%2C-166.9731%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221866.6223%2C-169.8638%201873.5929%2C-161.885%201863.168%2C-163.7754%201866.6223%2C-169.8638%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2028%20--%3E%0A%3Cg%20id%3D%22node29%22%20class%3D%22node%22%3E%0A%3Ctitle%3E28%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221716.8044%2C-58.6014%201611.7312%2C-58.6014%201611.7312%2C.2014%201716.8044%2C.2014%201716.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221664.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.366%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221664.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2087%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221664.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B66%2C%2021%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2027%26%2345%3B%26gt%3B28%20--%3E%0A%3Cg%20id%3D%22edge28%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E27-%26gt%3B28%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1706.0666%2C-94.3048C1700.3826%2C-85.4515%201694.3228%2C-76.013%201688.6317%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221691.4389%2C-65.0427%201683.091%2C-58.5186%201685.5485%2C-68.8246%201691.4389%2C-65.0427%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2029%20--%3E%0A%3Cg%20id%3D%22node30%22%20class%3D%22node%22%3E%0A%3Ctitle%3E29%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221854.8043%2C-58.6014%201735.7313%2C-58.6014%201735.7313%2C.2014%201854.8043%2C.2014%201854.8043%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221795.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.488%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221795.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20261%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221795.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B151%2C%20110%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2027%26%2345%3B%26gt%3B29%20--%3E%0A%3Cg%20id%3D%22edge29%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E27-%26gt%3B29%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1754.1023%2C-94.3048C1759.7002%2C-85.4515%201765.6681%2C-76.013%201771.273%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221774.3437%2C-68.8413%201776.7298%2C-58.5186%201768.4272%2C-65.1003%201774.3437%2C-68.8413%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2031%20--%3E%0A%3Cg%20id%3D%22node32%22%20class%3D%22node%22%3E%0A%3Ctitle%3E31%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221978.8044%2C-58.6014%201873.7312%2C-58.6014%201873.7312%2C.2014%201978.8044%2C.2014%201978.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221926.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.313%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221926.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2067%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221926.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B13%2C%2054%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2030%26%2345%3B%26gt%3B31%20--%3E%0A%3Cg%20id%3D%22edge31%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E30-%26gt%3B31%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1926.2678%2C-94.3048C1926.2678%2C-85.9126%201926.2678%2C-76.9946%201926.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221929.7679%2C-68.5186%201926.2678%2C-58.5186%201922.7679%2C-68.5187%201929.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2032%20--%3E%0A%3Cg%20id%3D%22node33%22%20class%3D%22node%22%3E%0A%3Ctitle%3E32%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%222095.3045%2C-58.6014%201997.2311%2C-58.6014%201997.2311%2C.2014%202095.3045%2C.2014%202095.3045%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222046.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.375%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222046.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2024%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222046.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B18%2C%206%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2030%26%2345%3B%26gt%3B32%20--%3E%0A%3Cg%20id%3D%22edge32%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E30-%26gt%3B32%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1970.27%2C-94.3048C1981.3581%2C-84.806%201993.2326%2C-74.6335%202004.222%2C-65.2192%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222006.7264%2C-67.6825%202012.0437%2C-58.5186%202002.1723%2C-62.3665%202006.7264%2C-67.6825%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2034%20--%3E%0A%3Cg%20id%3D%22node35%22%20class%3D%22node%22%3E%0A%3Ctitle%3E34%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%222980.8042%2C-392.1003%202847.7314%2C-392.1003%202847.7314%2C-316.6997%202980.8042%2C-316.6997%202980.8042%2C-392.1003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222914.2678%22%20y%3D%22-375.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2060.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222914.2678%22%20y%3D%22-358.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.487%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222914.2678%22%20y%3D%22-341.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%209877%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222914.2678%22%20y%3D%22-325%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B5728%2C%204149%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2033%26%2345%3B%26gt%3B34%20--%3E%0A%3Cg%20id%3D%22edge34%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E33-%26gt%3B34%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3119.7291%2C-438.3974C3080.6252%2C-422.4108%203031.0484%2C-402.1426%202990.3916%2C-385.5212%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222991.6431%2C-382.2517%202981.0623%2C-381.7071%202988.9941%2C-388.7311%202991.6431%2C-382.2517%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2049%20--%3E%0A%3Cg%20id%3D%22node50%22%20class%3D%22node%22%3E%0A%3Ctitle%3E49%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223514.8043%2C-392.1003%203395.7313%2C-392.1003%203395.7313%2C-316.6997%203514.8043%2C-316.6997%203514.8043%2C-392.1003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223455.2678%22%20y%3D%22-375.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B3%5D%20%26lt%3B%3D%2070.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223455.2678%22%20y%3D%22-358.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.415%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223455.2678%22%20y%3D%22-341.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201143%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223455.2678%22%20y%3D%22-325%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B336%2C%20807%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2033%26%2345%3B%26gt%3B49%20--%3E%0A%3Cg%20id%3D%22edge49%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E33-%26gt%3B49%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3253.1078%2C-437.9695C3293.672%2C-421.2009%203345.3384%2C-399.8429%203386.2851%2C-382.9163%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223387.7438%2C-386.1006%203395.6482%2C-379.0457%203385.0695%2C-379.6315%203387.7438%2C-386.1006%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2035%20--%3E%0A%3Cg%20id%3D%22node36%22%20class%3D%22node%22%3E%0A%3Ctitle%3E35%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%222589.8042%2C-280.9003%202456.7314%2C-280.9003%202456.7314%2C-205.4997%202589.8042%2C-205.4997%202589.8042%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222523.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B4%5D%20%26lt%3B%3D%20119.0%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222523.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.472%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222523.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%207292%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222523.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B4511%2C%202781%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2034%26%2345%3B%26gt%3B35%20--%3E%0A%3Cg%20id%3D%22edge35%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E34-%26gt%3B35%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M2847.4968%2C-335.4104C2778.8567%2C-315.8892%202672.1052%2C-285.5292%202600.0237%2C-265.0293%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222600.6664%2C-261.5734%202590.0904%2C-262.2043%202598.7515%2C-268.3064%202600.6664%2C-261.5734%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2042%20--%3E%0A%3Cg%20id%3D%22node43%22%20class%3D%22node%22%3E%0A%3Ctitle%3E42%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%222980.8042%2C-280.9003%202847.7314%2C-280.9003%202847.7314%2C-205.4997%202980.8042%2C-205.4997%202980.8042%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222914.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B3%5D%20%26lt%3B%3D%2052.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222914.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.498%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222914.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%202585%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222914.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1217%2C%201368%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2034%26%2345%3B%26gt%3B42%20--%3E%0A%3Cg%20id%3D%22edge42%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E34-%26gt%3B42%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M2914.2678%2C-316.7528C2914.2678%2C-308.42%202914.2678%2C-299.4845%202914.2678%2C-290.8137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222917.7679%2C-290.8084%202914.2678%2C-280.8085%202910.7679%2C-290.8085%202917.7679%2C-290.8084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2036%20--%3E%0A%3Cg%20id%3D%22node37%22%20class%3D%22node%22%3E%0A%3Ctitle%3E36%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%222374.3042%2C-169.7003%202248.2314%2C-169.7003%202248.2314%2C-94.2997%202374.3042%2C-94.2997%202374.3042%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222311.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B3%5D%20%26lt%3B%3D%2091.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222311.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.415%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222311.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201854%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222311.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1310%2C%20544%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2035%26%2345%3B%26gt%3B36%20--%3E%0A%3Cg%20id%3D%22edge36%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E35-%26gt%3B36%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M2456.4807%2C-208.1683C2433.2345%2C-195.975%202407.0595%2C-182.2454%202383.3999%2C-169.8353%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222384.9481%2C-166.6952%202374.4666%2C-165.1496%202381.6965%2C-172.8942%202384.9481%2C-166.6952%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2039%20--%3E%0A%3Cg%20id%3D%22node40%22%20class%3D%22node%22%3E%0A%3Ctitle%3E39%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%222589.8042%2C-169.7003%202456.7314%2C-169.7003%202456.7314%2C-94.2997%202589.8042%2C-94.2997%202589.8042%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222523.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B10%5D%20%26lt%3B%3D%200.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222523.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.484%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222523.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%205438%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222523.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B3201%2C%202237%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2035%26%2345%3B%26gt%3B39%20--%3E%0A%3Cg%20id%3D%22edge39%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E35-%26gt%3B39%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M2523.2678%2C-205.5528C2523.2678%2C-197.22%202523.2678%2C-188.2845%202523.2678%2C-179.6137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222526.7679%2C-179.6084%202523.2678%2C-169.6085%202519.7679%2C-179.6085%202526.7679%2C-179.6084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2037%20--%3E%0A%3Cg%20id%3D%22node38%22%20class%3D%22node%22%3E%0A%3Ctitle%3E37%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%222240.3042%2C-58.6014%202114.2314%2C-58.6014%202114.2314%2C.2014%202240.3042%2C.2014%202240.3042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222177.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.402%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222177.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201755%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222177.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1265%2C%20490%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2036%26%2345%3B%26gt%3B37%20--%3E%0A%3Cg%20id%3D%22edge37%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E36-%26gt%3B37%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M2262.132%2C-94.3048C2249.5098%2C-84.6215%202235.9751%2C-74.2382%202223.5052%2C-64.6717%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222225.5493%2C-61.8285%202215.4847%2C-58.5186%202221.2885%2C-67.3824%202225.5493%2C-61.8285%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2038%20--%3E%0A%3Cg%20id%3D%22node39%22%20class%3D%22node%22%3E%0A%3Ctitle%3E38%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%222363.8044%2C-58.6014%202258.7312%2C-58.6014%202258.7312%2C.2014%202363.8044%2C.2014%202363.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222311.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.496%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222311.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2099%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222311.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B45%2C%2054%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2036%26%2345%3B%26gt%3B38%20--%3E%0A%3Cg%20id%3D%22edge38%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E36-%26gt%3B38%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M2311.2678%2C-94.3048C2311.2678%2C-85.9126%202311.2678%2C-76.9946%202311.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222314.7679%2C-68.5186%202311.2678%2C-58.5186%202307.7679%2C-68.5187%202314.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2040%20--%3E%0A%3Cg%20id%3D%22node41%22%20class%3D%22node%22%3E%0A%3Ctitle%3E40%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%222501.8043%2C-58.6014%202382.7313%2C-58.6014%202382.7313%2C.2014%202501.8043%2C.2014%202501.8043%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222442.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222442.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201094%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222442.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B564%2C%20530%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2039%26%2345%3B%26gt%3B40%20--%3E%0A%3Cg%20id%3D%22edge40%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E39-%26gt%3B40%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M2493.5663%2C-94.3048C2486.4452%2C-85.2671%202478.8435%2C-75.6195%202471.733%2C-66.5954%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222474.3073%2C-64.2072%202465.3691%2C-58.5186%202468.809%2C-68.5395%202474.3073%2C-64.2072%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2041%20--%3E%0A%3Cg%20id%3D%22node42%22%20class%3D%22node%22%3E%0A%3Ctitle%3E41%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%222653.8042%2C-58.6014%202520.7314%2C-58.6014%202520.7314%2C.2014%202653.8042%2C.2014%202653.8042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222587.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.477%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222587.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%204344%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222587.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B2637%2C%201707%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2039%26%2345%3B%26gt%3B41%20--%3E%0A%3Cg%20id%3D%22edge41%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E39-%26gt%3B41%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M2546.7356%2C-94.3048C2552.2474%2C-85.4515%202558.1235%2C-76.013%202563.6422%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222566.701%2C-68.8577%202569.015%2C-58.5186%202560.7585%2C-65.1581%202566.701%2C-68.8577%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2043%20--%3E%0A%3Cg%20id%3D%22node44%22%20class%3D%22node%22%3E%0A%3Ctitle%3E43%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%222897.8044%2C-169.7003%202792.7312%2C-169.7003%202792.7312%2C-94.2997%202897.8044%2C-94.2997%202897.8044%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222845.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B5%5D%20%26lt%3B%3D%2079.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222845.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.448%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222845.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20115%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222845.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B76%2C%2039%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2042%26%2345%3B%26gt%3B43%20--%3E%0A%3Cg%20id%3D%22edge43%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E42-%26gt%3B43%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M2890.9075%2C-205.5528C2885.456%2C-196.7671%202879.5888%2C-187.3115%202873.9366%2C-178.2025%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222876.8505%2C-176.2602%202868.604%2C-169.6085%202870.9025%2C-179.951%202876.8505%2C-176.2602%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2046%20--%3E%0A%3Cg%20id%3D%22node47%22%20class%3D%22node%22%3E%0A%3Ctitle%3E46%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223049.8042%2C-169.7003%202916.7314%2C-169.7003%202916.7314%2C-94.2997%203049.8042%2C-94.2997%203049.8042%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222983.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B10%5D%20%26lt%3B%3D%200.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222983.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.497%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222983.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%202470%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222983.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1141%2C%201329%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2042%26%2345%3B%26gt%3B46%20--%3E%0A%3Cg%20id%3D%22edge46%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E42-%26gt%3B46%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M2937.6281%2C-205.5528C2943.0796%2C-196.7671%202948.9468%2C-187.3115%202954.599%2C-178.2025%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222957.6331%2C-179.951%202959.9316%2C-169.6085%202951.6851%2C-176.2602%202957.6331%2C-179.951%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2044%20--%3E%0A%3Cg%20id%3D%22node45%22%20class%3D%22node%22%3E%0A%3Ctitle%3E44%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%222777.8044%2C-58.6014%202672.7312%2C-58.6014%202672.7312%2C.2014%202777.8044%2C.2014%202777.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222725.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.334%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222725.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2052%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222725.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B41%2C%2011%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2043%26%2345%3B%26gt%3B44%20--%3E%0A%3Cg%20id%3D%22edge44%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E43-%26gt%3B44%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M2801.2656%2C-94.3048C2790.1775%2C-84.806%202778.303%2C-74.6335%202767.3136%2C-65.2192%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222769.3633%2C-62.3665%202759.4919%2C-58.5186%202764.8092%2C-67.6825%202769.3633%2C-62.3665%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2045%20--%3E%0A%3Cg%20id%3D%22node46%22%20class%3D%22node%22%3E%0A%3Ctitle%3E45%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%222901.8044%2C-58.6014%202796.7312%2C-58.6014%202796.7312%2C.2014%202901.8044%2C.2014%202901.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222849.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.494%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222849.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2063%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222849.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B35%2C%2028%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2043%26%2345%3B%26gt%3B45%20--%3E%0A%3Cg%20id%3D%22edge45%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E43-%26gt%3B45%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M2846.7345%2C-94.3048C2847.0611%2C-85.9126%202847.4081%2C-76.9946%202847.7371%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222851.2354%2C-68.6472%202848.127%2C-58.5186%202844.2407%2C-68.375%202851.2354%2C-68.6472%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2047%20--%3E%0A%3Cg%20id%3D%22node48%22%20class%3D%22node%22%3E%0A%3Ctitle%3E47%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223039.8043%2C-58.6014%202920.7313%2C-58.6014%202920.7313%2C.2014%203039.8043%2C.2014%203039.8043%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222980.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.478%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222980.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20520%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%222980.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B205%2C%20315%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2046%26%2345%3B%26gt%3B47%20--%3E%0A%3Cg%20id%3D%22edge47%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E46-%26gt%3B47%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M2982.1677%2C-94.3048C2981.9228%2C-85.9126%202981.6626%2C-76.9946%202981.4158%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%222984.9137%2C-68.4122%202981.1234%2C-58.5186%202977.9167%2C-68.6165%202984.9137%2C-68.4122%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2048%20--%3E%0A%3Cg%20id%3D%22node49%22%20class%3D%22node%22%3E%0A%3Ctitle%3E48%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223184.3042%2C-58.6014%203058.2314%2C-58.6014%203058.2314%2C.2014%203184.3042%2C.2014%203184.3042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223121.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.499%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223121.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201950%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223121.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B936%2C%201014%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2046%26%2345%3B%26gt%3B48%20--%3E%0A%3Cg%20id%3D%22edge48%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E46-%26gt%3B48%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3033.8703%2C-94.3048C3046.8693%2C-84.6215%203060.808%2C-74.2382%203073.6502%2C-64.6717%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223075.9815%2C-67.2994%203081.9101%2C-58.5186%203071.7997%2C-61.6858%203075.9815%2C-67.2994%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2050%20--%3E%0A%3Cg%20id%3D%22node51%22%20class%3D%22node%22%3E%0A%3Ctitle%3E50%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223514.8043%2C-280.9003%203395.7313%2C-280.9003%203395.7313%2C-205.4997%203514.8043%2C-205.4997%203514.8043%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223455.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2060.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223455.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.466%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223455.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20538%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223455.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B199%2C%20339%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2049%26%2345%3B%26gt%3B50%20--%3E%0A%3Cg%20id%3D%22edge50%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E49-%26gt%3B50%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3455.2678%2C-316.7528C3455.2678%2C-308.42%203455.2678%2C-299.4845%203455.2678%2C-290.8137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223458.7679%2C-290.8084%203455.2678%2C-280.8085%203451.7679%2C-290.8085%203458.7679%2C-290.8084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2057%20--%3E%0A%3Cg%20id%3D%22node58%22%20class%3D%22node%22%3E%0A%3Ctitle%3E57%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223828.8043%2C-280.9003%203709.7313%2C-280.9003%203709.7313%2C-205.4997%203828.8043%2C-205.4997%203828.8043%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223769.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B2%5D%20%26lt%3B%3D%20185.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223769.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.35%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223769.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20605%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223769.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B137%2C%20468%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2049%26%2345%3B%26gt%3B57%20--%3E%0A%3Cg%20id%3D%22edge57%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E49-%26gt%3B57%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3515.0679%2C-333.2224C3567.7758%2C-314.5564%203644.5802%2C-287.3569%203699.9572%2C-267.7457%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223701.4324%2C-270.9363%203709.6903%2C-264.2988%203699.0956%2C-264.3379%203701.4324%2C-270.9363%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2051%20--%3E%0A%3Cg%20id%3D%22node52%22%20class%3D%22node%22%3E%0A%3Ctitle%3E51%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223447.8043%2C-169.7003%203328.7313%2C-169.7003%203328.7313%2C-94.2997%203447.8043%2C-94.2997%203447.8043%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223388.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B2%5D%20%26lt%3B%3D%20161.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223388.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.489%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223388.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20366%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223388.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B156%2C%20210%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2050%26%2345%3B%26gt%3B51%20--%3E%0A%3Cg%20id%3D%22edge51%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E50-%26gt%3B51%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3432.5847%2C-205.5528C3427.2911%2C-196.7671%203421.594%2C-187.3115%203416.1056%2C-178.2025%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223419.0863%2C-176.3676%203410.9276%2C-169.6085%203413.0905%2C-179.9802%203419.0863%2C-176.3676%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2054%20--%3E%0A%3Cg%20id%3D%22node55%22%20class%3D%22node%22%3E%0A%3Ctitle%3E54%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223578.3044%2C-169.7003%203466.2312%2C-169.7003%203466.2312%2C-94.2997%203578.3044%2C-94.2997%203578.3044%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223522.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2063.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223522.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.375%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223522.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20172%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223522.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B43%2C%20129%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2050%26%2345%3B%26gt%3B54%20--%3E%0A%3Cg%20id%3D%22edge54%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E50-%26gt%3B54%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3477.9509%2C-205.5528C3483.2445%2C-196.7671%203488.9416%2C-187.3115%203494.43%2C-178.2025%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223497.4451%2C-179.9802%203499.608%2C-169.6085%203491.4493%2C-176.3676%203497.4451%2C-179.9802%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2052%20--%3E%0A%3Cg%20id%3D%22node53%22%20class%3D%22node%22%3E%0A%3Ctitle%3E52%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223315.3044%2C-58.6014%203203.2312%2C-58.6014%203203.2312%2C.2014%203315.3044%2C.2014%203315.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223259.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.458%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223259.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20163%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223259.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B58%2C%20105%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2051%26%2345%3B%26gt%3B52%20--%3E%0A%3Cg%20id%3D%22edge52%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E51-%26gt%3B52%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3340.9655%2C-94.3048C3328.93%2C-84.7137%203316.0327%2C-74.4359%203304.1232%2C-64.9452%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223306.0605%2C-62.0136%203296.0587%2C-58.5186%203301.6979%2C-67.488%203306.0605%2C-62.0136%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2053%20--%3E%0A%3Cg%20id%3D%22node54%22%20class%3D%22node%22%3E%0A%3Ctitle%3E53%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223446.3044%2C-58.6014%203334.2312%2C-58.6014%203334.2312%2C.2014%203446.3044%2C.2014%203446.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223390.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.499%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223390.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20203%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223390.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B98%2C%20105%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2051%26%2345%3B%26gt%3B53%20--%3E%0A%3Cg%20id%3D%22edge53%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E51-%26gt%3B53%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3389.0012%2C-94.3048C3389.1644%2C-85.9126%203389.3379%2C-76.9946%203389.5025%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223393.0021%2C-68.5849%203389.6974%2C-58.5186%203386.0034%2C-68.4486%203393.0021%2C-68.5849%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2055%20--%3E%0A%3Cg%20id%3D%22node56%22%20class%3D%22node%22%3E%0A%3Ctitle%3E55%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223577.3044%2C-58.6014%203465.2312%2C-58.6014%203465.2312%2C.2014%203577.3044%2C.2014%203577.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223521.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.407%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223521.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20144%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223521.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B41%2C%20103%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2054%26%2345%3B%26gt%3B55%20--%3E%0A%3Cg%20id%3D%22edge55%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E54-%26gt%3B55%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3521.9011%2C-94.3048C3521.8195%2C-85.9126%203521.7327%2C-76.9946%203521.6505%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223525.1502%2C-68.4841%203521.553%2C-58.5186%203518.1505%2C-68.5522%203525.1502%2C-68.4841%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2056%20--%3E%0A%3Cg%20id%3D%22node57%22%20class%3D%22node%22%3E%0A%3Ctitle%3E56%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223694.3045%2C-58.6014%203596.2311%2C-58.6014%203596.2311%2C.2014%203694.3045%2C.2014%203694.3045%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223645.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.133%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223645.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2028%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223645.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B2%2C%2026%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2054%26%2345%3B%26gt%3B56%20--%3E%0A%3Cg%20id%3D%22edge56%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E54-%26gt%3B56%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3567.37%2C-94.3048C3578.8457%2C-84.7137%203591.1432%2C-74.4359%203602.4987%2C-64.9452%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223604.7596%2C-67.6171%203610.1881%2C-58.5186%203600.2706%2C-62.246%203604.7596%2C-67.6171%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2058%20--%3E%0A%3Cg%20id%3D%22node59%22%20class%3D%22node%22%3E%0A%3Ctitle%3E58%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223828.8043%2C-169.7003%203709.7313%2C-169.7003%203709.7313%2C-94.2997%203828.8043%2C-94.2997%203828.8043%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223769.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2060.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223769.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.343%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223769.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20600%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223769.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B132%2C%20468%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2057%26%2345%3B%26gt%3B58%20--%3E%0A%3Cg%20id%3D%22edge58%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E57-%26gt%3B58%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3769.2678%2C-205.5528C3769.2678%2C-197.22%203769.2678%2C-188.2845%203769.2678%2C-179.6137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223772.7679%2C-179.6084%203769.2678%2C-169.6085%203765.7679%2C-179.6085%203772.7679%2C-179.6084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2061%20--%3E%0A%3Cg%20id%3D%22node62%22%20class%3D%22node%22%3E%0A%3Ctitle%3E61%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223938.8046%2C-161.4014%203847.731%2C-161.4014%203847.731%2C-102.5986%203938.8046%2C-102.5986%203938.8046%2C-161.4014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223893.2678%22%20y%3D%22-144.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.0%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223893.2678%22%20y%3D%22-127.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%205%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223893.2678%22%20y%3D%22-111%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B5%2C%200%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2057%26%2345%3B%26gt%3B61%20--%3E%0A%3Cg%20id%3D%22edge61%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E57-%26gt%3B61%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3811.2485%2C-205.5528C3824.6397%2C-193.5439%203839.4267%2C-180.2833%203852.7147%2C-168.367%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223855.2352%2C-170.8079%203860.3433%2C-161.5258%203850.5617%2C-165.5965%203855.2352%2C-170.8079%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2059%20--%3E%0A%3Cg%20id%3D%22node60%22%20class%3D%22node%22%3E%0A%3Ctitle%3E59%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223825.3044%2C-58.6014%203713.2312%2C-58.6014%203713.2312%2C.2014%203825.3044%2C.2014%203825.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223769.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.377%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223769.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20389%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223769.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B98%2C%20291%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2058%26%2345%3B%26gt%3B59%20--%3E%0A%3Cg%20id%3D%22edge59%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E58-%26gt%3B59%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3769.2678%2C-94.3048C3769.2678%2C-85.9126%203769.2678%2C-76.9946%203769.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223772.7679%2C-68.5186%203769.2678%2C-58.5186%203765.7679%2C-68.5187%203772.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2060%20--%3E%0A%3Cg%20id%3D%22node61%22%20class%3D%22node%22%3E%0A%3Ctitle%3E60%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%223956.3044%2C-58.6014%203844.2312%2C-58.6014%203844.2312%2C.2014%203956.3044%2C.2014%203956.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223900.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.27%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223900.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20211%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%223900.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B34%2C%20177%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2058%26%2345%3B%26gt%3B60%20--%3E%0A%3Cg%20id%3D%22edge60%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E58-%26gt%3B60%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M3817.3035%2C-94.3048C3829.5256%2C-84.7137%203842.6229%2C-74.4359%203854.717%2C-64.9452%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%223857.2003%2C-67.4455%203862.9065%2C-58.5186%203852.8788%2C-61.9387%203857.2003%2C-67.4455%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2063%20--%3E%0A%3Cg%20id%3D%22node64%22%20class%3D%22node%22%3E%0A%3Ctitle%3E63%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225097.8042%2C-503.3003%204964.7314%2C-503.3003%204964.7314%2C-427.8997%205097.8042%2C-427.8997%205097.8042%2C-503.3003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225031.2678%22%20y%3D%22-486.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B6%5D%20%26lt%3B%3D%202.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225031.2678%22%20y%3D%22-469.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.481%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225031.2678%22%20y%3D%22-453%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%206533%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225031.2678%22%20y%3D%22-436.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B2636%2C%203897%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2062%26%2345%3B%26gt%3B63%20--%3E%0A%3Cg%20id%3D%22edge63%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E62-%26gt%3B63%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5031.2678%2C-539.1528C5031.2678%2C-530.82%205031.2678%2C-521.8845%205031.2678%2C-513.2137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225034.7679%2C-513.2084%205031.2678%2C-503.2085%205027.7679%2C-513.2085%205034.7679%2C-513.2084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2094%20--%3E%0A%3Cg%20id%3D%22node95%22%20class%3D%22node%22%3E%0A%3Ctitle%3E94%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226641.3042%2C-503.3003%206501.2314%2C-503.3003%206501.2314%2C-427.8997%206641.3042%2C-427.8997%206641.3042%2C-503.3003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226571.2678%22%20y%3D%22-486.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B4%5D%20%26lt%3B%3D%20149.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226571.2678%22%20y%3D%22-469.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.27%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226571.2678%22%20y%3D%22-453%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2013522%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226571.2678%22%20y%3D%22-436.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B2178%2C%2011344%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2062%26%2345%3B%26gt%3B94%20--%3E%0A%3Cg%20id%3D%22edge94%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E62-%26gt%3B94%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5101.4401%2C-571.733C5354.8778%2C-553.4328%206222.0739%2C-490.8145%206491.0486%2C-471.3925%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226491.4032%2C-474.8761%206501.1252%2C-470.6648%206490.899%2C-467.8942%206491.4032%2C-474.8761%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2064%20--%3E%0A%3Cg%20id%3D%22node65%22%20class%3D%22node%22%3E%0A%3Ctitle%3E64%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224838.8042%2C-392.1003%204705.7314%2C-392.1003%204705.7314%2C-316.6997%204838.8042%2C-316.6997%204838.8042%2C-392.1003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224772.2678%22%20y%3D%22-375.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2058.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224772.2678%22%20y%3D%22-358.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.495%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224772.2678%22%20y%3D%22-341.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%205294%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224772.2678%22%20y%3D%22-325%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B2379%2C%202915%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2063%26%2345%3B%26gt%3B64%20--%3E%0A%3Cg%20id%3D%22edge64%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E63-%26gt%3B64%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4964.5689%2C-436.9632C4929.0584%2C-421.7171%204885.2353%2C-402.9019%204848.3964%2C-387.0853%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224849.7601%2C-383.8619%204839.1904%2C-383.1328%204846.9984%2C-390.2941%204849.7601%2C-383.8619%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2079%20--%3E%0A%3Cg%20id%3D%22node80%22%20class%3D%22node%22%3E%0A%3Ctitle%3E79%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225344.8043%2C-392.1003%205225.7313%2C-392.1003%205225.7313%2C-316.6997%205344.8043%2C-316.6997%205344.8043%2C-392.1003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225285.2678%22%20y%3D%22-375.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B7%5D%20%26lt%3B%3D%202.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225285.2678%22%20y%3D%22-358.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.329%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225285.2678%22%20y%3D%22-341.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201239%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225285.2678%22%20y%3D%22-325%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B257%2C%20982%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2063%26%2345%3B%26gt%3B79%20--%3E%0A%3Cg%20id%3D%22edge79%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E63-%26gt%3B79%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5098.0033%2C-436.3835C5134.4531%2C-420.426%205179.5246%2C-400.6939%205216.3154%2C-384.587%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225217.7234%2C-387.7914%205225.4803%2C-380.5747%205214.9161%2C-381.379%205217.7234%2C-387.7914%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2065%20--%3E%0A%3Cg%20id%3D%22node66%22%20class%3D%22node%22%3E%0A%3Ctitle%3E65%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224448.8042%2C-280.9003%204315.7314%2C-280.9003%204315.7314%2C-205.4997%204448.8042%2C-205.4997%204448.8042%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224382.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B5%5D%20%26lt%3B%3D%2088.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224382.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224382.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%203797%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224382.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1839%2C%201958%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2064%26%2345%3B%26gt%3B65%20--%3E%0A%3Cg%20id%3D%22edge65%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E64-%26gt%3B65%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4705.6675%2C-335.4104C4637.2031%2C-315.8892%204530.7246%2C-285.5292%204458.8274%2C-265.0293%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224459.496%2C-261.5805%204448.9195%2C-262.2043%204457.5765%2C-268.3122%204459.496%2C-261.5805%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2072%20--%3E%0A%3Cg%20id%3D%22node73%22%20class%3D%22node%22%3E%0A%3Ctitle%3E72%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224831.8043%2C-280.9003%204712.7313%2C-280.9003%204712.7313%2C-205.4997%204831.8043%2C-205.4997%204831.8043%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224772.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2061.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224772.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.461%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224772.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201497%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224772.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B540%2C%20957%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2064%26%2345%3B%26gt%3B72%20--%3E%0A%3Cg%20id%3D%22edge72%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E64-%26gt%3B72%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4772.2678%2C-316.7528C4772.2678%2C-308.42%204772.2678%2C-299.4845%204772.2678%2C-290.8137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224775.7679%2C-290.8084%204772.2678%2C-280.8085%204768.7679%2C-290.8085%204775.7679%2C-290.8084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2066%20--%3E%0A%3Cg%20id%3D%22node67%22%20class%3D%22node%22%3E%0A%3Ctitle%3E66%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224238.8042%2C-169.7003%204105.7314%2C-169.7003%204105.7314%2C-94.2997%204238.8042%2C-94.2997%204238.8042%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224172.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2041.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224172.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.498%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224172.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%202245%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224172.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1189%2C%201056%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2065%26%2345%3B%26gt%3B66%20--%3E%0A%3Cg%20id%3D%22edge66%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E65-%26gt%3B66%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4315.5339%2C-207.8628C4294.0004%2C-196.4603%204269.9795%2C-183.7407%204247.8539%2C-172.0246%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224249.3034%2C-168.8318%204238.828%2C-167.2452%204246.0276%2C-175.0181%204249.3034%2C-168.8318%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2069%20--%3E%0A%3Cg%20id%3D%22node70%22%20class%3D%22node%22%3E%0A%3Ctitle%3E69%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224441.8043%2C-169.7003%204322.7313%2C-169.7003%204322.7313%2C-94.2997%204441.8043%2C-94.2997%204441.8043%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224382.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B8%5D%20%26lt%3B%3D%200.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224382.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.487%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224382.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201552%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224382.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B650%2C%20902%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2065%26%2345%3B%26gt%3B69%20--%3E%0A%3Cg%20id%3D%22edge69%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E65-%26gt%3B69%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4382.2678%2C-205.5528C4382.2678%2C-197.22%204382.2678%2C-188.2845%204382.2678%2C-179.6137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224385.7679%2C-179.6084%204382.2678%2C-169.6085%204378.7679%2C-179.6085%204385.7679%2C-179.6084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2067%20--%3E%0A%3Cg%20id%3D%22node68%22%20class%3D%22node%22%3E%0A%3Ctitle%3E67%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224087.3044%2C-58.6014%203975.2312%2C-58.6014%203975.2312%2C.2014%204087.3044%2C.2014%204087.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224031.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.439%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224031.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20157%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224031.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B106%2C%2051%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2066%26%2345%3B%26gt%3B67%20--%3E%0A%3Cg%20id%3D%22edge67%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E66-%26gt%3B67%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4120.5652%2C-94.3048C4107.2837%2C-84.6215%204093.0419%2C-74.2382%204079.9206%2C-64.6717%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224081.6235%2C-61.5818%204071.4811%2C-58.5186%204077.4996%2C-67.2381%204081.6235%2C-61.5818%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2068%20--%3E%0A%3Cg%20id%3D%22node69%22%20class%3D%22node%22%3E%0A%3Ctitle%3E68%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224238.8042%2C-58.6014%204105.7314%2C-58.6014%204105.7314%2C.2014%204238.8042%2C.2014%204238.8042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224172.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.499%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224172.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%202088%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224172.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1083%2C%201005%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2066%26%2345%3B%26gt%3B68%20--%3E%0A%3Cg%20id%3D%22edge68%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E66-%26gt%3B68%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4172.2678%2C-94.3048C4172.2678%2C-85.9126%204172.2678%2C-76.9946%204172.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224175.7679%2C-68.5186%204172.2678%2C-58.5186%204168.7679%2C-68.5187%204175.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2070%20--%3E%0A%3Cg%20id%3D%22node71%22%20class%3D%22node%22%3E%0A%3Ctitle%3E70%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224376.8043%2C-58.6014%204257.7313%2C-58.6014%204257.7313%2C.2014%204376.8043%2C.2014%204376.8043%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224317.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.482%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224317.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201391%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224317.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B562%2C%20829%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2069%26%2345%3B%26gt%3B70%20--%3E%0A%3Cg%20id%3D%22edge70%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E69-%26gt%3B70%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4358.4333%2C-94.3048C4352.8354%2C-85.4515%204346.8675%2C-76.013%204341.2626%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224344.1084%2C-65.1003%204335.8058%2C-58.5186%204338.1919%2C-68.8413%204344.1084%2C-65.1003%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2071%20--%3E%0A%3Cg%20id%3D%22node72%22%20class%3D%22node%22%3E%0A%3Ctitle%3E71%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224500.8044%2C-58.6014%204395.7312%2C-58.6014%204395.7312%2C.2014%204500.8044%2C.2014%204500.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224448.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.496%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224448.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20161%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224448.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B88%2C%2073%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2069%26%2345%3B%26gt%3B71%20--%3E%0A%3Cg%20id%3D%22edge71%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E69-%26gt%3B71%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4406.469%2C-94.3048C4412.153%2C-85.4515%204418.2128%2C-76.013%204423.9039%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224426.9871%2C-68.8246%204429.4446%2C-58.5186%204421.0967%2C-65.0427%204426.9871%2C-68.8246%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2073%20--%3E%0A%3Cg%20id%3D%22node74%22%20class%3D%22node%22%3E%0A%3Ctitle%3E73%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224762.8043%2C-169.7003%204643.7313%2C-169.7003%204643.7313%2C-94.2997%204762.8043%2C-94.2997%204762.8043%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224703.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B8%5D%20%26lt%3B%3D%200.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224703.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.482%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224703.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20849%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224703.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B343%2C%20506%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2072%26%2345%3B%26gt%3B73%20--%3E%0A%3Cg%20id%3D%22edge73%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E72-%26gt%3B73%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4748.9075%2C-205.5528C4743.456%2C-196.7671%204737.5888%2C-187.3115%204731.9366%2C-178.2025%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224734.8505%2C-176.2602%204726.604%2C-169.6085%204728.9025%2C-179.951%204734.8505%2C-176.2602%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2076%20--%3E%0A%3Cg%20id%3D%22node77%22%20class%3D%22node%22%3E%0A%3Ctitle%3E76%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224900.8043%2C-169.7003%204781.7313%2C-169.7003%204781.7313%2C-94.2997%204900.8043%2C-94.2997%204900.8043%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224841.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B3%5D%20%26lt%3B%3D%2055.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224841.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.423%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224841.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20648%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224841.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B197%2C%20451%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2072%26%2345%3B%26gt%3B76%20--%3E%0A%3Cg%20id%3D%22edge76%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E72-%26gt%3B76%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4795.6281%2C-205.5528C4801.0796%2C-196.7671%204806.9468%2C-187.3115%204812.599%2C-178.2025%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224815.6331%2C-179.951%204817.9316%2C-169.6085%204809.6851%2C-176.2602%204815.6331%2C-179.951%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2074%20--%3E%0A%3Cg%20id%3D%22node75%22%20class%3D%22node%22%3E%0A%3Ctitle%3E74%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224638.8043%2C-58.6014%204519.7313%2C-58.6014%204519.7313%2C.2014%204638.8043%2C.2014%204638.8043%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224579.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.475%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224579.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20779%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224579.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B303%2C%20476%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2073%26%2345%3B%26gt%3B74%20--%3E%0A%3Cg%20id%3D%22edge74%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E73-%26gt%3B74%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4657.7989%2C-94.3048C4646.2299%2C-84.7137%204633.8325%2C-74.4359%204622.3846%2C-64.9452%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224624.565%2C-62.2065%204614.6327%2C-58.5186%204620.0974%2C-67.5954%204624.565%2C-62.2065%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2075%20--%3E%0A%3Cg%20id%3D%22node76%22%20class%3D%22node%22%3E%0A%3Ctitle%3E75%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224762.8044%2C-58.6014%204657.7312%2C-58.6014%204657.7312%2C.2014%204762.8044%2C.2014%204762.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224710.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.49%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224710.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2070%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224710.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B40%2C%2030%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2073%26%2345%3B%26gt%3B75%20--%3E%0A%3Cg%20id%3D%22edge75%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E73-%26gt%3B75%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4705.8346%2C-94.3048C4706.406%2C-85.9126%204707.0133%2C-76.9946%204707.5891%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224711.0839%2C-68.7333%204708.2714%2C-58.5186%204704.1%2C-68.2577%204711.0839%2C-68.7333%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2077%20--%3E%0A%3Cg%20id%3D%22node78%22%20class%3D%22node%22%3E%0A%3Ctitle%3E77%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%224886.8044%2C-58.6014%204781.7312%2C-58.6014%204781.7312%2C.2014%204886.8044%2C.2014%204886.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224834.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.499%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224834.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2025%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224834.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B13%2C%2012%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2076%26%2345%3B%26gt%3B77%20--%3E%0A%3Cg%20id%3D%22edge77%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E76-%26gt%3B77%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4838.701%2C-94.3048C4838.1296%2C-85.9126%204837.5223%2C-76.9946%204836.9465%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224840.4356%2C-68.2577%204836.2642%2C-58.5186%204833.4517%2C-68.7333%204840.4356%2C-68.2577%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2078%20--%3E%0A%3Cg%20id%3D%22node79%22%20class%3D%22node%22%3E%0A%3Ctitle%3E78%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225024.8043%2C-58.6014%204905.7313%2C-58.6014%204905.7313%2C.2014%205024.8043%2C.2014%205024.8043%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224965.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.416%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224965.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20623%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%224965.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B184%2C%20439%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2076%26%2345%3B%26gt%3B78%20--%3E%0A%3Cg%20id%3D%22edge78%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E76-%26gt%3B78%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M4886.7367%2C-94.3048C4898.3057%2C-84.7137%204910.7031%2C-74.4359%204922.151%2C-64.9452%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%224924.4382%2C-67.5954%204929.9029%2C-58.5186%204919.9706%2C-62.2065%204924.4382%2C-67.5954%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2080%20--%3E%0A%3Cg%20id%3D%22node81%22%20class%3D%22node%22%3E%0A%3Ctitle%3E80%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225344.8043%2C-280.9003%205225.7313%2C-280.9003%205225.7313%2C-205.4997%205344.8043%2C-205.4997%205344.8043%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225285.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B3%5D%20%26lt%3B%3D%2084.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225285.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.287%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225285.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20748%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225285.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B130%2C%20618%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2079%26%2345%3B%26gt%3B80%20--%3E%0A%3Cg%20id%3D%22edge80%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E79-%26gt%3B80%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5285.2678%2C-316.7528C5285.2678%2C-308.42%205285.2678%2C-299.4845%205285.2678%2C-290.8137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225288.7679%2C-290.8084%205285.2678%2C-280.8085%205281.7679%2C-290.8085%205288.7679%2C-290.8084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2087%20--%3E%0A%3Cg%20id%3D%22node88%22%20class%3D%22node%22%3E%0A%3Ctitle%3E87%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225711.8043%2C-280.9003%205592.7313%2C-280.9003%205592.7313%2C-205.4997%205711.8043%2C-205.4997%205711.8043%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225652.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B2%5D%20%26lt%3B%3D%20156.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225652.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.384%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225652.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20491%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225652.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B127%2C%20364%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2079%26%2345%3B%26gt%3B87%20--%3E%0A%3Cg%20id%3D%22edge87%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E79-%26gt%3B87%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5345.0495%2C-336.2863C5410.0964%2C-316.5773%205513.9659%2C-285.1051%205582.6431%2C-264.2961%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225583.9563%2C-267.5554%205592.5117%2C-261.3059%205581.9264%2C-260.8562%205583.9563%2C-267.5554%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2081%20--%3E%0A%3Cg%20id%3D%22node82%22%20class%3D%22node%22%3E%0A%3Ctitle%3E81%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225276.3044%2C-169.7003%205164.2312%2C-169.7003%205164.2312%2C-94.2997%205276.3044%2C-94.2997%205276.3044%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225220.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B3%5D%20%26lt%3B%3D%2080.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225220.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.314%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225220.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20497%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225220.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B97%2C%20400%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2080%26%2345%3B%26gt%3B81%20--%3E%0A%3Cg%20id%3D%22edge81%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E80-%26gt%3B81%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5263.2618%2C-205.5528C5258.1792%2C-196.8577%205252.7131%2C-187.5064%205247.4394%2C-178.4843%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225250.3193%2C-176.4755%205242.2512%2C-169.6085%205244.276%2C-180.008%205250.3193%2C-176.4755%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2084%20--%3E%0A%3Cg%20id%3D%22node85%22%20class%3D%22node%22%3E%0A%3Ctitle%3E84%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225407.3044%2C-169.7003%205295.2312%2C-169.7003%205295.2312%2C-94.2997%205407.3044%2C-94.2997%205407.3044%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225351.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2050.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225351.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.228%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225351.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20251%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225351.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B33%2C%20218%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2080%26%2345%3B%26gt%3B84%20--%3E%0A%3Cg%20id%3D%22edge84%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E80-%26gt%3B84%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5307.6124%2C-205.5528C5312.7731%2C-196.8577%205318.3234%2C-187.5064%205323.6782%2C-178.4843%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225326.852%2C-179.9943%205328.9462%2C-169.6085%205320.8324%2C-176.4215%205326.852%2C-179.9943%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2082%20--%3E%0A%3Cg%20id%3D%22node83%22%20class%3D%22node%22%3E%0A%3Ctitle%3E82%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225155.3044%2C-58.6014%205043.2312%2C-58.6014%205043.2312%2C.2014%205155.3044%2C.2014%205155.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225099.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.285%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225099.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20436%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225099.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B75%2C%20361%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2081%26%2345%3B%26gt%3B82%20--%3E%0A%3Cg%20id%3D%22edge82%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E81-%26gt%3B82%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5175.8989%2C-94.3048C5164.7184%2C-84.806%205152.7449%2C-74.6335%205141.6639%2C-65.2192%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225143.6642%2C-62.326%205133.7771%2C-58.5186%205139.1319%2C-67.6607%205143.6642%2C-62.326%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2083%20--%3E%0A%3Cg%20id%3D%22node84%22%20class%3D%22node%22%3E%0A%3Ctitle%3E83%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225278.8044%2C-58.6014%205173.7312%2C-58.6014%205173.7312%2C.2014%205278.8044%2C.2014%205278.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225226.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.461%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225226.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2061%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225226.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B22%2C%2039%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2081%26%2345%3B%26gt%3B83%20--%3E%0A%3Cg%20id%3D%22edge83%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E81-%26gt%3B83%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5222.4679%2C-94.3048C5222.9577%2C-85.9126%205223.4782%2C-76.9946%205223.9718%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225227.4679%2C-68.7056%205224.5566%2C-58.5186%205220.4798%2C-68.2977%205227.4679%2C-68.7056%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2085%20--%3E%0A%3Cg%20id%3D%22node86%22%20class%3D%22node%22%3E%0A%3Ctitle%3E85%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225395.3045%2C-58.6014%205297.2311%2C-58.6014%205297.2311%2C.2014%205395.3045%2C.2014%205395.3045%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225346.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.098%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225346.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2058%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225346.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B3%2C%2055%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2084%26%2345%3B%26gt%3B85%20--%3E%0A%3Cg%20id%3D%22edge85%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E84-%26gt%3B85%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5349.4344%2C-94.3048C5349.0262%2C-85.9126%205348.5924%2C-76.9946%205348.1811%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225351.6756%2C-68.3368%205347.6938%2C-58.5186%205344.6838%2C-68.6769%205351.6756%2C-68.3368%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2086%20--%3E%0A%3Cg%20id%3D%22node87%22%20class%3D%22node%22%3E%0A%3Ctitle%3E86%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225526.3044%2C-58.6014%205414.2312%2C-58.6014%205414.2312%2C.2014%205526.3044%2C.2014%205526.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225470.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.263%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225470.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20193%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225470.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B30%2C%20163%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2084%26%2345%3B%26gt%3B86%20--%3E%0A%3Cg%20id%3D%22edge86%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E84-%26gt%3B86%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5394.9033%2C-94.3048C5405.899%2C-84.806%205417.6746%2C-74.6335%205428.5724%2C-65.2192%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225431.0496%2C-67.7044%205436.3289%2C-58.5186%205426.4735%2C-62.4073%205431.0496%2C-67.7044%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2088%20--%3E%0A%3Cg%20id%3D%22node89%22%20class%3D%22node%22%3E%0A%3Ctitle%3E88%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225704.8044%2C-169.7003%205599.7312%2C-169.7003%205599.7312%2C-94.2997%205704.8044%2C-94.2997%205704.8044%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225652.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2056.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225652.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.247%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225652.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20104%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225652.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B15%2C%2089%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2087%26%2345%3B%26gt%3B88%20--%3E%0A%3Cg%20id%3D%22edge88%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E87-%26gt%3B88%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5652.2678%2C-205.5528C5652.2678%2C-197.22%205652.2678%2C-188.2845%205652.2678%2C-179.6137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225655.7679%2C-179.6084%205652.2678%2C-169.6085%205648.7679%2C-179.6085%205655.7679%2C-179.6084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2091%20--%3E%0A%3Cg%20id%3D%22node92%22%20class%3D%22node%22%3E%0A%3Ctitle%3E91%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225900.8043%2C-169.7003%205781.7313%2C-169.7003%205781.7313%2C-94.2997%205900.8043%2C-94.2997%205900.8043%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225841.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B3%5D%20%26lt%3B%3D%2083.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225841.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.411%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225841.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20387%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225841.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B112%2C%20275%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2087%26%2345%3B%26gt%3B91%20--%3E%0A%3Cg%20id%3D%22edge91%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E87-%26gt%3B91%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5711.8092%2C-208.1683C5731.2256%2C-196.7444%205752.934%2C-183.972%205772.9429%2C-172.1996%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225774.8571%2C-175.1343%205781.7011%2C-167.0466%205771.3074%2C-169.1011%205774.8571%2C-175.1343%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2089%20--%3E%0A%3Cg%20id%3D%22node90%22%20class%3D%22node%22%3E%0A%3Ctitle%3E89%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225643.3045%2C-58.6014%205545.2311%2C-58.6014%205545.2311%2C.2014%205643.3045%2C.2014%205643.3045%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225594.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.091%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225594.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2042%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225594.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B2%2C%2040%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2088%26%2345%3B%26gt%3B89%20--%3E%0A%3Cg%20id%3D%22edge89%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E88-%26gt%3B89%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5631.0001%2C-94.3048C5626.0571%2C-85.5437%205620.7907%2C-76.2096%205615.8349%2C-67.4259%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225618.7717%2C-65.5082%205610.8094%2C-58.5186%205612.6751%2C-68.9479%205618.7717%2C-65.5082%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2090%20--%3E%0A%3Cg%20id%3D%22node91%22%20class%3D%22node%22%3E%0A%3Ctitle%3E90%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225766.8044%2C-58.6014%205661.7312%2C-58.6014%205661.7312%2C.2014%205766.8044%2C.2014%205766.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225714.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.331%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225714.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2062%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225714.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B13%2C%2049%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2088%26%2345%3B%26gt%3B90%20--%3E%0A%3Cg%20id%3D%22edge90%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E88-%26gt%3B90%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5675.0023%2C-94.3048C5680.3418%2C-85.4515%205686.0343%2C-76.013%205691.3805%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225694.4179%2C-68.8894%205696.5854%2C-58.5186%205688.4237%2C-65.2742%205694.4179%2C-68.8894%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2092%20--%3E%0A%3Cg%20id%3D%22node93%22%20class%3D%22node%22%3E%0A%3Ctitle%3E92%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%225897.3044%2C-58.6014%205785.2312%2C-58.6014%205785.2312%2C.2014%205897.3044%2C.2014%205897.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225841.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.451%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225841.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20239%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225841.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B82%2C%20157%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2091%26%2345%3B%26gt%3B92%20--%3E%0A%3Cg%20id%3D%22edge92%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E91-%26gt%3B92%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5841.2678%2C-94.3048C5841.2678%2C-85.9126%205841.2678%2C-76.9946%205841.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225844.7679%2C-68.5186%205841.2678%2C-58.5186%205837.7679%2C-68.5187%205844.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2093%20--%3E%0A%3Cg%20id%3D%22node94%22%20class%3D%22node%22%3E%0A%3Ctitle%3E93%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226028.3044%2C-58.6014%205916.2312%2C-58.6014%205916.2312%2C.2014%206028.3044%2C.2014%206028.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225972.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.323%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225972.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20148%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%225972.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B30%2C%20118%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2091%26%2345%3B%26gt%3B93%20--%3E%0A%3Cg%20id%3D%22edge93%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E91-%26gt%3B93%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M5889.3035%2C-94.3048C5901.5256%2C-84.7137%205914.6229%2C-74.4359%205926.717%2C-64.9452%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%225929.2003%2C-67.4455%205934.9065%2C-58.5186%205924.8788%2C-61.9387%205929.2003%2C-67.4455%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2095%20--%3E%0A%3Cg%20id%3D%22node96%22%20class%3D%22node%22%3E%0A%3Ctitle%3E95%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226637.8042%2C-392.1003%206504.7314%2C-392.1003%206504.7314%2C-316.6997%206637.8042%2C-316.6997%206637.8042%2C-392.1003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226571.2678%22%20y%3D%22-375.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B7%5D%20%26lt%3B%3D%202.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226571.2678%22%20y%3D%22-358.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.302%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226571.2678%22%20y%3D%22-341.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%206849%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226571.2678%22%20y%3D%22-325%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1269%2C%205580%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2094%26%2345%3B%26gt%3B95%20--%3E%0A%3Cg%20id%3D%22edge95%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E94-%26gt%3B95%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6571.2678%2C-427.9528C6571.2678%2C-419.62%206571.2678%2C-410.6845%206571.2678%2C-402.0137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226574.7679%2C-402.0084%206571.2678%2C-392.0085%206567.7679%2C-402.0085%206574.7679%2C-402.0084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20110%20--%3E%0A%3Cg%20id%3D%22node111%22%20class%3D%22node%22%3E%0A%3Ctitle%3E110%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227370.3042%2C-392.1003%207244.2314%2C-392.1003%207244.2314%2C-316.6997%207370.3042%2C-316.6997%207370.3042%2C-392.1003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227307.2678%22%20y%3D%22-375.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B5%5D%20%26lt%3B%3D%2068.0%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227307.2678%22%20y%3D%22-358.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.235%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227307.2678%22%20y%3D%22-341.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%206673%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227307.2678%22%20y%3D%22-325%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B909%2C%205764%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2094%26%2345%3B%26gt%3B110%20--%3E%0A%3Cg%20id%3D%22edge110%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E94-%26gt%3B110%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6641.5052%2C-454.988C6781.159%2C-433.8882%207092.4428%2C-386.8573%207234.301%2C-365.4243%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227234.8525%2C-368.8808%207244.2173%2C-363.9261%207233.8067%2C-361.9594%207234.8525%2C-368.8808%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2096%20--%3E%0A%3Cg%20id%3D%22node97%22%20class%3D%22node%22%3E%0A%3Ctitle%3E96%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226513.8042%2C-280.9003%206380.7314%2C-280.9003%206380.7314%2C-205.4997%206513.8042%2C-205.4997%206513.8042%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226447.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2060.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226447.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.294%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226447.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%206133%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226447.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B1099%2C%205034%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2095%26%2345%3B%26gt%3B96%20--%3E%0A%3Cg%20id%3D%22edge96%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E95-%26gt%3B96%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6529.2871%2C-316.7528C6518.8841%2C-307.4237%206507.6388%2C-297.3392%206496.9084%2C-287.7164%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226498.9869%2C-284.8792%206489.2053%2C-280.8085%206494.3135%2C-290.0906%206498.9869%2C-284.8792%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20103%20--%3E%0A%3Cg%20id%3D%22node104%22%20class%3D%22node%22%3E%0A%3Ctitle%3E103%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226768.8043%2C-280.9003%206649.7313%2C-280.9003%206649.7313%2C-205.4997%206768.8043%2C-205.4997%206768.8043%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226709.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B0%5D%20%26lt%3B%3D%2062.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226709.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.362%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226709.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20716%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226709.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B170%2C%20546%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2095%26%2345%3B%26gt%3B103%20--%3E%0A%3Cg%20id%3D%22edge103%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E95-%26gt%3B103%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6617.9883%2C-316.7528C6629.7906%2C-307.2425%206642.5671%2C-296.9473%206654.7176%2C-287.1564%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226657.0048%2C-289.8083%206662.5954%2C-280.8085%206652.6127%2C-284.3576%206657.0048%2C-289.8083%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2097%20--%3E%0A%3Cg%20id%3D%22node98%22%20class%3D%22node%22%3E%0A%3Ctitle%3E97%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226318.3042%2C-169.7003%206192.2314%2C-169.7003%206192.2314%2C-94.2997%206318.3042%2C-94.2997%206318.3042%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226255.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B6%5D%20%26lt%3B%3D%201.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226255.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.303%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226255.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%205034%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226255.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B938%2C%204096%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2096%26%2345%3B%26gt%3B97%20--%3E%0A%3Cg%20id%3D%22edge97%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E96-%26gt%3B97%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6382.2654%2C-205.5528C6364.5954%2C-195.3189%206345.356%2C-184.1761%206327.321%2C-173.7308%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226328.984%2C-170.6493%206318.5764%2C-168.6662%206325.4757%2C-176.7068%206328.984%2C-170.6493%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20100%20--%3E%0A%3Cg%20id%3D%22node101%22%20class%3D%22node%22%3E%0A%3Ctitle%3E100%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226506.8043%2C-169.7003%206387.7313%2C-169.7003%206387.7313%2C-94.2997%206506.8043%2C-94.2997%206506.8043%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226447.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B5%5D%20%26lt%3B%3D%2062.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226447.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.25%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226447.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201099%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226447.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B161%2C%20938%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2096%26%2345%3B%26gt%3B100%20--%3E%0A%3Cg%20id%3D%22edge100%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E96-%26gt%3B100%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6447.2678%2C-205.5528C6447.2678%2C-197.22%206447.2678%2C-188.2845%206447.2678%2C-179.6137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226450.7679%2C-179.6084%206447.2678%2C-169.6085%206443.7679%2C-179.6085%206450.7679%2C-179.6084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2098%20--%3E%0A%3Cg%20id%3D%22node99%22%20class%3D%22node%22%3E%0A%3Ctitle%3E98%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226173.3042%2C-58.6014%206047.2314%2C-58.6014%206047.2314%2C.2014%206173.3042%2C.2014%206173.3042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226110.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.315%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226110.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%203412%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226110.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B668%2C%202744%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2097%26%2345%3B%26gt%3B98%20--%3E%0A%3Cg%20id%3D%22edge98%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E97-%26gt%3B98%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6202.0985%2C-94.3048C6188.3101%2C-84.5293%206173.5153%2C-74.0403%206159.9156%2C-64.3986%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226161.804%2C-61.447%206151.6219%2C-58.5186%206157.7555%2C-67.1575%206161.804%2C-61.447%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2099%20--%3E%0A%3Cg%20id%3D%22node100%22%20class%3D%22node%22%3E%0A%3Ctitle%3E99%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226318.3042%2C-58.6014%206192.2314%2C-58.6014%206192.2314%2C.2014%206318.3042%2C.2014%206318.3042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226255.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.278%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226255.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201622%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226255.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B270%2C%201352%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2097%26%2345%3B%26gt%3B99%20--%3E%0A%3Cg%20id%3D%22edge99%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E97-%26gt%3B99%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6255.2678%2C-94.3048C6255.2678%2C-85.9126%206255.2678%2C-76.9946%206255.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226258.7679%2C-68.5186%206255.2678%2C-58.5186%206251.7679%2C-68.5187%206258.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20101%20--%3E%0A%3Cg%20id%3D%22node102%22%20class%3D%22node%22%3E%0A%3Ctitle%3E101%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226427.8046%2C-58.6014%206336.731%2C-58.6014%206336.731%2C.2014%206427.8046%2C.2014%206427.8046%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226382.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.497%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226382.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2013%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226382.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B6%2C%207%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20100%26%2345%3B%26gt%3B101%20--%3E%0A%3Cg%20id%3D%22edge101%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E100-%26gt%3B101%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6423.4333%2C-94.3048C6417.8354%2C-85.4515%206411.8675%2C-76.013%206406.2626%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226409.1084%2C-65.1003%206400.8058%2C-58.5186%206403.1919%2C-68.8413%206409.1084%2C-65.1003%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20102%20--%3E%0A%3Cg%20id%3D%22node103%22%20class%3D%22node%22%3E%0A%3Ctitle%3E102%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226565.8043%2C-58.6014%206446.7313%2C-58.6014%206446.7313%2C.2014%206565.8043%2C.2014%206565.8043%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226506.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.245%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226506.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201086%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226506.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B155%2C%20931%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20100%26%2345%3B%26gt%3B102%20--%3E%0A%3Cg%20id%3D%22edge102%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E100-%26gt%3B102%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6468.9022%2C-94.3048C6473.9304%2C-85.5437%206479.2876%2C-76.2096%206484.3288%2C-67.4259%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226487.4988%2C-68.9339%206489.441%2C-58.5186%206481.4276%2C-65.4495%206487.4988%2C-68.9339%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20104%20--%3E%0A%3Cg%20id%3D%22node105%22%20class%3D%22node%22%3E%0A%3Ctitle%3E104%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226768.8043%2C-169.7003%206649.7313%2C-169.7003%206649.7313%2C-94.2997%206768.8043%2C-94.2997%206768.8043%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226709.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B3%5D%20%26lt%3B%3D%2090.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226709.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.377%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226709.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20643%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226709.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B162%2C%20481%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20103%26%2345%3B%26gt%3B104%20--%3E%0A%3Cg%20id%3D%22edge104%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E103-%26gt%3B104%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6709.2678%2C-205.5528C6709.2678%2C-197.22%206709.2678%2C-188.2845%206709.2678%2C-179.6137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226712.7679%2C-179.6084%206709.2678%2C-169.6085%206705.7679%2C-179.6085%206712.7679%2C-179.6084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20107%20--%3E%0A%3Cg%20id%3D%22node108%22%20class%3D%22node%22%3E%0A%3Ctitle%3E107%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226951.3045%2C-169.7003%206853.2311%2C-169.7003%206853.2311%2C-94.2997%206951.3045%2C-94.2997%206951.3045%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226902.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B2%5D%20%26lt%3B%3D%20165.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226902.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.195%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226902.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2073%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226902.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B8%2C%2065%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20103%26%2345%3B%26gt%3B107%20--%3E%0A%3Cg%20id%3D%22edge107%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E103-%26gt%3B107%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6769.0125%2C-208.7772C6792.965%2C-194.9765%206820.4696%2C-179.1293%206844.2469%2C-165.4297%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226846.2447%2C-168.318%206853.1621%2C-160.293%206842.75%2C-162.2527%206846.2447%2C-168.318%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20105%20--%3E%0A%3Cg%20id%3D%22node106%22%20class%3D%22node%22%3E%0A%3Ctitle%3E105%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226703.8043%2C-58.6014%206584.7313%2C-58.6014%206584.7313%2C.2014%206703.8043%2C.2014%206703.8043%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226644.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.353%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226644.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20489%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226644.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B112%2C%20377%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20104%26%2345%3B%26gt%3B105%20--%3E%0A%3Cg%20id%3D%22edge105%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E104-%26gt%3B105%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6685.4333%2C-94.3048C6679.8354%2C-85.4515%206673.8675%2C-76.013%206668.2626%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226671.1084%2C-65.1003%206662.8058%2C-58.5186%206665.1919%2C-68.8413%206671.1084%2C-65.1003%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20106%20--%3E%0A%3Cg%20id%3D%22node107%22%20class%3D%22node%22%3E%0A%3Ctitle%3E106%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226834.3044%2C-58.6014%206722.2312%2C-58.6014%206722.2312%2C.2014%206834.3044%2C.2014%206834.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226778.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.439%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226778.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20154%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226778.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B50%2C%20104%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20104%26%2345%3B%26gt%3B106%20--%3E%0A%3Cg%20id%3D%22edge106%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E104-%26gt%3B106%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6734.5691%2C-94.3048C6740.5733%2C-85.3593%206746.9786%2C-75.8163%206752.9822%2C-66.8718%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226755.9219%2C-68.7723%206758.589%2C-58.5186%206750.1098%2C-64.8711%206755.9219%2C-68.7723%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20108%20--%3E%0A%3Cg%20id%3D%22node109%22%20class%3D%22node%22%3E%0A%3Ctitle%3E108%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%226951.3045%2C-58.6014%206853.2311%2C-58.6014%206853.2311%2C.2014%206951.3045%2C.2014%206951.3045%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226902.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.087%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226902.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2044%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%226902.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B2%2C%2042%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20107%26%2345%3B%26gt%3B108%20--%3E%0A%3Cg%20id%3D%22edge108%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E107-%26gt%3B108%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6902.2678%2C-94.3048C6902.2678%2C-85.9126%206902.2678%2C-76.9946%206902.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226905.7679%2C-68.5186%206902.2678%2C-58.5186%206898.7679%2C-68.5187%206905.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20109%20--%3E%0A%3Cg%20id%3D%22node110%22%20class%3D%22node%22%3E%0A%3Ctitle%3E109%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227068.3045%2C-58.6014%206970.2311%2C-58.6014%206970.2311%2C.2014%207068.3045%2C.2014%207068.3045%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227019.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.328%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227019.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2029%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227019.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B6%2C%2023%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20107%26%2345%3B%26gt%3B109%20--%3E%0A%3Cg%20id%3D%22edge109%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E107-%26gt%3B109%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M6945.1699%2C-94.3048C6955.9809%2C-84.806%206967.5585%2C-74.6335%206978.2732%2C-65.2192%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%226980.6972%2C-67.7484%206985.8993%2C-58.5186%206976.0769%2C-62.4899%206980.6972%2C-67.7484%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20111%20--%3E%0A%3Cg%20id%3D%22node112%22%20class%3D%22node%22%3E%0A%3Ctitle%3E111%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227359.8044%2C-280.9003%207254.7312%2C-280.9003%207254.7312%2C-205.4997%207359.8044%2C-205.4997%207359.8044%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227307.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B4%5D%20%26lt%3B%3D%20249.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227307.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.483%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227307.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2066%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227307.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B27%2C%2039%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20110%26%2345%3B%26gt%3B111%20--%3E%0A%3Cg%20id%3D%22edge111%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E110-%26gt%3B111%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7307.2678%2C-316.7528C7307.2678%2C-308.42%207307.2678%2C-299.4845%207307.2678%2C-290.8137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227310.7679%2C-290.8084%207307.2678%2C-280.8085%207303.7679%2C-290.8085%207310.7679%2C-290.8084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20118%20--%3E%0A%3Cg%20id%3D%22node119%22%20class%3D%22node%22%3E%0A%3Ctitle%3E118%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227719.3042%2C-280.9003%207593.2314%2C-280.9003%207593.2314%2C-205.4997%207719.3042%2C-205.4997%207719.3042%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227656.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B7%5D%20%26lt%3B%3D%202.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227656.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.231%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227656.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%206607%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227656.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B882%2C%205725%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20110%26%2345%3B%26gt%3B118%20--%3E%0A%3Cg%20id%3D%22edge118%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E110-%26gt%3B118%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7370.4691%2C-334.2625C7430.3212%2C-315.1922%207520.1547%2C-286.569%207583.0874%2C-266.5171%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227584.4825%2C-269.746%207592.948%2C-263.3752%207582.3574%2C-263.0764%207584.4825%2C-269.746%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20112%20--%3E%0A%3Cg%20id%3D%22node113%22%20class%3D%22node%22%3E%0A%3Ctitle%3E112%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227301.8044%2C-169.7003%207196.7312%2C-169.7003%207196.7312%2C-94.2997%207301.8044%2C-94.2997%207301.8044%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227249.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B3%5D%20%26lt%3B%3D%2058.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227249.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.436%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227249.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2056%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227249.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B18%2C%2038%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20111%26%2345%3B%26gt%3B112%20--%3E%0A%3Cg%20id%3D%22edge112%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E111-%26gt%3B112%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7287.6316%2C-205.5528C7283.0964%2C-196.8577%207278.219%2C-187.5064%207273.5132%2C-178.4843%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227276.6116%2C-176.8563%207268.8837%2C-169.6085%207270.4051%2C-180.0935%207276.6116%2C-176.8563%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20115%20--%3E%0A%3Cg%20id%3D%22node116%22%20class%3D%22node%22%3E%0A%3Ctitle%3E115%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227411.8046%2C-169.7003%207320.731%2C-169.7003%207320.731%2C-94.2997%207411.8046%2C-94.2997%207411.8046%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227366.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B3%5D%20%26lt%3B%3D%2063.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227366.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.18%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227366.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2010%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227366.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B9%2C%201%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20111%26%2345%3B%26gt%3B115%20--%3E%0A%3Cg%20id%3D%22edge115%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E111-%26gt%3B115%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7327.2425%2C-205.5528C7331.8559%2C-196.8577%207336.8175%2C-187.5064%207341.6044%2C-178.4843%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227344.7185%2C-180.0825%207346.3137%2C-169.6085%207338.5349%2C-176.8017%207344.7185%2C-180.0825%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20113%20--%3E%0A%3Cg%20id%3D%22node114%22%20class%3D%22node%22%3E%0A%3Ctitle%3E113%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227177.8046%2C-58.6014%207086.731%2C-58.6014%207086.731%2C.2014%207177.8046%2C.2014%207177.8046%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227132.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.0%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227132.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%208%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227132.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B0%2C%208%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20112%26%2345%3B%26gt%3B113%20--%3E%0A%3Cg%20id%3D%22edge113%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E112-%26gt%3B113%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7206.3657%2C-94.3048C7195.5547%2C-84.806%207183.9771%2C-74.6335%207173.2624%2C-65.2192%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227175.4587%2C-62.4899%207165.6363%2C-58.5186%207170.8384%2C-67.7484%207175.4587%2C-62.4899%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20114%20--%3E%0A%3Cg%20id%3D%22node115%22%20class%3D%22node%22%3E%0A%3Ctitle%3E114%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227301.8044%2C-58.6014%207196.7312%2C-58.6014%207196.7312%2C.2014%207301.8044%2C.2014%207301.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227249.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.469%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227249.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2048%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227249.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B18%2C%2030%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20112%26%2345%3B%26gt%3B114%20--%3E%0A%3Cg%20id%3D%22edge114%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E112-%26gt%3B114%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7249.2678%2C-94.3048C7249.2678%2C-85.9126%207249.2678%2C-76.9946%207249.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227252.7679%2C-68.5186%207249.2678%2C-58.5186%207245.7679%2C-68.5187%207252.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20116%20--%3E%0A%3Cg%20id%3D%22node117%22%20class%3D%22node%22%3E%0A%3Ctitle%3E116%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227411.8046%2C-58.6014%207320.731%2C-58.6014%207320.731%2C.2014%207411.8046%2C.2014%207411.8046%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227366.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.0%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227366.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%207%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227366.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B7%2C%200%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20115%26%2345%3B%26gt%3B116%20--%3E%0A%3Cg%20id%3D%22edge116%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E115-%26gt%3B116%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7366.2678%2C-94.3048C7366.2678%2C-85.9126%207366.2678%2C-76.9946%207366.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227369.7679%2C-68.5186%207366.2678%2C-58.5186%207362.7679%2C-68.5187%207369.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20117%20--%3E%0A%3Cg%20id%3D%22node118%22%20class%3D%22node%22%3E%0A%3Ctitle%3E117%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227521.8046%2C-58.6014%207430.731%2C-58.6014%207430.731%2C.2014%207521.8046%2C.2014%207521.8046%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227476.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.444%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227476.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%203%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227476.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B2%2C%201%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20115%26%2345%3B%26gt%3B117%20--%3E%0A%3Cg%20id%3D%22edge117%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E115-%26gt%3B117%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7406.6031%2C-94.3048C7416.6686%2C-84.8982%207427.4409%2C-74.8309%207437.4322%2C-65.4936%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227439.9793%2C-67.9037%207444.8957%2C-58.5186%207435.1998%2C-62.7894%207439.9793%2C-67.9037%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20119%20--%3E%0A%3Cg%20id%3D%22node120%22%20class%3D%22node%22%3E%0A%3Ctitle%3E119%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227719.3042%2C-169.7003%207593.2314%2C-169.7003%207593.2314%2C-94.2997%207719.3042%2C-94.2997%207719.3042%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227656.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B4%5D%20%26lt%3B%3D%2012255.0%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227656.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.225%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227656.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%206014%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227656.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B778%2C%205236%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20118%26%2345%3B%26gt%3B119%20--%3E%0A%3Cg%20id%3D%22edge119%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E118-%26gt%3B119%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7656.2678%2C-205.5528C7656.2678%2C-197.22%207656.2678%2C-188.2845%207656.2678%2C-179.6137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227659.7679%2C-179.6084%207656.2678%2C-169.6085%207652.7679%2C-179.6085%207659.7679%2C-179.6084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20122%20--%3E%0A%3Cg%20id%3D%22node123%22%20class%3D%22node%22%3E%0A%3Ctitle%3E122%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227909.8043%2C-169.7003%207790.7313%2C-169.7003%207790.7313%2C-94.2997%207909.8043%2C-94.2997%207909.8043%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227850.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EX%5B2%5D%20%26lt%3B%3D%20166.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227850.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.289%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227850.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20593%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227850.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B104%2C%20489%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20118%26%2345%3B%26gt%3B122%20--%3E%0A%3Cg%20id%3D%22edge122%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E118-%26gt%3B122%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7719.5222%2C-206.9428C7739.4602%2C-195.5145%207761.5712%2C-182.8405%207781.8516%2C-171.2159%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227783.7865%2C-174.141%207790.7218%2C-166.1315%207780.3054%2C-168.068%207783.7865%2C-174.141%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20120%20--%3E%0A%3Cg%20id%3D%22node121%22%20class%3D%22node%22%3E%0A%3Ctitle%3E120%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227666.3042%2C-58.6014%207540.2314%2C-58.6014%207540.2314%2C.2014%207666.3042%2C.2014%207666.3042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227603.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.225%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227603.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%206011%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227603.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B776%2C%205235%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20119%26%2345%3B%26gt%3B120%20--%3E%0A%3Cg%20id%3D%22edge120%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E119-%26gt%3B120%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7636.8335%2C-94.3048C7632.3166%2C-85.5437%207627.5043%2C-76.2096%207622.9757%2C-67.4259%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227626.0768%2C-65.803%207618.3834%2C-58.5186%207619.8551%2C-69.0108%207626.0768%2C-65.803%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20121%20--%3E%0A%3Cg%20id%3D%22node122%22%20class%3D%22node%22%3E%0A%3Ctitle%3E121%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227775.8046%2C-58.6014%207684.731%2C-58.6014%207684.731%2C.2014%207775.8046%2C.2014%207775.8046%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227730.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.444%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227730.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%203%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227730.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B2%2C%201%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20119%26%2345%3B%26gt%3B121%20--%3E%0A%3Cg%20id%3D%22edge121%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E119-%26gt%3B121%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7683.4025%2C-94.3048C7689.8419%2C-85.3593%207696.7113%2C-75.8163%207703.1499%2C-66.8718%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227706.1613%2C-68.6794%207709.1629%2C-58.5186%207700.4801%2C-64.5898%207706.1613%2C-68.6794%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20123%20--%3E%0A%3Cg%20id%3D%22node124%22%20class%3D%22node%22%3E%0A%3Ctitle%3E123%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%227906.3044%2C-58.6014%207794.2312%2C-58.6014%207794.2312%2C.2014%207906.3044%2C.2014%207906.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227850.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.242%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227850.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20390%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227850.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B55%2C%20335%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20122%26%2345%3B%26gt%3B123%20--%3E%0A%3Cg%20id%3D%22edge123%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E122-%26gt%3B123%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7850.2678%2C-94.3048C7850.2678%2C-85.9126%207850.2678%2C-76.9946%207850.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227853.7679%2C-68.5186%207850.2678%2C-58.5186%207846.7679%2C-68.5187%207853.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20124%20--%3E%0A%3Cg%20id%3D%22node125%22%20class%3D%22node%22%3E%0A%3Ctitle%3E124%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%228037.3044%2C-58.6014%207925.2312%2C-58.6014%207925.2312%2C.2014%208037.3044%2C.2014%208037.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227981.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.366%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227981.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20203%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%227981.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B49%2C%20154%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20122%26%2345%3B%26gt%3B124%20--%3E%0A%3Cg%20id%3D%22edge124%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E122-%26gt%3B124%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M7898.3035%2C-94.3048C7910.5256%2C-84.7137%207923.6229%2C-74.4359%207935.717%2C-64.9452%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%227938.2003%2C-67.4455%207943.9065%2C-58.5186%207933.8788%2C-61.9387%207938.2003%2C-67.4455%22%2F%3E%0A%3C%2Fg%3E%0A%3C%2Fg%3E%0A%3C%2Fsvg%3E**)

## Построение случайного леса

1. Обучите случайный лес из 100 деревьев с параметрами по умолчанию и **random_state=2019**.
2. Постройте предсказания, вычислите долю правильных ответов и сравните с метриками, полученными ранее.
3. Подберите гиперпараметры случайного леса на своё усмотрение. На сколько процентов удалось улучшить предыдущий результат?

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=2019, max_depth=6)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_valid)

accuracy_score(y_valid, y_pred)

In [ ]:
# нет ограничения на глубину дерева
rf = RandomForestClassifier(n_estimators=100, random_state=2019)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_valid)

accuracy_score(y_valid, y_pred)

In [ ]:
# кросс-валидация по гиперпараметру (количество деревьев в лесу)
rf_params_n_estimators = {'n_estimators': np.arange(100, 161, 10)}
rf_n_estimators = RandomForestClassifier(random_state=22)
rf_grid_n_estimators = GridSearchCV(rf_n_estimators, rf_params_n_estimators, cv=5, scoring='accuracy')
rf_grid_n_estimators.fit(X_train, y_train)

print(rf_grid_n_estimators.best_score_)
print(rf_grid_n_estimators.best_params_)
print(rf_grid_n_estimators.best_estimator_)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

ax.plot(rf_params_n_estimators['n_estimators'], rf_grid_n_estimators.cv_results_['mean_test_score']) 
ax.set_xlabel('n_estimators')
ax.set_ylabel('Mean accuracy on test set')

Теперь подбираем max_depth. Пробуем значения от 2 до 10. Устанавливаем предыдущий параметр n_estimators = 160.

In [ ]:
rf_params_max_depth = {'max_depth': np.arange(2, 11)}
rf_max_depth = RandomForestClassifier(n_estimators=160, random_state=22)
rf_grid_max_depth = GridSearchCV(rf_max_depth, rf_params_max_depth, cv=5, scoring='accuracy') 
rf_grid_max_depth.fit(X_train, y_train)

print(rf_grid_max_depth.best_score_)
print(rf_grid_max_depth.best_params_)
print(rf_grid_max_depth.best_estimator_)

In [ ]:
fig, ax = plt.subplots() 

ax.plot(rf_params_max_depth['max_depth'], rf_grid_max_depth.cv_results_['mean_test_score']) # accuracy vs max_depth
ax.set_xlabel('max_depth')
ax.set_ylabel('Mean accuracy on test set')

## Важность признаков

Выясним, какой фактор является наиболее значимым при выявлении сердечно-сосудистых заболеваний.

Вычисление важности признаков при помощи дерева решений

In [ ]:
feature = pd.Series(tree.feature_importances_, index=X_train.columns).sort_values(ascending=False)
feature

In [ ]:
# дерево после подбора гиперпараметров
feature = pd.Series(best_tree.feature_importances_, index=X_train.columns).sort_values(ascending=False)
feature

Вычисление степени важности признаков при помощи случайных лесов

In [ ]:
feature = pd.Series(rf.feature_importances_, index=X_train.columns).sort_values(ascending=False)
feature